In [ ]:
import csv
from astropy.io import fits
import numpy as np
import sys
import math
import re
import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib.patches as patch
import random
from scipy.stats import gaussian_kde
from scipy.interpolate import interpn
import matplotlib.cm as cm

In [ ]:
def vlm_object_probability(file,model,output_name):
    
    '''This is the main program used to find the low-mass objects. It takes as input a file of sources with ZYJHK
    magnitues, and a model with 1 and 5 Myr lines. It is pretty specific to the ones I used, so I would suggest
    starting with those if you are going to make modifications. It takes the input data, makes a poly-fit error,
    and applies that to the model, along with Av for the extincted side. Then it uses those borders to flag sources
    as being inside. Then it sums all those flags, and puts the results into columns, which are then aligned into
    a single fits file. This verison returns a full list of all objects.'''
    
    RA_dms = []
    Dec_dms = []
    RA_Rad = []
    Dec_Rad = []
    RA_ICRS = []
    Dec_ICRS = []
    zmag = []
    ymag = []
    jmag = []
    hmag = []
    kmag = []
    zobsmag = []
    yobsmag = []
    jobsmag = []
    hobsmag = []
    kobsmag = []
    zaper = []
    yaper = []
    japer = []
    haper = []
    kaper = []
    zapererr = []
    yapererr = []
    japererr = []
    hapererr = []
    kapererr = []
    
    intest = [[[[[0 for z in range(5)] for y in range(4)] for x in range(2)] for w in range(2)] for v in range(2)]
    
    hdul = fits.open(file)
    allpoints = list(range(len(hdul[1].data)))
    
    Filters = [0 for z in allpoints]
    YND = []
    YNC = []
    YFD = []
    YFC = []
    OND = []
    ONC = []
    OFD = []
    OFC = []
    ZY = []
    ZJ = []
    ZH = []
    ZK = []
    YJ = []
    YH = []
    YK = []
    JH = []
    JK = []
    HK = []
    Max = [0 for z in allpoints]
    Total = [0 for z in allpoints]
    Percentage = [0 for z in allpoints]
    Y = [0 for z in allpoints]
    O = [0 for z in allpoints]
    N = [0 for z in allpoints]
    F = [0 for z in allpoints]
    D = [0 for z in allpoints]
    C = [0 for z in allpoints]
    YNDperc = [0 for z in allpoints]
    YNCperc = [0 for z in allpoints]
    YFDperc = [0 for z in allpoints]
    YFCperc = [0 for z in allpoints]
    ONDperc = [0 for z in allpoints]
    ONCperc = [0 for z in allpoints]
    OFDperc = [0 for z in allpoints]
    OFCperc = [0 for z in allpoints]
    Max_Perc = [0 for z in allpoints]

    ra_dms = hdul[1].data['RA_dms']
    dec_dms = hdul[1].data['Dec_dms']
    ra_rad = hdul[1].data['RA_Rad']
    dec_rad = hdul[1].data['Dec_Rad']
    ra_icrs = hdul[1].data['RA_ICRS']
    dec_icrs = hdul[1].data['Dec_ICRS']
    mag_z = hdul[1].data['Mag_Z']
    obs_mag_z = hdul[1].data['Obs_Mag_Z']
    aper_z = hdul[1].data['Aper_Z']
    aper_z_err = hdul[1].data['Aper_Z_err']
    mag_y = hdul[1].data['Mag_Y']
    obs_mag_y = hdul[1].data['Obs_Mag_Y']
    aper_y = hdul[1].data['Aper_Y']
    aper_y_err = hdul[1].data['Aper_Y_err']
    mag_j = hdul[1].data['Mag_J']
    obs_mag_j = hdul[1].data['Obs_Mag_J']
    aper_j = hdul[1].data['Aper_J']
    aper_j_err = hdul[1].data['Aper_J_err']
    mag_h = hdul[1].data['Mag_H']
    obs_mag_h = hdul[1].data['Obs_Mag_H']
    aper_h = hdul[1].data['Aper_H']
    aper_h_err = hdul[1].data['Aper_H_err']
    mag_k = hdul[1].data['Mag_K']
    obs_mag_k = hdul[1].data['Obs_Mag_K']
    aper_k = hdul[1].data['Aper_K']
    aper_k_err = hdul[1].data['Aper_K_err']
    
    err_z = aper_z_err/(aper_z)
    err_y = aper_y_err/(aper_y)
    err_j = aper_j_err/(aper_j)
    err_h = aper_h_err/(aper_h)
    err_k = aper_k_err/(aper_k)
    
    model = fits.open('model_iso.fits')

    mass1 = model[1].data['Mass'][0:28]
    teff1 = model[1].data['Teff'][0:28]
    zmd1 = model[1].data['ZmagD'][0:28]
    ymd1 = model[1].data['YmagD'][0:28]
    jmd1 = model[1].data['JmagD'][0:28]
    hmd1 = model[1].data['HmagD'][0:28]
    kmd1 = model[1].data['KsmagD'][0:28]
    zmc1 = model[1].data['ZmagC'][0:28]
    ymc1 = model[1].data['YmagC'][0:28]
    jmc1 = model[1].data['JmagC'][0:28]
    hmc1 = model[1].data['HmagC'][0:28]
    kmc1 = model[1].data['KsmagC'][0:28]
    
    mass5 = model[1].data['Mass'][28:57]
    teff5 = model[1].data['Teff'][28:57]
    zmd5 = model[1].data['ZmagD'][28:57]
    ymd5 = model[1].data['YmagD'][28:57]
    jmd5 = model[1].data['JmagD'][28:57]
    hmd5 = model[1].data['HmagD'][28:57]
    kmd5 = model[1].data['KsmagD'][28:57]
    zmc5 = model[1].data['ZmagC'][28:57]
    ymc5 = model[1].data['YmagC'][28:57]
    jmc5 = model[1].data['JmagC'][28:57]
    hmc5 = model[1].data['HmagC'][28:57]
    kmc5 = model[1].data['KsmagC'][28:57]

    massy_fit = np.polyfit(teff1, mass1, 14)
    massy_poly = np.poly1d(massy_fit)
    zcy_fit = np.polyfit(teff1, zmc1, 14)
    zcy_poly = np.poly1d(zcy_fit)
    zdy_fit = np.polyfit(teff1, zmd1, 14)
    zdy_poly = np.poly1d(zdy_fit)
    ycy_fit = np.polyfit(teff1, ymc1, 14)
    ycy_poly = np.poly1d(ycy_fit)
    ydy_fit = np.polyfit(teff1, ymd1, 14)
    ydy_poly = np.poly1d(ydy_fit)
    jcy_fit = np.polyfit(teff1, jmc1, 14)
    jcy_poly = np.poly1d(jcy_fit)
    jdy_fit = np.polyfit(teff1, jmd1, 14)
    jdy_poly = np.poly1d(jdy_fit)
    hcy_fit = np.polyfit(teff1, hmc1, 14)
    hcy_poly = np.poly1d(hcy_fit)
    hdy_fit = np.polyfit(teff1, hmd1, 14)
    hdy_poly = np.poly1d(hdy_fit)
    kcy_fit = np.polyfit(teff1, kmc1, 14)
    kcy_poly = np.poly1d(kcy_fit)
    kdy_fit = np.polyfit(teff1, kmd1, 14)
    kdy_poly = np.poly1d(kdy_fit)
    
    masso_fit = np.polyfit(teff5, mass5, 14)
    masso_poly = np.poly1d(masso_fit)
    zco_fit = np.polyfit(teff5, zmc5, 14)
    zco_poly = np.poly1d(zco_fit)
    zdo_fit = np.polyfit(teff5, zmd5, 14)
    zdo_poly = np.poly1d(zdo_fit)
    yco_fit = np.polyfit(teff5, ymc5, 14)
    yco_poly = np.poly1d(yco_fit)
    ydo_fit = np.polyfit(teff5, ymd5, 14)
    ydo_poly = np.poly1d(ydo_fit)
    jco_fit = np.polyfit(teff5, jmc5, 14)
    jco_poly = np.poly1d(jco_fit)
    jdo_fit = np.polyfit(teff5, jmd5, 14)
    jdo_poly = np.poly1d(jdo_fit)
    hco_fit = np.polyfit(teff5, hmc5, 14)
    hco_poly = np.poly1d(hco_fit)
    hdo_fit = np.polyfit(teff5, hmd5, 14)
    hdo_poly = np.poly1d(hdo_fit)
    kco_fit = np.polyfit(teff5, kmc5, 14)
    kco_poly = np.poly1d(kco_fit)
    kdo_fit = np.polyfit(teff5, kmd5, 14)
    kdo_poly = np.poly1d(kdo_fit)
    
    idz = np.isfinite(mag_z) & np.isfinite(err_z)
    zerr_fit = np.polyfit(mag_z[idz],err_z[idz],8)
    zerr_poly = np.poly1d(zerr_fit)
    idy = np.isfinite(mag_y) & np.isfinite(err_y)
    yerr_fit = np.polyfit(mag_y[idy],err_y[idy],8)
    yerr_poly = np.poly1d(yerr_fit)
    idj = np.isfinite(mag_j) & np.isfinite(err_j)
    jerr_fit = np.polyfit(mag_j[idj],err_j[idj],8)
    jerr_poly = np.poly1d(jerr_fit)
    idh = np.isfinite(mag_h) & np.isfinite(err_h)
    herr_fit = np.polyfit(mag_h[idh],err_h[idh],8)
    herr_poly = np.poly1d(herr_fit)
    idk = np.isfinite(mag_k) & np.isfinite(err_k)
    kerr_fit = np.polyfit(mag_k[idk],err_k[idk],8)
    kerr_poly = np.poly1d(kerr_fit)
    
    bd_range_1myr = range(1300,2931)
    full_rangey = range(1300,4000)
    full_rangeo = range(1300,4000)
    full_rangec = range(1300,4000)
    full_ranged = range(1650,4000)
    
    mu_far = Distance(420)
    mu_near = Distance(350)
    Av = 1.1
    
    Az = 0.5006359319 * Av
    Ay = 0.3913227425 * Av
    Aj = 0.281344478 * Av
    Ah = 0.1812818241 * Av
    Ak = 0.1180678823 * Av
    
    Azy = Az - Ay
    Azj = Az - Aj
    Azh = Az - Ah
    Azk = Az - Ak
    Ayj = Ay - Aj
    Ayh = Ay - Ah
    Ayk = Ay - Ak
    Ajh = Aj - Ah
    Ajk = Aj - Ak
    Ahk = Ah - Ak
    
    poly = [[[zdy_poly(full_ranged),zdo_poly(full_ranged)],[zcy_poly(full_rangec),zco_poly(full_rangec)]],
            [[ydy_poly(full_ranged),ydo_poly(full_ranged)],[ycy_poly(full_rangec),yco_poly(full_rangec)]],
            [[jdy_poly(full_ranged),jdo_poly(full_ranged)],[jcy_poly(full_rangec),jco_poly(full_rangec)]],
            [[hdy_poly(full_ranged),hdo_poly(full_ranged)],[hcy_poly(full_rangec),hco_poly(full_rangec)]],
            [[kdy_poly(full_ranged),kdo_poly(full_ranged)],[kcy_poly(full_rangec),kco_poly(full_rangec)]]]
    
    observed_data = [mag_z,mag_y,mag_j,mag_h,mag_k]
    observed_array = np.array(observed_data)

    ext = [[Az,Ay,Aj,Ah,Ak],[[0,Azy,Azj,Azh,Azk],[0,0,Ayj,Ayh,Ayk],[0,0,0,Ajh,Ajk],[0,0,0,0,Ahk]]]
    
    mu = [mu_near,mu_far]
    
    for alpha in range(2):
        for beta in range(2):
            for gamma in range(2):
                for delta in range(4):
                    for epsilon in range(delta+1,5):
                        
                        err_poly_max = [zerr_poly(poly[delta][gamma][beta] + mu[alpha] + 1*ext[0][delta]),
                                    yerr_poly(poly[delta][gamma][beta] + mu[alpha] + 1*ext[0][delta]),
                                    jerr_poly(poly[delta][gamma][beta] + mu[alpha] + 1*ext[0][delta]),
                                    herr_poly(poly[delta][gamma][beta] + mu[alpha] + 1*ext[0][delta]),
                                    kerr_poly(poly[delta][gamma][beta] + mu[alpha] + 1*ext[0][delta])]
                        
                        err_poly_max_ep = [zerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 1*ext[0][epsilon]),
                                    yerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 1*ext[0][epsilon]),
                                    jerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 1*ext[0][epsilon]),
                                    herr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 1*ext[0][epsilon]),
                                    kerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 1*ext[0][epsilon])]
                        
                        err_poly_no = [zerr_poly(poly[delta][gamma][beta] + mu[alpha] + 0*ext[0][delta]),
                                    yerr_poly(poly[delta][gamma][beta] + mu[alpha] + 0*ext[0][delta]),
                                    jerr_poly(poly[delta][gamma][beta] + mu[alpha] + 0*ext[0][delta]),
                                    herr_poly(poly[delta][gamma][beta] + mu[alpha] + 0*ext[0][delta]),
                                    kerr_poly(poly[delta][gamma][beta] + mu[alpha] + 0*ext[0][delta])]
                        
                        err_poly_no_ep = [zerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 0*ext[0][epsilon]),
                                    yerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 0*ext[0][epsilon]),
                                    jerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 0*ext[0][epsilon]),
                                    herr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 0*ext[0][epsilon]),
                                    kerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 0*ext[0][epsilon])]
                            
                        maxext = list(zip((poly[delta][gamma][beta] - 
                                           poly[epsilon][gamma][beta] + 
                                           (1*ext[1][delta][epsilon])) + 
                                          np.sqrt(err_poly_max[delta]**2 + err_poly_max_ep[epsilon]**2),
                                          poly[delta][gamma][beta] + mu[alpha] + 
                                          (1*ext[0][delta]) - err_poly_max[delta]))
                        noext = list(zip((poly[delta][gamma][beta] - 
                                          poly[epsilon][gamma][beta] + 
                                          (0*ext[1][delta][epsilon])) - 
                                         np.sqrt(err_poly_no[delta]**2 + err_poly_no_ep[epsilon]**2),
                                         poly[delta][gamma][beta] + mu[alpha] + 
                                         (0*ext[0][delta]) + err_poly_no[delta]))
                        
                        sidepoints = maxext + noext[::-1]
                        polypath = path.Path(sidepoints, closed=True)
                        
                        points = list(zip(observed_data[delta]-observed_data[epsilon],observed_data[delta]))
                        intest[beta][alpha][gamma][delta][epsilon] = polypath.contains_points(points)
    
    for i in allpoints:
        Filters[i] = np.sum(~np.isnan(observed_array[:,i]))
        
        outynd = []
        outync = []
        outyfd = []
        outyfc = []
        outond = []
        outonc = []
        outofd = []
        outofc = []

        for d in range(4):
            for e in range(d+1,5):
                outynd.append(intest[0][0][0][d][e][i])
                outync.append(intest[0][0][1][d][e][i])
                outyfd.append(intest[0][1][0][d][e][i])
                outyfc.append(intest[0][1][1][d][e][i])
                outond.append(intest[1][0][0][d][e][i])
                outonc.append(intest[1][0][1][d][e][i])
                outofd.append(intest[1][1][0][d][e][i])
                outofc.append(intest[1][1][1][d][e][i])
        YND.append(np.sum(outynd))
        YNC.append(np.sum(outync))
        YFD.append(np.sum(outyfd))
        YFC.append(np.sum(outyfc))
        OND.append(np.sum(outond))
        ONC.append(np.sum(outonc))
        OFD.append(np.sum(outofd))
        OFC.append(np.sum(outofc))

        outzy = []
        outzj = []
        outzh = []
        outzk = []
        outyj = []
        outyh = []
        outyk = []
        outjh = []
        outjk = []
        outhk = []

        for a in range(2):
            for b in range(2):
                for c in range(2):
                    outzy.append(intest[a][b][c][0][1][i])
                    outzj.append(intest[a][b][c][0][2][i])
                    outzh.append(intest[a][b][c][0][3][i])
                    outzk.append(intest[a][b][c][0][4][i])
                    outyj.append(intest[a][b][c][1][2][i])
                    outyh.append(intest[a][b][c][1][3][i])
                    outyk.append(intest[a][b][c][1][4][i])
                    outjh.append(intest[a][b][c][2][3][i])
                    outjk.append(intest[a][b][c][2][4][i])
                    outhk.append(intest[a][b][c][3][4][i])
        ZY.append(np.sum(outzy))
        ZJ.append(np.sum(outzj))
        ZH.append(np.sum(outzh))
        ZK.append(np.sum(outzk))
        YJ.append(np.sum(outyj))
        YH.append(np.sum(outyh))
        YK.append(np.sum(outyk))
        JH.append(np.sum(outjh))
        JK.append(np.sum(outjk))
        HK.append(np.sum(outhk))
        
        if Filters[i] == 2:
            Max[i] = 1*8
        if Filters[i] == 3:
            Max[i] = 3*8
        if Filters[i] == 4:
            Max[i] = 6*8
        if Filters[i] == 5:
            Max[i] = 10*8
           
        Y[i] = np.sum(YND[i] + YNC[i] + YFD[i] + YFC[i])
        O[i] = np.sum(OND[i] + ONC[i] + OFD[i] + OFC[i])
        N[i] = np.sum(YND[i] + YNC[i] + OND[i] + ONC[i])
        F[i] = np.sum(YFD[i] + YFC[i] + OFD[i] + OFC[i])
        D[i] = np.sum(YND[i] + OND[i] + YFD[i] + OFD[i])
        C[i] = np.sum(YNC[i] + ONC[i] + YFC[i] + OFC[i])
    
        Total[i] = sum([YND[i],YNC[i],YFD[i],YFC[i],OND[i],ONC[i],OFD[i],OFC[i]])
        YNDperc[i] = YND[i]/(Max[i]/8)
        YNCperc[i] = YNC[i]/(Max[i]/8)
        YFDperc[i] = YFD[i]/(Max[i]/8)
        YFCperc[i] = YFC[i]/(Max[i]/8)
        ONDperc[i] = OND[i]/(Max[i]/8)
        ONCperc[i] = ONC[i]/(Max[i]/8)
        OFDperc[i] = OFD[i]/(Max[i]/8)
        OFCperc[i] = OFC[i]/(Max[i]/8)
        Percentage[i] = Total[i]/Max[i]
        Max_Perc[i] = max(YNDperc[i],YNCperc[i],YFDperc[i],YFCperc[i],
                          ONDperc[i],ONCperc[i],OFDperc[i],OFCperc[i],
                          Percentage[i])
    
    YND_zy = intest[0][0][0][0][1]
    YND_zj = intest[0][0][0][0][2]
    YND_zh = intest[0][0][0][0][3]
    YND_zk = intest[0][0][0][0][4]
    YND_yj = intest[0][0][0][1][2]
    YND_yh = intest[0][0][0][1][3]
    YND_yk = intest[0][0][0][1][4]
    YND_jh = intest[0][0][0][2][3]
    YND_jk = intest[0][0][0][2][4]
    YND_hk = intest[0][0][0][3][4]
    YNC_zy = intest[0][0][1][0][1]
    YNC_zj = intest[0][0][1][0][2]
    YNC_zh = intest[0][0][1][0][3]
    YNC_zk = intest[0][0][1][0][4]
    YNC_yj = intest[0][0][1][1][2]
    YNC_yh = intest[0][0][1][1][3]
    YNC_yk = intest[0][0][1][1][4]
    YNC_jh = intest[0][0][1][2][3]
    YNC_jk = intest[0][0][1][2][4]
    YNC_hk = intest[0][0][1][3][4]
    YFD_zy = intest[0][1][0][0][1]
    YFD_zj = intest[0][1][0][0][2]
    YFD_zh = intest[0][1][0][0][3]
    YFD_zk = intest[0][1][0][0][4]
    YFD_yj = intest[0][1][0][1][2]
    YFD_yh = intest[0][1][0][1][3]
    YFD_yk = intest[0][1][0][1][4]
    YFD_jh = intest[0][1][0][2][3]
    YFD_jk = intest[0][1][0][2][4]
    YFD_hk = intest[0][1][0][3][4]
    YFC_zy = intest[0][1][1][0][1]
    YFC_zj = intest[0][1][1][0][2]
    YFC_zh = intest[0][1][1][0][3]
    YFC_zk = intest[0][1][1][0][4]
    YFC_yj = intest[0][1][1][1][2]
    YFC_yh = intest[0][1][1][1][3]
    YFC_yk = intest[0][1][1][1][4]
    YFC_jh = intest[0][1][1][2][3]
    YFC_jk = intest[0][1][1][2][4]
    YFC_hk = intest[0][1][1][3][4]
    OND_zy = intest[1][0][0][0][1]
    OND_zj = intest[1][0][0][0][2]
    OND_zh = intest[1][0][0][0][3]
    OND_zk = intest[1][0][0][0][4]
    OND_yj = intest[1][0][0][1][2]
    OND_yh = intest[1][0][0][1][3]
    OND_yk = intest[1][0][0][1][4]
    OND_jh = intest[1][0][0][2][3]
    OND_jk = intest[1][0][0][2][4]
    OND_hk = intest[1][0][0][3][4]
    ONC_zy = intest[1][0][1][0][1]
    ONC_zj = intest[1][0][1][0][2]
    ONC_zh = intest[1][0][1][0][3]
    ONC_zk = intest[1][0][1][0][4]
    ONC_yj = intest[1][0][1][1][2]
    ONC_yh = intest[1][0][1][1][3]
    ONC_yk = intest[1][0][1][1][4]
    ONC_jh = intest[1][0][1][2][3]
    ONC_jk = intest[1][0][1][2][4]
    ONC_hk = intest[1][0][1][3][4]
    OFD_zy = intest[1][1][0][0][1]
    OFD_zj = intest[1][1][0][0][2]
    OFD_zh = intest[1][1][0][0][3]
    OFD_zk = intest[1][1][0][0][4]
    OFD_yj = intest[1][1][0][1][2]
    OFD_yh = intest[1][1][0][1][3]
    OFD_yk = intest[1][1][0][1][4]
    OFD_jh = intest[1][1][0][2][3]
    OFD_jk = intest[1][1][0][2][4]
    OFD_hk = intest[1][1][0][3][4]
    OFC_zy = intest[1][1][1][0][1]
    OFC_zj = intest[1][1][1][0][2]
    OFC_zh = intest[1][1][1][0][3]
    OFC_zk = intest[1][1][1][0][4]
    OFC_yj = intest[1][1][1][1][2]
    OFC_yh = intest[1][1][1][1][3]
    OFC_yk = intest[1][1][1][1][4]
    OFC_jh = intest[1][1][1][2][3]
    OFC_jk = intest[1][1][1][2][4]
    OFC_hk = intest[1][1][1][3][4]
    
    output_fits = output_name + '.fits'
    output_reg = output_name + '.reg'
    
    for j in allpoints:
        RA_dms.append(hdul[1].data['RA_dms'][j])
        Dec_dms.append(hdul[1].data['Dec_dms'][j])
        RA_Rad.append(hdul[1].data['RA_Rad'][j])
        Dec_Rad.append(hdul[1].data['Dec_Rad'][j])
        RA_ICRS.append(hdul[1].data['RA_ICRS'][j])
        Dec_ICRS.append(hdul[1].data['Dec_ICRS'][j])
        zmag.append(hdul[1].data['Mag_Z'][j])
        ymag.append(hdul[1].data['Mag_Y'][j])
        jmag.append(hdul[1].data['Mag_J'][j])
        hmag.append(hdul[1].data['Mag_H'][j])
        kmag.append(hdul[1].data['Mag_K'][j])
        zobsmag.append(hdul[1].data['Obs_Mag_Z'][j])
        yobsmag.append(hdul[1].data['Obs_Mag_Y'][j])
        jobsmag.append(hdul[1].data['Obs_Mag_J'][j])
        hobsmag.append(hdul[1].data['Obs_Mag_H'][j])
        kobsmag.append(hdul[1].data['Obs_Mag_K'][j])
        zaper.append(hdul[1].data['Aper_Z'][j])
        yaper.append(hdul[1].data['Aper_Y'][j])
        japer.append(hdul[1].data['Aper_J'][j])
        haper.append(hdul[1].data['Aper_H'][j])
        kaper.append(hdul[1].data['Aper_K'][j])
        zapererr.append(hdul[1].data['Aper_Z_err'][j])
        yapererr.append(hdul[1].data['Aper_Y_err'][j])
        japererr.append(hdul[1].data['Aper_J_err'][j])
        hapererr.append(hdul[1].data['Aper_H_err'][j])
        kapererr.append(hdul[1].data['Aper_K_err'][j])
    
    col1 = fits.Column(name='RA_dms',format='11A',unit='dms',array=RA_dms)
    col2 = fits.Column(name='Dec_dms',format='11A',unit='dms',array=Dec_dms)
    col3 = fits.Column(name='RA_Rad',format='D',unit='radians',array=RA_Rad)
    col4 = fits.Column(name='Dec_Rad',format='D',unit='radians',array=Dec_Rad)
    col3a = fits.Column(name='RA_ICRS',format='D',unit='degree',array=RA_ICRS)
    col4a = fits.Column(name='Dec_ICRS',format='D',unit='degree',array=Dec_ICRS)
    col5 = fits.Column(name='Mag_Z',format='D',unit='mag',array=zmag)
    col6 = fits.Column(name='Obs_Mag_Z',format='D',unit='mag',array=zobsmag)
    col7 = fits.Column(name='Aper_Z',format='E',unit='ADU',array=zaper)
    col8 = fits.Column(name='Aper_Z_err',format='E',unit='ADU',array=zapererr)
    col9 = fits.Column(name='Mag_Y',format='D',unit='mag',array=ymag)
    col10 = fits.Column(name='Obs_Mag_Y',format='D',unit='mag',array=yobsmag)
    col11 = fits.Column(name='Aper_Y',format='E',unit='ADU',array=yaper)
    col12 = fits.Column(name='Aper_Y_err',format='E',unit='ADU',array=yapererr)
    col13 = fits.Column(name='Mag_J',format='D',unit='mag',array=jmag)
    col14 = fits.Column(name='Obs_Mag_J',format='D',unit='mag',array=jobsmag)
    col15 = fits.Column(name='Aper_J',format='E',unit='ADU',array=japer)
    col16 = fits.Column(name='Aper_J_err',format='E',unit='ADU',array=japererr)
    col17 = fits.Column(name='Mag_H',format='D',unit='mag',array=hmag)
    col18 = fits.Column(name='Obs_Mag_H',format='D',unit='mag',array=hobsmag)
    col19 = fits.Column(name='Aper_H',format='E',unit='ADU',array=haper)
    col20 = fits.Column(name='Aper_H_err',format='E',unit='ADU',array=hapererr)
    col21 = fits.Column(name='Mag_K',format='D',unit='mag',array=kmag)
    col22 = fits.Column(name='Obs_Mag_K',format='D',unit='mag',array=kobsmag)
    col23 = fits.Column(name='Aper_K',format='E',unit='ADU',array=kaper)
    col24 = fits.Column(name='Aper_K_err',format='E',unit='ADU',array=kapererr)
    col25 = fits.Column(name='YND_zy',format='11A',unit='Flag',array=YND_zy)
    col26 = fits.Column(name='YND_zj',format='11A',unit='Flag',array=YND_zj)
    col27 = fits.Column(name='YND_zh',format='11A',unit='Flag',array=YND_zh)
    col28 = fits.Column(name='YND_zk',format='11A',unit='Flag',array=YND_zk)
    col29 = fits.Column(name='YND_yj',format='11A',unit='Flag',array=YND_yj)
    col30 = fits.Column(name='YND_yh',format='11A',unit='Flag',array=YND_yh)
    col31 = fits.Column(name='YND_yk',format='11A',unit='Flag',array=YND_yk)
    col32 = fits.Column(name='YND_jh',format='11A',unit='Flag',array=YND_jh)
    col33 = fits.Column(name='YND_jk',format='11A',unit='Flag',array=YND_jk)
    col34 = fits.Column(name='YND_hk',format='11A',unit='Flag',array=YND_hk)
    col35 = fits.Column(name='YNC_zy',format='11A',unit='Flag',array=YNC_zy)
    col36 = fits.Column(name='YNC_zj',format='11A',unit='Flag',array=YNC_zj)
    col37 = fits.Column(name='YNC_zh',format='11A',unit='Flag',array=YNC_zh)
    col38 = fits.Column(name='YNC_zk',format='11A',unit='Flag',array=YNC_zk)
    col39 = fits.Column(name='YNC_yj',format='11A',unit='Flag',array=YNC_yj)
    col40 = fits.Column(name='YNC_yh',format='11A',unit='Flag',array=YNC_yh)
    col41 = fits.Column(name='YNC_yk',format='11A',unit='Flag',array=YNC_yk)
    col42 = fits.Column(name='YNC_jh',format='11A',unit='Flag',array=YNC_jh)
    col43 = fits.Column(name='YNC_jk',format='11A',unit='Flag',array=YNC_jk)
    col44 = fits.Column(name='YNC_hk',format='11A',unit='Flag',array=YNC_hk)
    col45 = fits.Column(name='YFD_zy',format='11A',unit='Flag',array=YFD_zy)
    col46 = fits.Column(name='YFD_zj',format='11A',unit='Flag',array=YFD_zj)
    col47 = fits.Column(name='YFD_zh',format='11A',unit='Flag',array=YFD_zh)
    col48 = fits.Column(name='YFD_zk',format='11A',unit='Flag',array=YFD_zk)
    col49 = fits.Column(name='YFD_yj',format='11A',unit='Flag',array=YFD_yj)
    col50 = fits.Column(name='YFD_yh',format='11A',unit='Flag',array=YFD_yh)
    col51 = fits.Column(name='YFD_yk',format='11A',unit='Flag',array=YFD_yk)
    col52 = fits.Column(name='YFD_jh',format='11A',unit='Flag',array=YFD_jh)
    col53 = fits.Column(name='YFD_jk',format='11A',unit='Flag',array=YFD_jk)
    col54 = fits.Column(name='YFD_hk',format='11A',unit='Flag',array=YFD_hk)
    col55 = fits.Column(name='YFC_zy',format='11A',unit='Flag',array=YFC_zy)
    col56 = fits.Column(name='YFC_zj',format='11A',unit='Flag',array=YFC_zj)
    col57 = fits.Column(name='YFC_zh',format='11A',unit='Flag',array=YFC_zh)
    col58 = fits.Column(name='YFC_zk',format='11A',unit='Flag',array=YFC_zk)
    col59 = fits.Column(name='YFC_yj',format='11A',unit='Flag',array=YFC_yj)
    col60 = fits.Column(name='YFC_yh',format='11A',unit='Flag',array=YFC_yh)
    col61 = fits.Column(name='YFC_yk',format='11A',unit='Flag',array=YFC_yk)
    col62 = fits.Column(name='YFC_jh',format='11A',unit='Flag',array=YFC_jh)
    col63 = fits.Column(name='YFC_jk',format='11A',unit='Flag',array=YFC_jk)
    col64 = fits.Column(name='YFC_hk',format='11A',unit='Flag',array=YFC_hk)
    col65 = fits.Column(name='OND_zy',format='11A',unit='Flag',array=OND_zy)
    col66 = fits.Column(name='OND_zj',format='11A',unit='Flag',array=OND_zj)
    col67 = fits.Column(name='OND_zh',format='11A',unit='Flag',array=OND_zh)
    col68 = fits.Column(name='OND_zk',format='11A',unit='Flag',array=OND_zk)
    col69 = fits.Column(name='OND_yj',format='11A',unit='Flag',array=OND_yj)
    col70 = fits.Column(name='OND_yh',format='11A',unit='Flag',array=OND_yh)
    col71 = fits.Column(name='OND_yk',format='11A',unit='Flag',array=OND_yk)
    col72 = fits.Column(name='OND_jh',format='11A',unit='Flag',array=OND_jh)
    col73 = fits.Column(name='OND_jk',format='11A',unit='Flag',array=OND_jk)
    col74 = fits.Column(name='OND_hk',format='11A',unit='Flag',array=OND_hk)
    col75 = fits.Column(name='ONC_zy',format='11A',unit='Flag',array=ONC_zy)
    col76 = fits.Column(name='ONC_zj',format='11A',unit='Flag',array=ONC_zj)
    col77 = fits.Column(name='ONC_zh',format='11A',unit='Flag',array=ONC_zh)
    col78 = fits.Column(name='ONC_zk',format='11A',unit='Flag',array=ONC_zk)
    col79 = fits.Column(name='ONC_yj',format='11A',unit='Flag',array=ONC_yj)
    col80 = fits.Column(name='ONC_yh',format='11A',unit='Flag',array=ONC_yh)
    col81 = fits.Column(name='ONC_yk',format='11A',unit='Flag',array=ONC_yk)
    col82 = fits.Column(name='ONC_jh',format='11A',unit='Flag',array=ONC_jh)
    col83 = fits.Column(name='ONC_jk',format='11A',unit='Flag',array=ONC_jk)
    col84 = fits.Column(name='ONC_hk',format='11A',unit='Flag',array=ONC_hk)
    col85 = fits.Column(name='OFD_zy',format='11A',unit='Flag',array=OFD_zy)
    col86 = fits.Column(name='OFD_zj',format='11A',unit='Flag',array=OFD_zj)
    col87 = fits.Column(name='OFD_zh',format='11A',unit='Flag',array=OFD_zh)
    col88 = fits.Column(name='OFD_zk',format='11A',unit='Flag',array=OFD_zk)
    col89 = fits.Column(name='OFD_yj',format='11A',unit='Flag',array=OFD_yj)
    col90 = fits.Column(name='OFD_yh',format='11A',unit='Flag',array=OFD_yh)
    col91 = fits.Column(name='OFD_yk',format='11A',unit='Flag',array=OFD_yk)
    col92 = fits.Column(name='OFD_jh',format='11A',unit='Flag',array=OFD_jh)
    col93 = fits.Column(name='OFD_jk',format='11A',unit='Flag',array=OFD_jk)
    col94 = fits.Column(name='OFD_hk',format='11A',unit='Flag',array=OFD_hk)
    col95 = fits.Column(name='OFC_zy',format='11A',unit='Flag',array=OFC_zy)
    col96 = fits.Column(name='OFC_zj',format='11A',unit='Flag',array=OFC_zj)
    col97 = fits.Column(name='OFC_zh',format='11A',unit='Flag',array=OFC_zh)
    col98 = fits.Column(name='OFC_zk',format='11A',unit='Flag',array=OFC_zk)
    col99 = fits.Column(name='OFC_yj',format='11A',unit='Flag',array=OFC_yj)
    col100 = fits.Column(name='OFC_yh',format='11A',unit='Flag',array=OFC_yh)
    col101 = fits.Column(name='OFC_yk',format='11A',unit='Flag',array=OFC_yk)
    col102 = fits.Column(name='OFC_jh',format='11A',unit='Flag',array=OFC_jh)
    col103 = fits.Column(name='OFC_jk',format='11A',unit='Flag',array=OFC_jk)
    col104 = fits.Column(name='OFC_hk',format='11A',unit='Flag',array=OFC_hk)
    col105 = fits.Column(name='YND Total',format='F',unit='Counts',array=YND)
    col106 = fits.Column(name='YNC Total',format='F',unit='Counts',array=YNC)
    col107 = fits.Column(name='YFD Total',format='F',unit='Counts',array=YFD)
    col108 = fits.Column(name='YFC Total',format='F',unit='Counts',array=YFC)
    col109 = fits.Column(name='OND Total',format='F',unit='Counts',array=OND)
    col110 = fits.Column(name='ONC Total',format='F',unit='Counts',array=ONC)
    col111 = fits.Column(name='OFD Total',format='F',unit='Counts',array=OFD)
    col112 = fits.Column(name='OFC Total',format='F',unit='Counts',array=OFC)
    col132 = fits.Column(name='YND Perc',format='F',unit='Percentage',array=YNDperc)
    col133 = fits.Column(name='YNC Perc',format='F',unit='Percentage',array=YNCperc)
    col134 = fits.Column(name='YFD Perc',format='F',unit='Percentage',array=YFDperc)
    col135 = fits.Column(name='YFC Perc',format='F',unit='Percentage',array=YFCperc)
    col136 = fits.Column(name='OND Perc',format='F',unit='Percentage',array=ONDperc)
    col137 = fits.Column(name='ONC Perc',format='F',unit='Percentage',array=ONCperc)
    col138 = fits.Column(name='OFD Perc',format='F',unit='Percentage',array=OFDperc)
    col139 = fits.Column(name='OFC Perc',format='F',unit='Percentage',array=OFCperc)
    col113 = fits.Column(name='Y Total',format='F',unit='Counts',array=Y)
    col114 = fits.Column(name='O Total',format='F',unit='Counts',array=O)
    col115 = fits.Column(name='N Total',format='F',unit='Counts',array=N)
    col116 = fits.Column(name='F Total',format='F',unit='Counts',array=F)
    col117 = fits.Column(name='D Total',format='F',unit='Counts',array=D)
    col118 = fits.Column(name='C Total',format='F',unit='Counts',array=C)
    col119 = fits.Column(name='ZY Total',format='F',unit='Counts',array=ZY)
    col120 = fits.Column(name='ZJ Total',format='F',unit='Counts',array=ZJ)
    col121 = fits.Column(name='ZH Total',format='F',unit='Counts',array=ZH)
    col122 = fits.Column(name='ZK Total',format='F',unit='Counts',array=ZK)
    col123 = fits.Column(name='YJ Total',format='F',unit='Counts',array=YJ)
    col124 = fits.Column(name='YH Total',format='F',unit='Counts',array=YH)
    col125 = fits.Column(name='YK Total',format='F',unit='Counts',array=YK)
    col126 = fits.Column(name='JH Total',format='F',unit='Counts',array=JH)
    col127 = fits.Column(name='JK Total',format='F',unit='Counts',array=JK)
    col128 = fits.Column(name='HK Total',format='F',unit='Counts',array=HK)
    col129 = fits.Column(name='Filters',format='F',unit='Number',array=Filters)
    col130 = fits.Column(name='Total Counts',format='F',unit='Number',array=Total)
    col131 = fits.Column(name='Percentage Count',format='F',unit='Percentage',array=Percentage)
    col143 = fits.Column(name='Max_Percentage',format='F',unit='Percentage',array=np.array(Max_Perc))

    cols = fits.ColDefs([col1,col2,col3,col4,col3a,col4a,col5,col6,col7,col8,col9,col10,
                         col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,
                         col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,
                         col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,
                         col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,
                         col51,col52,col53,col54,col55,col56,col57,col58,col59,col60,
                         col61,col62,col63,col64,col65,col66,col67,col68,col69,col70,
                         col71,col72,col73,col74,col75,col76,col77,col78,col79,col80,
                         col81,col82,col83,col84,col85,col86,col87,col88,col89,col90,
                         col91,col92,col93,col94,col95,col96,col97,col98,col99,col100,
                         col101,col102,col103,col104,col105,col132,col106,col133,col107,
                         col134,col108,col135,col109,col136,col110,col137,col111,col138,
                         col112,col139,col113,col114,col115,col116,col117,col118,col119,
                         col120,col121,col122,col123,col124,col125,col126,col127,col128,
                         col129,col130,col131,col143])
    
    '''Here we make the fits file'''

    data = fits.BinTableHDU.from_columns(cols)
    data.writeto(output_fits)

    '''Now we make the reg file'''

    pos_data = []

    for v in range(len(RA_dms)):

        pos_data.append([RA_dms[v],Dec_dms[v]])

    with open(output_reg, 'w', newline='') as csvfile:
        reg_file = csv.writer(csvfile, delimiter=',', quotechar=' ', quoting=csv.QUOTE_ALL)
        for w in range(len(pos_data)):
            reg_file.writerow([pos_data[w][0] , pos_data[w][1]])
            
def Distance(d=10):
    '''This is just the distance modulus, for use in parsecs.'''
    mu = 5*math.log(d,10)-5
    return mu

In [ ]:
def vlm_object_added(file,model,output_name,percnum=.5):
    
    '''This is very similar to the previous code, but this one removes sources that do not have a high enough 
    percentage of flags (percnum), but adds back in any that have at least that percentage in an individual set-up.
    In other words, this was used to determine the final candidate list.'''
    
    RA_dms = []
    Dec_dms = []
    RA_Rad = []
    Dec_Rad = []
    RA_ICRS = []
    Dec_ICRS = []
    zmag = []
    ymag = []
    jmag = []
    hmag = []
    kmag = []
    zobsmag = []
    yobsmag = []
    jobsmag = []
    hobsmag = []
    kobsmag = []
    zaper = []
    yaper = []
    japer = []
    haper = []
    kaper = []
    zapererr = []
    yapererr = []
    japererr = []
    hapererr = []
    kapererr = []
    
    intest = [[[[[0 for z in range(5)] for y in range(4)] for x in range(2)] for w in range(2)] for v in range(2)]
    
    hdul = fits.open(file)
    allpoints = list(range(len(hdul[1].data)))
    
    Filters = [0 for z in allpoints]
    YND = []
    YNC = []
    YFD = []
    YFC = []
    OND = []
    ONC = []
    OFD = []
    OFC = []
    ZY = []
    ZJ = []
    ZH = []
    ZK = []
    YJ = []
    YH = []
    YK = []
    JH = []
    JK = []
    HK = []
    Max = [0 for z in allpoints]
    Total = [0 for z in allpoints]
    Percentage = [0 for z in allpoints]
    Y = [0 for z in allpoints]
    O = [0 for z in allpoints]
    N = [0 for z in allpoints]
    F = [0 for z in allpoints]
    D = [0 for z in allpoints]
    C = [0 for z in allpoints]
    AgeDiff = [0 for z in allpoints]
    DisDiff = [0 for z in allpoints]
    ModDiff = [0 for z in allpoints]
    YNDperc = [0 for z in allpoints]
    YNCperc = [0 for z in allpoints]
    YFDperc = [0 for z in allpoints]
    YFCperc = [0 for z in allpoints]
    ONDperc = [0 for z in allpoints]
    ONCperc = [0 for z in allpoints]
    OFDperc = [0 for z in allpoints]
    OFCperc = [0 for z in allpoints]
    Max_Perc = [0 for z in allpoints]

    ra_dms = hdul[1].data['RA_dms']
    dec_dms = hdul[1].data['Dec_dms']
    ra_rad = hdul[1].data['RA_Rad']
    dec_rad = hdul[1].data['Dec_Rad']
    ra_icrs = hdul[1].data['RA_ICRS']
    dec_icrs = hdul[1].data['Dec_ICRS']
    mag_z = hdul[1].data['Mag_Z']
    obs_mag_z = hdul[1].data['Obs_Mag_Z']
    aper_z = hdul[1].data['Aper_Z']
    aper_z_err = hdul[1].data['Aper_Z_err']
    mag_y = hdul[1].data['Mag_Y']
    obs_mag_y = hdul[1].data['Obs_Mag_Y']
    aper_y = hdul[1].data['Aper_Y']
    aper_y_err = hdul[1].data['Aper_Y_err']
    mag_j = hdul[1].data['Mag_J']
    obs_mag_j = hdul[1].data['Obs_Mag_J']
    aper_j = hdul[1].data['Aper_J']
    aper_j_err = hdul[1].data['Aper_J_err']
    mag_h = hdul[1].data['Mag_H']
    obs_mag_h = hdul[1].data['Obs_Mag_H']
    aper_h = hdul[1].data['Aper_H']
    aper_h_err = hdul[1].data['Aper_H_err']
    mag_k = hdul[1].data['Mag_K']
    obs_mag_k = hdul[1].data['Obs_Mag_K']
    aper_k = hdul[1].data['Aper_K']
    aper_k_err = hdul[1].data['Aper_K_err']
    
    err_z = aper_z_err/(aper_z)
    err_y = aper_y_err/(aper_y)
    err_j = aper_j_err/(aper_j)
    err_h = aper_h_err/(aper_h)
    err_k = aper_k_err/(aper_k)
    
    model = fits.open('model_iso.fits')

    mass1 = model[1].data['Mass'][0:28]
    teff1 = model[1].data['Teff'][0:28]
    zmd1 = model[1].data['ZmagD'][0:28]
    ymd1 = model[1].data['YmagD'][0:28]
    jmd1 = model[1].data['JmagD'][0:28]
    hmd1 = model[1].data['HmagD'][0:28]
    kmd1 = model[1].data['KsmagD'][0:28]
    zmc1 = model[1].data['ZmagC'][0:28]
    ymc1 = model[1].data['YmagC'][0:28]
    jmc1 = model[1].data['JmagC'][0:28]
    hmc1 = model[1].data['HmagC'][0:28]
    kmc1 = model[1].data['KsmagC'][0:28]
    
    mass5 = model[1].data['Mass'][28:57]
    teff5 = model[1].data['Teff'][28:57]
    zmd5 = model[1].data['ZmagD'][28:57]
    ymd5 = model[1].data['YmagD'][28:57]
    jmd5 = model[1].data['JmagD'][28:57]
    hmd5 = model[1].data['HmagD'][28:57]
    kmd5 = model[1].data['KsmagD'][28:57]
    zmc5 = model[1].data['ZmagC'][28:57]
    ymc5 = model[1].data['YmagC'][28:57]
    jmc5 = model[1].data['JmagC'][28:57]
    hmc5 = model[1].data['HmagC'][28:57]
    kmc5 = model[1].data['KsmagC'][28:57]

    massy_fit = np.polyfit(teff1, mass1, 14)
    massy_poly = np.poly1d(massy_fit)
    zcy_fit = np.polyfit(teff1, zmc1, 14)
    zcy_poly = np.poly1d(zcy_fit)
    zdy_fit = np.polyfit(teff1, zmd1, 14)
    zdy_poly = np.poly1d(zdy_fit)
    ycy_fit = np.polyfit(teff1, ymc1, 14)
    ycy_poly = np.poly1d(ycy_fit)
    ydy_fit = np.polyfit(teff1, ymd1, 14)
    ydy_poly = np.poly1d(ydy_fit)
    jcy_fit = np.polyfit(teff1, jmc1, 14)
    jcy_poly = np.poly1d(jcy_fit)
    jdy_fit = np.polyfit(teff1, jmd1, 14)
    jdy_poly = np.poly1d(jdy_fit)
    hcy_fit = np.polyfit(teff1, hmc1, 14)
    hcy_poly = np.poly1d(hcy_fit)
    hdy_fit = np.polyfit(teff1, hmd1, 14)
    hdy_poly = np.poly1d(hdy_fit)
    kcy_fit = np.polyfit(teff1, kmc1, 14)
    kcy_poly = np.poly1d(kcy_fit)
    kdy_fit = np.polyfit(teff1, kmd1, 14)
    kdy_poly = np.poly1d(kdy_fit)
    
    masso_fit = np.polyfit(teff5, mass5, 14)
    masso_poly = np.poly1d(masso_fit)
    zco_fit = np.polyfit(teff5, zmc5, 14)
    zco_poly = np.poly1d(zco_fit)
    zdo_fit = np.polyfit(teff5, zmd5, 14)
    zdo_poly = np.poly1d(zdo_fit)
    yco_fit = np.polyfit(teff5, ymc5, 14)
    yco_poly = np.poly1d(yco_fit)
    ydo_fit = np.polyfit(teff5, ymd5, 14)
    ydo_poly = np.poly1d(ydo_fit)
    jco_fit = np.polyfit(teff5, jmc5, 14)
    jco_poly = np.poly1d(jco_fit)
    jdo_fit = np.polyfit(teff5, jmd5, 14)
    jdo_poly = np.poly1d(jdo_fit)
    hco_fit = np.polyfit(teff5, hmc5, 14)
    hco_poly = np.poly1d(hco_fit)
    hdo_fit = np.polyfit(teff5, hmd5, 14)
    hdo_poly = np.poly1d(hdo_fit)
    kco_fit = np.polyfit(teff5, kmc5, 14)
    kco_poly = np.poly1d(kco_fit)
    kdo_fit = np.polyfit(teff5, kmd5, 14)
    kdo_poly = np.poly1d(kdo_fit)
    
    idz = np.isfinite(mag_z) & np.isfinite(err_z)
    zerr_fit = np.polyfit(mag_z[idz],err_z[idz],8)
    zerr_poly = np.poly1d(zerr_fit)
    idy = np.isfinite(mag_y) & np.isfinite(err_y)
    yerr_fit = np.polyfit(mag_y[idy],err_y[idy],8)
    yerr_poly = np.poly1d(yerr_fit)
    idj = np.isfinite(mag_j) & np.isfinite(err_j)
    jerr_fit = np.polyfit(mag_j[idj],err_j[idj],8)
    jerr_poly = np.poly1d(jerr_fit)
    idh = np.isfinite(mag_h) & np.isfinite(err_h)
    herr_fit = np.polyfit(mag_h[idh],err_h[idh],8)
    herr_poly = np.poly1d(herr_fit)
    idk = np.isfinite(mag_k) & np.isfinite(err_k)
    kerr_fit = np.polyfit(mag_k[idk],err_k[idk],8)
    kerr_poly = np.poly1d(kerr_fit)
    
    bd_range_1myr = range(1300,2931)
    full_rangey = range(1300,4000)
    full_rangeo = range(1300,4000)
    full_rangec = range(1300,4000)
    full_ranged = range(1650,4000)
    
    mu_far = Distance(420)
    mu_near = Distance(350)
    Av = 1.1
    
    Az = 0.5006359319 * Av
    Ay = 0.3913227425 * Av
    Aj = 0.281344478 * Av
    Ah = 0.1812818241 * Av
    Ak = 0.1180678823 * Av
    
    Azy = Az - Ay
    Azj = Az - Aj
    Azh = Az - Ah
    Azk = Az - Ak
    Ayj = Ay - Aj
    Ayh = Ay - Ah
    Ayk = Ay - Ak
    Ajh = Aj - Ah
    Ajk = Aj - Ak
    Ahk = Ah - Ak
    
    poly = [[[zdy_poly(full_ranged),zdo_poly(full_ranged)],[zcy_poly(full_rangec),zco_poly(full_rangec)]],
            [[ydy_poly(full_ranged),ydo_poly(full_ranged)],[ycy_poly(full_rangec),yco_poly(full_rangec)]],
            [[jdy_poly(full_ranged),jdo_poly(full_ranged)],[jcy_poly(full_rangec),jco_poly(full_rangec)]],
            [[hdy_poly(full_ranged),hdo_poly(full_ranged)],[hcy_poly(full_rangec),hco_poly(full_rangec)]],
            [[kdy_poly(full_ranged),kdo_poly(full_ranged)],[kcy_poly(full_rangec),kco_poly(full_rangec)]]]
    
    observed_data = [mag_z,mag_y,mag_j,mag_h,mag_k]
    observed_array = np.array(observed_data)

    ext = [[Az,Ay,Aj,Ah,Ak],[[0,Azy,Azj,Azh,Azk],[0,0,Ayj,Ayh,Ayk],[0,0,0,Ajh,Ajk],[0,0,0,0,Ahk]]]
    
    mu = [mu_near,mu_far]
    
    for alpha in range(2):
        for beta in range(2):
            for gamma in range(2):
                for delta in range(4):
                    for epsilon in range(delta+1,5):
                        
                        err_poly_max = [zerr_poly(poly[delta][gamma][beta] + mu[alpha] + 1*ext[0][delta]),
                                    yerr_poly(poly[delta][gamma][beta] + mu[alpha] + 1*ext[0][delta]),
                                    jerr_poly(poly[delta][gamma][beta] + mu[alpha] + 1*ext[0][delta]),
                                    herr_poly(poly[delta][gamma][beta] + mu[alpha] + 1*ext[0][delta]),
                                    kerr_poly(poly[delta][gamma][beta] + mu[alpha] + 1*ext[0][delta])]
                        
                        err_poly_max_ep = [zerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 1*ext[0][epsilon]),
                                    yerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 1*ext[0][epsilon]),
                                    jerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 1*ext[0][epsilon]),
                                    herr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 1*ext[0][epsilon]),
                                    kerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 1*ext[0][epsilon])]
                        
                        err_poly_no = [zerr_poly(poly[delta][gamma][beta] + mu[alpha] + 0*ext[0][delta]),
                                    yerr_poly(poly[delta][gamma][beta] + mu[alpha] + 0*ext[0][delta]),
                                    jerr_poly(poly[delta][gamma][beta] + mu[alpha] + 0*ext[0][delta]),
                                    herr_poly(poly[delta][gamma][beta] + mu[alpha] + 0*ext[0][delta]),
                                    kerr_poly(poly[delta][gamma][beta] + mu[alpha] + 0*ext[0][delta])]
                        
                        err_poly_no_ep = [zerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 0*ext[0][epsilon]),
                                    yerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 0*ext[0][epsilon]),
                                    jerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 0*ext[0][epsilon]),
                                    herr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 0*ext[0][epsilon]),
                                    kerr_poly(poly[epsilon][gamma][beta] + mu[alpha] + 0*ext[0][epsilon])]
                            
                        maxext = list(zip((poly[delta][gamma][beta] - 
                                           poly[epsilon][gamma][beta] + 
                                           (1*ext[1][delta][epsilon])) + 
                                          np.sqrt(err_poly_max[delta]**2 + err_poly_max_ep[epsilon]**2),
                                          poly[delta][gamma][beta] + mu[alpha] + 
                                          (1*ext[0][delta]) - err_poly_max[delta]))
                        noext = list(zip((poly[delta][gamma][beta] - 
                                          poly[epsilon][gamma][beta] + 
                                          (0*ext[1][delta][epsilon])) - 
                                         np.sqrt(err_poly_no[delta]**2 + err_poly_no_ep[epsilon]**2),
                                         poly[delta][gamma][beta] + mu[alpha] + 
                                         (0*ext[0][delta]) + err_poly_no[delta]))
                        
                        sidepoints = maxext + noext[::-1]
                        polypath = path.Path(sidepoints, closed=True)
                        
                        points = list(zip(observed_data[delta]-observed_data[epsilon],observed_data[delta]))
                        intest[beta][alpha][gamma][delta][epsilon] = polypath.contains_points(points)
    
    for i in allpoints:
        Filters[i] = np.sum(~np.isnan(observed_array[:,i]))
        
        outynd = []
        outync = []
        outyfd = []
        outyfc = []
        outond = []
        outonc = []
        outofd = []
        outofc = []

        for d in range(4):
            for e in range(d+1,5):
                outynd.append(intest[0][0][0][d][e][i])
                outync.append(intest[0][0][1][d][e][i])
                outyfd.append(intest[0][1][0][d][e][i])
                outyfc.append(intest[0][1][1][d][e][i])
                outond.append(intest[1][0][0][d][e][i])
                outonc.append(intest[1][0][1][d][e][i])
                outofd.append(intest[1][1][0][d][e][i])
                outofc.append(intest[1][1][1][d][e][i])
        YND.append(np.sum(outynd))
        YNC.append(np.sum(outync))
        YFD.append(np.sum(outyfd))
        YFC.append(np.sum(outyfc))
        OND.append(np.sum(outond))
        ONC.append(np.sum(outonc))
        OFD.append(np.sum(outofd))
        OFC.append(np.sum(outofc))

        outzy = []
        outzj = []
        outzh = []
        outzk = []
        outyj = []
        outyh = []
        outyk = []
        outjh = []
        outjk = []
        outhk = []

        for a in range(2):
            for b in range(2):
                for c in range(2):
                    outzy.append(intest[a][b][c][0][1][i])
                    outzj.append(intest[a][b][c][0][2][i])
                    outzh.append(intest[a][b][c][0][3][i])
                    outzk.append(intest[a][b][c][0][4][i])
                    outyj.append(intest[a][b][c][1][2][i])
                    outyh.append(intest[a][b][c][1][3][i])
                    outyk.append(intest[a][b][c][1][4][i])
                    outjh.append(intest[a][b][c][2][3][i])
                    outjk.append(intest[a][b][c][2][4][i])
                    outhk.append(intest[a][b][c][3][4][i])
        ZY.append(np.sum(outzy))
        ZJ.append(np.sum(outzj))
        ZH.append(np.sum(outzh))
        ZK.append(np.sum(outzk))
        YJ.append(np.sum(outyj))
        YH.append(np.sum(outyh))
        YK.append(np.sum(outyk))
        JH.append(np.sum(outjh))
        JK.append(np.sum(outjk))
        HK.append(np.sum(outhk))
        
        if Filters[i] == 2:
            Max[i] = 1*8
        if Filters[i] == 3:
            Max[i] = 3*8
        if Filters[i] == 4:
            Max[i] = 6*8
        if Filters[i] == 5:
            Max[i] = 10*8
           
        Y[i] = np.sum(YND[i] + YNC[i] + YFD[i] + YFC[i])
        O[i] = np.sum(OND[i] + ONC[i] + OFD[i] + OFC[i])
        N[i] = np.sum(YND[i] + YNC[i] + OND[i] + ONC[i])
        F[i] = np.sum(YFD[i] + YFC[i] + OFD[i] + OFC[i])
        D[i] = np.sum(YND[i] + OND[i] + YFD[i] + OFD[i])
        C[i] = np.sum(YNC[i] + ONC[i] + YFC[i] + OFC[i])
        AgeDiff[i] = Y[i] - O[i]
        DisDiff[i] = N[i] - F[i]
        ModDiff[i] = D[i] - C[i]
    
        Total[i] = sum([YND[i],YNC[i],YFD[i],YFC[i],OND[i],ONC[i],OFD[i],OFC[i]])
        YNDperc[i] = YND[i]/(Max[i]/8)
        YNCperc[i] = YNC[i]/(Max[i]/8)
        YFDperc[i] = YFD[i]/(Max[i]/8)
        YFCperc[i] = YFC[i]/(Max[i]/8)
        ONDperc[i] = OND[i]/(Max[i]/8)
        ONCperc[i] = ONC[i]/(Max[i]/8)
        OFDperc[i] = OFD[i]/(Max[i]/8)
        OFCperc[i] = OFC[i]/(Max[i]/8)
        Percentage[i] = Total[i]/Max[i]
        Max_Perc[i] = max(YNDperc[i],YNCperc[i],YFDperc[i],YFCperc[i],
                          ONDperc[i],ONCperc[i],OFDperc[i],OFCperc[i],Percentage[i])
    
    YND_zy = intest[0][0][0][0][1]
    YND_zj = intest[0][0][0][0][2]
    YND_zh = intest[0][0][0][0][3]
    YND_zk = intest[0][0][0][0][4]
    YND_yj = intest[0][0][0][1][2]
    YND_yh = intest[0][0][0][1][3]
    YND_yk = intest[0][0][0][1][4]
    YND_jh = intest[0][0][0][2][3]
    YND_jk = intest[0][0][0][2][4]
    YND_hk = intest[0][0][0][3][4]
    YNC_zy = intest[0][0][1][0][1]
    YNC_zj = intest[0][0][1][0][2]
    YNC_zh = intest[0][0][1][0][3]
    YNC_zk = intest[0][0][1][0][4]
    YNC_yj = intest[0][0][1][1][2]
    YNC_yh = intest[0][0][1][1][3]
    YNC_yk = intest[0][0][1][1][4]
    YNC_jh = intest[0][0][1][2][3]
    YNC_jk = intest[0][0][1][2][4]
    YNC_hk = intest[0][0][1][3][4]
    YFD_zy = intest[0][1][0][0][1]
    YFD_zj = intest[0][1][0][0][2]
    YFD_zh = intest[0][1][0][0][3]
    YFD_zk = intest[0][1][0][0][4]
    YFD_yj = intest[0][1][0][1][2]
    YFD_yh = intest[0][1][0][1][3]
    YFD_yk = intest[0][1][0][1][4]
    YFD_jh = intest[0][1][0][2][3]
    YFD_jk = intest[0][1][0][2][4]
    YFD_hk = intest[0][1][0][3][4]
    YFC_zy = intest[0][1][1][0][1]
    YFC_zj = intest[0][1][1][0][2]
    YFC_zh = intest[0][1][1][0][3]
    YFC_zk = intest[0][1][1][0][4]
    YFC_yj = intest[0][1][1][1][2]
    YFC_yh = intest[0][1][1][1][3]
    YFC_yk = intest[0][1][1][1][4]
    YFC_jh = intest[0][1][1][2][3]
    YFC_jk = intest[0][1][1][2][4]
    YFC_hk = intest[0][1][1][3][4]
    OND_zy = intest[1][0][0][0][1]
    OND_zj = intest[1][0][0][0][2]
    OND_zh = intest[1][0][0][0][3]
    OND_zk = intest[1][0][0][0][4]
    OND_yj = intest[1][0][0][1][2]
    OND_yh = intest[1][0][0][1][3]
    OND_yk = intest[1][0][0][1][4]
    OND_jh = intest[1][0][0][2][3]
    OND_jk = intest[1][0][0][2][4]
    OND_hk = intest[1][0][0][3][4]
    ONC_zy = intest[1][0][1][0][1]
    ONC_zj = intest[1][0][1][0][2]
    ONC_zh = intest[1][0][1][0][3]
    ONC_zk = intest[1][0][1][0][4]
    ONC_yj = intest[1][0][1][1][2]
    ONC_yh = intest[1][0][1][1][3]
    ONC_yk = intest[1][0][1][1][4]
    ONC_jh = intest[1][0][1][2][3]
    ONC_jk = intest[1][0][1][2][4]
    ONC_hk = intest[1][0][1][3][4]
    OFD_zy = intest[1][1][0][0][1]
    OFD_zj = intest[1][1][0][0][2]
    OFD_zh = intest[1][1][0][0][3]
    OFD_zk = intest[1][1][0][0][4]
    OFD_yj = intest[1][1][0][1][2]
    OFD_yh = intest[1][1][0][1][3]
    OFD_yk = intest[1][1][0][1][4]
    OFD_jh = intest[1][1][0][2][3]
    OFD_jk = intest[1][1][0][2][4]
    OFD_hk = intest[1][1][0][3][4]
    OFC_zy = intest[1][1][1][0][1]
    OFC_zj = intest[1][1][1][0][2]
    OFC_zh = intest[1][1][1][0][3]
    OFC_zk = intest[1][1][1][0][4]
    OFC_yj = intest[1][1][1][1][2]
    OFC_yh = intest[1][1][1][1][3]
    OFC_yk = intest[1][1][1][1][4]
    OFC_jh = intest[1][1][1][2][3]
    OFC_jk = intest[1][1][1][2][4]
    OFC_hk = intest[1][1][1][3][4]
    
    output_fits = output_name + '.fits'
    output_reg = output_name + '.reg'
    
    chosen = []
    
    for j in allpoints:
        RA_dms.append(hdul[1].data['RA_dms'][j])
        Dec_dms.append(hdul[1].data['Dec_dms'][j])
        RA_Rad.append(hdul[1].data['RA_Rad'][j])
        Dec_Rad.append(hdul[1].data['Dec_Rad'][j])
        RA_ICRS.append(hdul[1].data['RA_ICRS'][j])
        Dec_ICRS.append(hdul[1].data['Dec_ICRS'][j])
        zmag.append(hdul[1].data['Mag_Z'][j])
        ymag.append(hdul[1].data['Mag_Y'][j])
        jmag.append(hdul[1].data['Mag_J'][j])
        hmag.append(hdul[1].data['Mag_H'][j])
        kmag.append(hdul[1].data['Mag_K'][j])
        zobsmag.append(hdul[1].data['Obs_Mag_Z'][j])
        yobsmag.append(hdul[1].data['Obs_Mag_Y'][j])
        jobsmag.append(hdul[1].data['Obs_Mag_J'][j])
        hobsmag.append(hdul[1].data['Obs_Mag_H'][j])
        kobsmag.append(hdul[1].data['Obs_Mag_K'][j])
        zaper.append(hdul[1].data['Aper_Z'][j])
        yaper.append(hdul[1].data['Aper_Y'][j])
        japer.append(hdul[1].data['Aper_J'][j])
        haper.append(hdul[1].data['Aper_H'][j])
        kaper.append(hdul[1].data['Aper_K'][j])
        zapererr.append(hdul[1].data['Aper_Z_err'][j])
        yapererr.append(hdul[1].data['Aper_Y_err'][j])
        japererr.append(hdul[1].data['Aper_J_err'][j])
        hapererr.append(hdul[1].data['Aper_H_err'][j])
        kapererr.append(hdul[1].data['Aper_K_err'][j])
        
        if Percentage[j] >= percnum:
            chosen.append(j)
        else:
            if ZJ[j] > 3:
                if YNDperc[j] >= percnum:
                    chosen.append(j)
                elif YNCperc[j] >= percnum:
                    chosen.append(j)
                elif YFDperc[j] >= percnum:
                    chosen.append(j)
                elif YFCperc[j] >= percnum:
                    chosen.append(j)
                elif ONDperc[j] >= percnum:
                    chosen.append(j)
                elif ONCperc[j] >= percnum:
                    chosen.append(j)
                elif OFDperc[j] >= percnum:
                    chosen.append(j)
                elif OFCperc[j] >= percnum:
                    chosen.append(j)
    
    col1 = fits.Column(name='RA_dms',format='11A',unit='dms',array=np.array(RA_dms)[chosen])
    col2 = fits.Column(name='Dec_dms',format='11A',unit='dms',array=np.array(Dec_dms)[chosen])
    col3 = fits.Column(name='RA_Rad',format='D',unit='radians',array=np.array(RA_Rad)[chosen])
    col4 = fits.Column(name='Dec_Rad',format='D',unit='radians',array=np.array(Dec_Rad)[chosen])
    col3a = fits.Column(name='RA_ICRS',format='D',unit='deg',array=np.array(RA_ICRS)[chosen])
    col4a = fits.Column(name='Dec_ICRS',format='D',unit='deg',array=np.array(Dec_ICRS)[chosen])
    col5 = fits.Column(name='Mag_Z',format='D',unit='mag',array=np.array(zmag)[chosen])
    col6 = fits.Column(name='Obs_Mag_Z',format='D',unit='mag',array=np.array(zobsmag)[chosen])
    col7 = fits.Column(name='Aper_Z',format='E',unit='ADU',array=np.array(zaper)[chosen])
    col8 = fits.Column(name='Aper_Z_err',format='E',unit='ADU',array=np.array(zapererr)[chosen])
    col9 = fits.Column(name='Mag_Y',format='D',unit='mag',array=np.array(ymag)[chosen])
    col10 = fits.Column(name='Obs_Mag_Y',format='D',unit='mag',array=np.array(yobsmag)[chosen])
    col11 = fits.Column(name='Aper_Y',format='E',unit='ADU',array=np.array(yaper)[chosen])
    col12 = fits.Column(name='Aper_Y_err',format='E',unit='ADU',array=np.array(yapererr)[chosen])
    col13 = fits.Column(name='Mag_J',format='D',unit='mag',array=np.array(jmag)[chosen])
    col14 = fits.Column(name='Obs_Mag_J',format='D',unit='mag',array=np.array(jobsmag)[chosen])
    col15 = fits.Column(name='Aper_J',format='E',unit='ADU',array=np.array(japer)[chosen])
    col16 = fits.Column(name='Aper_J_err',format='E',unit='ADU',array=np.array(japererr)[chosen])
    col17 = fits.Column(name='Mag_H',format='D',unit='mag',array=np.array(hmag)[chosen])
    col18 = fits.Column(name='Obs_Mag_H',format='D',unit='mag',array=np.array(hobsmag)[chosen])
    col19 = fits.Column(name='Aper_H',format='E',unit='ADU',array=np.array(haper)[chosen])
    col20 = fits.Column(name='Aper_H_err',format='E',unit='ADU',array=np.array(hapererr)[chosen])
    col21 = fits.Column(name='Mag_K',format='D',unit='mag',array=np.array(kmag)[chosen])
    col22 = fits.Column(name='Obs_Mag_K',format='D',unit='mag',array=np.array(kobsmag)[chosen])
    col23 = fits.Column(name='Aper_K',format='E',unit='ADU',array=np.array(kaper)[chosen])
    col24 = fits.Column(name='Aper_K_err',format='E',unit='ADU',array=np.array(kapererr)[chosen])
    col25 = fits.Column(name='YND_zy',format='11A',unit='Flag',array=np.array(YND_zy)[chosen])
    col26 = fits.Column(name='YND_zj',format='11A',unit='Flag',array=np.array(YND_zj)[chosen])
    col27 = fits.Column(name='YND_zh',format='11A',unit='Flag',array=np.array(YND_zh)[chosen])
    col28 = fits.Column(name='YND_zk',format='11A',unit='Flag',array=np.array(YND_zk)[chosen])
    col29 = fits.Column(name='YND_yj',format='11A',unit='Flag',array=np.array(YND_yj)[chosen])
    col30 = fits.Column(name='YND_yh',format='11A',unit='Flag',array=np.array(YND_yh)[chosen])
    col31 = fits.Column(name='YND_yk',format='11A',unit='Flag',array=np.array(YND_yk)[chosen])
    col32 = fits.Column(name='YND_jh',format='11A',unit='Flag',array=np.array(YND_jh)[chosen])
    col33 = fits.Column(name='YND_jk',format='11A',unit='Flag',array=np.array(YND_jk)[chosen])
    col34 = fits.Column(name='YND_hk',format='11A',unit='Flag',array=np.array(YND_hk)[chosen])
    col35 = fits.Column(name='YNC_zy',format='11A',unit='Flag',array=np.array(YNC_zy)[chosen])
    col36 = fits.Column(name='YNC_zj',format='11A',unit='Flag',array=np.array(YNC_zj)[chosen])
    col37 = fits.Column(name='YNC_zh',format='11A',unit='Flag',array=np.array(YNC_zh)[chosen])
    col38 = fits.Column(name='YNC_zk',format='11A',unit='Flag',array=np.array(YNC_zk)[chosen])
    col39 = fits.Column(name='YNC_yj',format='11A',unit='Flag',array=np.array(YNC_yj)[chosen])
    col40 = fits.Column(name='YNC_yh',format='11A',unit='Flag',array=np.array(YNC_yh)[chosen])
    col41 = fits.Column(name='YNC_yk',format='11A',unit='Flag',array=np.array(YNC_yk)[chosen])
    col42 = fits.Column(name='YNC_jh',format='11A',unit='Flag',array=np.array(YNC_jh)[chosen])
    col43 = fits.Column(name='YNC_jk',format='11A',unit='Flag',array=np.array(YNC_jk)[chosen])
    col44 = fits.Column(name='YNC_hk',format='11A',unit='Flag',array=np.array(YNC_hk)[chosen])
    col45 = fits.Column(name='YFD_zy',format='11A',unit='Flag',array=np.array(YFD_zy)[chosen])
    col46 = fits.Column(name='YFD_zj',format='11A',unit='Flag',array=np.array(YFD_zj)[chosen])
    col47 = fits.Column(name='YFD_zh',format='11A',unit='Flag',array=np.array(YFD_zh)[chosen])
    col48 = fits.Column(name='YFD_zk',format='11A',unit='Flag',array=np.array(YFD_zk)[chosen])
    col49 = fits.Column(name='YFD_yj',format='11A',unit='Flag',array=np.array(YFD_yj)[chosen])
    col50 = fits.Column(name='YFD_yh',format='11A',unit='Flag',array=np.array(YFD_yh)[chosen])
    col51 = fits.Column(name='YFD_yk',format='11A',unit='Flag',array=np.array(YFD_yk)[chosen])
    col52 = fits.Column(name='YFD_jh',format='11A',unit='Flag',array=np.array(YFD_jh)[chosen])
    col53 = fits.Column(name='YFD_jk',format='11A',unit='Flag',array=np.array(YFD_jk)[chosen])
    col54 = fits.Column(name='YFD_hk',format='11A',unit='Flag',array=np.array(YFD_hk)[chosen])
    col55 = fits.Column(name='YFC_zy',format='11A',unit='Flag',array=np.array(YFC_zy)[chosen])
    col56 = fits.Column(name='YFC_zj',format='11A',unit='Flag',array=np.array(YFC_zj)[chosen])
    col57 = fits.Column(name='YFC_zh',format='11A',unit='Flag',array=np.array(YFC_zh)[chosen])
    col58 = fits.Column(name='YFC_zk',format='11A',unit='Flag',array=np.array(YFC_zk)[chosen])
    col59 = fits.Column(name='YFC_yj',format='11A',unit='Flag',array=np.array(YFC_yj)[chosen])
    col60 = fits.Column(name='YFC_yh',format='11A',unit='Flag',array=np.array(YFC_yh)[chosen])
    col61 = fits.Column(name='YFC_yk',format='11A',unit='Flag',array=np.array(YFC_yk)[chosen])
    col62 = fits.Column(name='YFC_jh',format='11A',unit='Flag',array=np.array(YFC_jh)[chosen])
    col63 = fits.Column(name='YFC_jk',format='11A',unit='Flag',array=np.array(YFC_jk)[chosen])
    col64 = fits.Column(name='YFC_hk',format='11A',unit='Flag',array=np.array(YFC_hk)[chosen])
    col65 = fits.Column(name='OND_zy',format='11A',unit='Flag',array=np.array(OND_zy)[chosen])
    col66 = fits.Column(name='OND_zj',format='11A',unit='Flag',array=np.array(OND_zj)[chosen])
    col67 = fits.Column(name='OND_zh',format='11A',unit='Flag',array=np.array(OND_zh)[chosen])
    col68 = fits.Column(name='OND_zk',format='11A',unit='Flag',array=np.array(OND_zk)[chosen])
    col69 = fits.Column(name='OND_yj',format='11A',unit='Flag',array=np.array(OND_yj)[chosen])
    col70 = fits.Column(name='OND_yh',format='11A',unit='Flag',array=np.array(OND_yh)[chosen])
    col71 = fits.Column(name='OND_yk',format='11A',unit='Flag',array=np.array(OND_yk)[chosen])
    col72 = fits.Column(name='OND_jh',format='11A',unit='Flag',array=np.array(OND_jh)[chosen])
    col73 = fits.Column(name='OND_jk',format='11A',unit='Flag',array=np.array(OND_jk)[chosen])
    col74 = fits.Column(name='OND_hk',format='11A',unit='Flag',array=np.array(OND_hk)[chosen])
    col75 = fits.Column(name='ONC_zy',format='11A',unit='Flag',array=np.array(ONC_zy)[chosen])
    col76 = fits.Column(name='ONC_zj',format='11A',unit='Flag',array=np.array(ONC_zj)[chosen])
    col77 = fits.Column(name='ONC_zh',format='11A',unit='Flag',array=np.array(ONC_zh)[chosen])
    col78 = fits.Column(name='ONC_zk',format='11A',unit='Flag',array=np.array(ONC_zk)[chosen])
    col79 = fits.Column(name='ONC_yj',format='11A',unit='Flag',array=np.array(ONC_yj)[chosen])
    col80 = fits.Column(name='ONC_yh',format='11A',unit='Flag',array=np.array(ONC_yh)[chosen])
    col81 = fits.Column(name='ONC_yk',format='11A',unit='Flag',array=np.array(ONC_yk)[chosen])
    col82 = fits.Column(name='ONC_jh',format='11A',unit='Flag',array=np.array(ONC_jh)[chosen])
    col83 = fits.Column(name='ONC_jk',format='11A',unit='Flag',array=np.array(ONC_jk)[chosen])
    col84 = fits.Column(name='ONC_hk',format='11A',unit='Flag',array=np.array(ONC_hk)[chosen])
    col85 = fits.Column(name='OFD_zy',format='11A',unit='Flag',array=np.array(OFD_zy)[chosen])
    col86 = fits.Column(name='OFD_zj',format='11A',unit='Flag',array=np.array(OFD_zj)[chosen])
    col87 = fits.Column(name='OFD_zh',format='11A',unit='Flag',array=np.array(OFD_zh)[chosen])
    col88 = fits.Column(name='OFD_zk',format='11A',unit='Flag',array=np.array(OFD_zk)[chosen])
    col89 = fits.Column(name='OFD_yj',format='11A',unit='Flag',array=np.array(OFD_yj)[chosen])
    col90 = fits.Column(name='OFD_yh',format='11A',unit='Flag',array=np.array(OFD_yh)[chosen])
    col91 = fits.Column(name='OFD_yk',format='11A',unit='Flag',array=np.array(OFD_yk)[chosen])
    col92 = fits.Column(name='OFD_jh',format='11A',unit='Flag',array=np.array(OFD_jh)[chosen])
    col93 = fits.Column(name='OFD_jk',format='11A',unit='Flag',array=np.array(OFD_jk)[chosen])
    col94 = fits.Column(name='OFD_hk',format='11A',unit='Flag',array=np.array(OFD_hk)[chosen])
    col95 = fits.Column(name='OFC_zy',format='11A',unit='Flag',array=np.array(OFC_zy)[chosen])
    col96 = fits.Column(name='OFC_zj',format='11A',unit='Flag',array=np.array(OFC_zj)[chosen])
    col97 = fits.Column(name='OFC_zh',format='11A',unit='Flag',array=np.array(OFC_zh)[chosen])
    col98 = fits.Column(name='OFC_zk',format='11A',unit='Flag',array=np.array(OFC_zk)[chosen])
    col99 = fits.Column(name='OFC_yj',format='11A',unit='Flag',array=np.array(OFC_yj)[chosen])
    col100 = fits.Column(name='OFC_yh',format='11A',unit='Flag',array=np.array(OFC_yh)[chosen])
    col101 = fits.Column(name='OFC_yk',format='11A',unit='Flag',array=np.array(OFC_yk)[chosen])
    col102 = fits.Column(name='OFC_jh',format='11A',unit='Flag',array=np.array(OFC_jh)[chosen])
    col103 = fits.Column(name='OFC_jk',format='11A',unit='Flag',array=np.array(OFC_jk)[chosen])
    col104 = fits.Column(name='OFC_hk',format='11A',unit='Flag',array=np.array(OFC_hk)[chosen])
    col105 = fits.Column(name='YND Total',format='F',unit='Counts',array=np.array(YND)[chosen])
    col106 = fits.Column(name='YNC Total',format='F',unit='Counts',array=np.array(YNC)[chosen])
    col107 = fits.Column(name='YFD Total',format='F',unit='Counts',array=np.array(YFD)[chosen])
    col108 = fits.Column(name='YFC Total',format='F',unit='Counts',array=np.array(YFC)[chosen])
    col109 = fits.Column(name='OND Total',format='F',unit='Counts',array=np.array(OND)[chosen])
    col110 = fits.Column(name='ONC Total',format='F',unit='Counts',array=np.array(ONC)[chosen])
    col111 = fits.Column(name='OFD Total',format='F',unit='Counts',array=np.array(OFD)[chosen])
    col112 = fits.Column(name='OFC Total',format='F',unit='Counts',array=np.array(OFC)[chosen])
    col132 = fits.Column(name='YND Perc',format='F',unit='Percentage',array=np.array(YNDperc)[chosen])
    col133 = fits.Column(name='YNC Perc',format='F',unit='Percentage',array=np.array(YNCperc)[chosen])
    col134 = fits.Column(name='YFD Perc',format='F',unit='Percentage',array=np.array(YFDperc)[chosen])
    col135 = fits.Column(name='YFC Perc',format='F',unit='Percentage',array=np.array(YFCperc)[chosen])
    col136 = fits.Column(name='OND Perc',format='F',unit='Percentage',array=np.array(ONDperc)[chosen])
    col137 = fits.Column(name='ONC Perc',format='F',unit='Percentage',array=np.array(ONCperc)[chosen])
    col138 = fits.Column(name='OFD Perc',format='F',unit='Percentage',array=np.array(OFDperc)[chosen])
    col139 = fits.Column(name='OFC Perc',format='F',unit='Percentage',array=np.array(OFCperc)[chosen])
    col113 = fits.Column(name='Y Total',format='F',unit='Counts',array=np.array(Y)[chosen])
    col114 = fits.Column(name='O Total',format='F',unit='Counts',array=np.array(O)[chosen])
    col140 = fits.Column(name='Age Diff',format='F',unit='Counts',array=np.array(AgeDiff)[chosen])
    col115 = fits.Column(name='N Total',format='F',unit='Counts',array=np.array(N)[chosen])
    col116 = fits.Column(name='F Total',format='F',unit='Counts',array=np.array(F)[chosen])
    col141 = fits.Column(name='Distance Diff',format='F',unit='Counts',array=np.array(DisDiff)[chosen])
    col117 = fits.Column(name='D Total',format='F',unit='Counts',array=np.array(D)[chosen])
    col118 = fits.Column(name='C Total',format='F',unit='Counts',array=np.array(C)[chosen])
    col142 = fits.Column(name='Model Diff',format='F',unit='Counts',array=np.array(ModDiff)[chosen])
    col119 = fits.Column(name='ZY Total',format='F',unit='Counts',array=np.array(ZY)[chosen])
    col120 = fits.Column(name='ZJ Total',format='F',unit='Counts',array=np.array(ZJ)[chosen])
    col121 = fits.Column(name='ZH Total',format='F',unit='Counts',array=np.array(ZH)[chosen])
    col122 = fits.Column(name='ZK Total',format='F',unit='Counts',array=np.array(ZK)[chosen])
    col123 = fits.Column(name='YJ Total',format='F',unit='Counts',array=np.array(YJ)[chosen])
    col124 = fits.Column(name='YH Total',format='F',unit='Counts',array=np.array(YH)[chosen])
    col125 = fits.Column(name='YK Total',format='F',unit='Counts',array=np.array(YK)[chosen])
    col126 = fits.Column(name='JH Total',format='F',unit='Counts',array=np.array(JH)[chosen])
    col127 = fits.Column(name='JK Total',format='F',unit='Counts',array=np.array(JK)[chosen])
    col128 = fits.Column(name='HK Total',format='F',unit='Counts',array=np.array(HK)[chosen])
    col129 = fits.Column(name='Filters',format='F',unit='Number',array=np.array(Filters)[chosen])
    col130 = fits.Column(name='Total Counts',format='F',unit='Number',array=np.array(Total)[chosen])
    col131 = fits.Column(name='Percentage Count',format='F',unit='Percentage',array=np.array(Percentage)[chosen])
    col143 = fits.Column(name='Max_Percentage',format='F',unit='Percentage',array=np.array(Max_Perc)[chosen])

    cols = fits.ColDefs([col1,col2,col3,col4,col3a,col4a,col5,col6,col7,col8,col9,col10,
                         col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,
                         col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,
                         col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,
                         col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,
                         col51,col52,col53,col54,col55,col56,col57,col58,col59,col60,
                         col61,col62,col63,col64,col65,col66,col67,col68,col69,col70,
                         col71,col72,col73,col74,col75,col76,col77,col78,col79,col80,
                         col81,col82,col83,col84,col85,col86,col87,col88,col89,col90,
                         col91,col92,col93,col94,col95,col96,col97,col98,col99,col100,
                         col101,col102,col103,col104,col105,col132,col106,col133,col107,
                         col134,col108,col135,col109,col136,col110,col137,col111,col138,
                         col112,col139,col113,col114,col140,col115,col116,col141,col117,
                         col118,col142,col119,col120,col121,col122,col123,col124,col125,
                         col126,col127,col128,col129,col130,col131,col143])
    
    '''Here we make the fits file'''

    data = fits.BinTableHDU.from_columns(cols)
    data.writeto(output_fits)

    '''Now we make the reg file'''

    pos_data = []
    RA_dms = np.array(RA_dms)
    Dec_dms = np.array(Dec_dms)

    for v in range(len(chosen)):

        pos_data.append([RA_dms[chosen][v],Dec_dms[chosen][v]])

    with open(output_reg, 'w', newline='') as csvfile:
        reg_file = csv.writer(csvfile, delimiter=',', quotechar=' ', quoting=csv.QUOTE_ALL)
        for w in range(len(pos_data)):
            reg_file.writerow([pos_data[w][0] , pos_data[w][1]])
            
def Distance(d=10):
    mu = 5*math.log(d,10)-5
    return mu

In [ ]:
vlm_object_probability('0606_final.fits','model_iso.fits','0607_final')

In [ ]:
vlm_object_added('0606_final.fits','model_iso.fits','0607_final_added')

In [ ]:
mu_far = Distance(420)
mu_near = Distance(350)

In [ ]:
bd_range_1myr = range(1300,2931)
full_range = range(1300,4000)
full_rangey = range(1300,4000)
full_rangeo = range(1600,4000)
full_rangec = range(1300,4000)
full_ranged = range(1650,4000)

Av = 1.1

Az = 0.5006359319 * Av
Ay = 0.3913227425 * Av
Aj = 0.281344478 * Av
Ah = 0.1812818241 * Av
Ak = 0.1180678823 * Av

Azy = Az - Ay
Azj = Az - Aj
Azh = Az - Ah
Azk = Az - Ak
Ayj = Ay - Aj
Ayh = Ay - Ah
Ayk = Ay - Ak
Ajh = Aj - Ah
Ajk = Aj - Ak
Ahk = Ah - Ak

In [ ]:
print(Azj)
print(Az)
print(mu_far)

In [ ]:
info = fits.open('final_3filt_wmissing_cut.fits')

ra = info[1].data['RA']
dec = info[1].data['Dec']
ra2000 = info[1].data['RA2000']
dec2000 = info[1].data['Dec2000']
mag_z = info[1].data['Mag_Z']
obs_mag_z = info[1].data['Obs_Mag_Z']
aper_z = info[1].data['Aper_Z']
aper_z_err = info[1].data['Aper_Z_err']
mag_y = info[1].data['Mag_Y']
obs_mag_y = info[1].data['Obs_Mag_Y']
aper_y = info[1].data['Aper_Y']
aper_y_err = info[1].data['Aper_Y_err']
mag_j = info[1].data['Mag_J']
obs_mag_j = info[1].data['Obs_Mag_J']
aper_j = info[1].data['Aper_J']
aper_j_err = info[1].data['Aper_J_err']
mag_h = info[1].data['Mag_H']
obs_mag_h = info[1].data['Obs_Mag_H']
aper_h = info[1].data['Aper_H']
aper_h_err = info[1].data['Aper_H_err']
mag_k = info[1].data['Mag_K']
obs_mag_k = info[1].data['Obs_Mag_K']
aper_k = info[1].data['Aper_K']
aper_k_err = info[1].data['Aper_K_err']

In [ ]:
model = fits.open('model_iso.fits')

mass1 = model[1].data['Mass'][0:28]
teff1 = model[1].data['Teff'][0:28]
zmd1 = model[1].data['ZmagD'][0:28]
ymd1 = model[1].data['YmagD'][0:28]
jmd1 = model[1].data['JmagD'][0:28]
hmd1 = model[1].data['HmagD'][0:28]
kmd1 = model[1].data['KsmagD'][0:28]
zmc1 = model[1].data['ZmagC'][0:28]
ymc1 = model[1].data['YmagC'][0:28]
jmc1 = model[1].data['JmagC'][0:28]
hmc1 = model[1].data['HmagC'][0:28]
kmc1 = model[1].data['KsmagC'][0:28]

mass_fit = np.polyfit(teff1, mass1, 14)
mass_poly = np.poly1d(mass_fit)
teff_fit = np.polyfit(mass1, teff1, 14)
teff_poly = np.poly1d(teff_fit)
zc_fit = np.polyfit(teff1, zmc1, 14)
zc_poly = np.poly1d(zc_fit)
zd_fit = np.polyfit(teff1, zmd1, 14)
zd_poly = np.poly1d(zd_fit)
yc_fit = np.polyfit(teff1, ymc1, 14)
yc_poly = np.poly1d(yc_fit)
yd_fit = np.polyfit(teff1, ymd1, 14)
yd_poly = np.poly1d(yd_fit)
jc_fit = np.polyfit(teff1, jmc1, 14)
jc_poly = np.poly1d(jc_fit)
jd_fit = np.polyfit(teff1, jmd1, 14)
jd_poly = np.poly1d(jd_fit)
hc_fit = np.polyfit(teff1, hmc1, 14)
hc_poly = np.poly1d(hc_fit)
hd_fit = np.polyfit(teff1, hmd1, 14)
hd_poly = np.poly1d(hd_fit)
kc_fit = np.polyfit(teff1, kmc1, 14)
kc_poly = np.poly1d(kc_fit)
kd_fit = np.polyfit(teff1, kmd1, 14)
kd_poly = np.poly1d(kd_fit)

In [ ]:
model = fits.open('model_iso.fits')

mass5 = model[1].data['Mass'][28:57]
teff5 = model[1].data['Teff'][28:57]
zmd5 = model[1].data['ZmagD'][28:57]
ymd5 = model[1].data['YmagD'][28:57]
jmd5 = model[1].data['JmagD'][28:57]
hmd5 = model[1].data['HmagD'][28:57]
kmd5 = model[1].data['KsmagD'][28:57]
zmc5 = model[1].data['ZmagC'][28:57]
ymc5 = model[1].data['YmagC'][28:57]
jmc5 = model[1].data['JmagC'][28:57]
hmc5 = model[1].data['HmagC'][28:57]
kmc5 = model[1].data['KsmagC'][28:57]


masso_fit = np.polyfit(teff5, mass5, 9)
masso_poly = np.poly1d(masso_fit)
zco_fit = np.polyfit(teff5, zmc5, 9)
zco_poly = np.poly1d(zco_fit)
zdo_fit = np.polyfit(teff5, zmd5, 9)
zdo_poly = np.poly1d(zdo_fit)
yco_fit = np.polyfit(teff5, ymc5, 9)
yco_poly = np.poly1d(yco_fit)
ydo_fit = np.polyfit(teff5, ymd5, 9)
ydo_poly = np.poly1d(ydo_fit)
jco_fit = np.polyfit(teff5, jmc5, 9)
jco_poly = np.poly1d(jco_fit)
jdo_fit = np.polyfit(teff5, jmd5, 9)
jdo_poly = np.poly1d(jdo_fit)
hco_fit = np.polyfit(teff5, hmc5, 9)
hco_poly = np.poly1d(hco_fit)
hdo_fit = np.polyfit(teff5, hmd5, 9)
hdo_poly = np.poly1d(hdo_fit)
kco_fit = np.polyfit(teff5, kmc5, 9)
kco_poly = np.poly1d(kco_fit)
kdo_fit = np.polyfit(teff5, kmd5, 9)
kdo_poly = np.poly1d(kdo_fit)

In [ ]:
err_z = aper_z_err/(aper_z)
err_y = aper_y_err/(aper_y)
err_j = aper_j_err/(aper_j)
err_h = aper_h_err/(aper_h)
err_k = aper_k_err/(aper_k)

In [ ]:
p = 8

idz = np.isfinite(mag_z) & np.isfinite(err_z)
zerr_fit = np.polyfit(mag_z[idz],err_z[idz],p)
zerr_poly = np.poly1d(zerr_fit)
idy = np.isfinite(mag_y) & np.isfinite(err_y)
yerr_fit = np.polyfit(mag_y[idy],err_y[idy],p)
yerr_poly = np.poly1d(yerr_fit)
idj = np.isfinite(mag_j) & np.isfinite(err_j)
jerr_fit = np.polyfit(mag_j[idj],err_j[idj],p)
jerr_poly = np.poly1d(jerr_fit)
idh = np.isfinite(mag_h) & np.isfinite(err_h)
herr_fit = np.polyfit(mag_h[idh],err_h[idh],p)
herr_poly = np.poly1d(herr_fit)
idk = np.isfinite(mag_k) & np.isfinite(err_k)
kerr_fit = np.polyfit(mag_k[idk],err_k[idk],p)
kerr_poly = np.poly1d(kerr_fit)

In [ ]:
'''I used this cell originally to calculate the temperatures of certain magnitude sources (y)'''

y = 11
temps = []

x = 1600
while (zd_poly(x) + mu_near + zerr_poly(zd_poly(x) + mu_near)) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('1:',x)

x = 1600
while zd_poly(x) + mu_near + Az - zerr_poly(zd_poly(x) + mu_near + Az) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('2:',x)

x = 1600
while zc_poly(x) + mu_near + zerr_poly(zc_poly(x) + mu_near) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('3:',x)

x = 1600
while zc_poly(x) + mu_near + Az - zerr_poly(zc_poly(x) + mu_near + Az) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('4:',x)

x = 1600
while zd_poly(x) + mu_far + zerr_poly(zd_poly(x) + mu_far) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('5:',x)

x = 1600
while zd_poly(x) + mu_far + Az - zerr_poly(zd_poly(x) + mu_far + Az) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('6:',x)

x = 1600
while zc_poly(x) + mu_far + zerr_poly(zc_poly(x) + mu_far) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('7:',x)

x = 1600
while zc_poly(x) + mu_far + Az - zerr_poly(zc_poly(x) + mu_far + Az) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('8:',x)

x = 1600
while zdo_poly(x) + mu_near + zerr_poly(zdo_poly(x) + mu_near) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('9:',x)

x = 1600
while zdo_poly(x) + mu_near + Az - zerr_poly(zdo_poly(x) + mu_near + Az) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('10:',x)

x = 1600
while zco_poly(x) + mu_near + zerr_poly(zco_poly(x) + mu_near) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('11:',x)

x = 1600
while zco_poly(x) + mu_near + Az - zerr_poly(zco_poly(x) + mu_near + Az) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('12:',x)

x = 1600
while zdo_poly(x) + mu_far + zerr_poly(zdo_poly(x) + mu_far) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('13:',x)

x = 1600
while zdo_poly(x) + mu_far + Az - zerr_poly(zdo_poly(x) + mu_far + Az) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('14:',x)

x = 1600
while zco_poly(x) + mu_far + zerr_poly(zco_poly(x) + mu_far) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('15:',x)

x = 1600
while zco_poly(x) + mu_far + Az - zerr_poly(zco_poly(x) + mu_far + Az) > y and x < 4500:
    x = x+1    
if x!=1600:
    temps.append(x)
    print('16:',x)

In [ ]:
brown_tempy = 2931.6
brown_tempo = 2971.6
tempy = 2306.4
tempo = 2068.6

In [ ]:
'''I used the border temperatures found just above to then determine the corresponding magnitudes in each set-up.'''
ynd_l_x = zd_poly(tempy) - jd_poly(tempy) - np.sqrt(zerr_poly(zd_poly(tempy) + mu_near)**2 + jerr_poly(jd_poly(tempy) + mu_near)**2)
ynd_l_y = zd_poly(tempy) + mu_near + zerr_poly(zd_poly(tempy) + mu_near)
ynd_r_x = zd_poly(tempy) - jd_poly(tempy) + Azj + np.sqrt(zerr_poly(zd_poly(tempy) + mu_near + Az)**2 + jerr_poly(jd_poly(tempy) + mu_near + Aj)**2)
ynd_r_y = zd_poly(tempy) + mu_near + Az - zerr_poly(zd_poly(tempy) + mu_near + Az)
ync_l_x = zc_poly(tempy) - jc_poly(tempy) - np.sqrt(zerr_poly(zc_poly(tempy) + mu_near)**2 + jerr_poly(jc_poly(tempy) + mu_near)**2)
ync_l_y = zc_poly(tempy) + mu_near + zerr_poly(zc_poly(tempy) + mu_near)
ync_r_x = zc_poly(tempy) - jc_poly(tempy) + Azj + np.sqrt(zerr_poly(zc_poly(tempy) + mu_near + Az)**2 + jerr_poly(jc_poly(tempy) + mu_near + Aj)**2)
ync_r_y = zc_poly(tempy) + mu_near + Az - zerr_poly(zc_poly(tempy) + mu_near + Az)
yfd_l_x = zd_poly(tempy) - jd_poly(tempy) - np.sqrt(zerr_poly(zd_poly(tempy) + mu_far)**2 + jerr_poly(jd_poly(tempy) + mu_far)**2)
yfd_l_y = zd_poly(tempy) + mu_far + zerr_poly(zd_poly(tempy) + mu_far)
yfd_r_x = zd_poly(tempy) - jd_poly(tempy) + Azj + np.sqrt(zerr_poly(zd_poly(tempy) + mu_far + Az)**2 + jerr_poly(jd_poly(tempy) + mu_far + Aj)**2)
yfd_r_y = zd_poly(tempy) + mu_far + Az - zerr_poly(zd_poly(tempy) + mu_far + Az)
yfc_l_x = zc_poly(tempy) - jc_poly(tempy) - np.sqrt(zerr_poly(zc_poly(tempy) + mu_far)**2 + jerr_poly(jc_poly(tempy) + mu_far)**2)
yfc_l_y = zc_poly(tempy) + mu_far + zerr_poly(zc_poly(tempy) + mu_far)
yfc_r_x = zc_poly(tempy) - jc_poly(tempy) + Azj + np.sqrt(zerr_poly(zc_poly(tempy) + mu_far + Az)**2 + jerr_poly(jc_poly(tempy) + mu_far + Aj)**2)
yfc_r_y = zc_poly(tempy) + mu_far + Az - zerr_poly(zc_poly(tempy) + mu_far + Az)
ond_l_x = zdo_poly(tempo) - jdo_poly(tempo) - np.sqrt(zerr_poly(zdo_poly(tempo) + mu_near)**2 + jerr_poly(jdo_poly(tempo) + mu_near)**2)
ond_l_y = zdo_poly(tempo) + mu_near + zerr_poly(zdo_poly(tempo) + mu_near)
ond_r_x = zdo_poly(tempo) - jdo_poly(tempo) + Azj + np.sqrt(zerr_poly(zdo_poly(tempo) + mu_near + Az)**2 + jerr_poly(jdo_poly(tempo) + mu_near + Aj)**2)
ond_r_y = zdo_poly(tempo) + mu_near + Az - zerr_poly(zdo_poly(tempo) + mu_near + Az)
onc_l_x = zco_poly(tempo) - jco_poly(tempo) - np.sqrt(zerr_poly(zco_poly(tempo) + mu_near)**2 + jerr_poly(jco_poly(tempo) + mu_near)**2)
onc_l_y = zco_poly(tempo) + mu_near + zerr_poly(zco_poly(tempo) + mu_near)
onc_r_x = zco_poly(tempo) - jco_poly(tempo) + Azj + np.sqrt(zerr_poly(zco_poly(tempo) + mu_near + Az)**2 + jerr_poly(jco_poly(tempo) + mu_near + Aj)**2)
onc_r_y = zco_poly(tempo) + mu_near + Az - zerr_poly(zco_poly(tempo) + mu_near + Az)
ofd_l_x = zdo_poly(tempo) - jdo_poly(tempo) - np.sqrt(zerr_poly(zdo_poly(tempo) + mu_far)**2 + jerr_poly(jdo_poly(tempo) + mu_far)**2)
ofd_l_y = zdo_poly(tempo) + mu_far + zerr_poly(zdo_poly(tempo) + mu_far)
ofd_r_x = zdo_poly(tempo) - jdo_poly(tempo) + Azj + np.sqrt(zerr_poly(zdo_poly(tempo) + mu_far + Az)**2 + jerr_poly(jdo_poly(tempo) + mu_far + Aj)**2)
ofd_r_y = zdo_poly(tempo) + mu_far + Az - zerr_poly(zdo_poly(tempo) + mu_far + Az)
ofc_l_x = zco_poly(tempo) - jco_poly(tempo) - np.sqrt(zerr_poly(zco_poly(tempo) + mu_far)**2 + jerr_poly(jco_poly(tempo) + mu_far)**2)
ofc_l_y = zco_poly(tempo) + mu_far + zerr_poly(zco_poly(tempo) + mu_far)
ofc_r_x = zco_poly(tempo) - jco_poly(tempo) + Azj + np.sqrt(zerr_poly(zco_poly(tempo) + mu_far + Az)**2 + jerr_poly(jco_poly(tempo) + mu_far + Aj)**2)
ofc_r_y = zco_poly(tempo) + mu_far + Az - zerr_poly(zco_poly(tempo) + mu_far + Az)

In [ ]:
ynd_l_x_brown = zd_poly(brown_tempy) - jd_poly(brown_tempy) - np.sqrt(zerr_poly(zd_poly(brown_tempy) + mu_near)**2 + jerr_poly(jd_poly(brown_tempy) + mu_near)**2)
ynd_l_y_brown = zd_poly(brown_tempy) + mu_near + zerr_poly(zd_poly(brown_tempy) + mu_near)
ynd_r_x_brown = zd_poly(brown_tempy) - jd_poly(brown_tempy) + Azj + np.sqrt(zerr_poly(zd_poly(brown_tempy) + mu_near + Az)**2 + jerr_poly(jd_poly(brown_tempy) + mu_near + Aj)**2)
ynd_r_y_brown = zd_poly(brown_tempy) + mu_near + Az - zerr_poly(zd_poly(brown_tempy) + mu_near + Az)
ync_l_x_brown = zc_poly(brown_tempy) - jc_poly(brown_tempy) - np.sqrt(zerr_poly(zc_poly(brown_tempy) + mu_near)**2 + jerr_poly(jc_poly(brown_tempy) + mu_near)**2)
ync_l_y_brown = zc_poly(brown_tempy) + mu_near + zerr_poly(zc_poly(brown_tempy) + mu_near)
ync_r_x_brown = zc_poly(brown_tempy) - jc_poly(brown_tempy) + Azj + np.sqrt(zerr_poly(zc_poly(brown_tempy) + mu_near + Az)**2 + jerr_poly(jc_poly(brown_tempy) + mu_near + Aj)**2)
ync_r_y_brown = zc_poly(brown_tempy) + mu_near + Az - zerr_poly(zc_poly(brown_tempy) + mu_near + Az)
yfd_l_x_brown = zd_poly(brown_tempy) - jd_poly(brown_tempy) - np.sqrt(zerr_poly(zd_poly(brown_tempy) + mu_far)**2 + jerr_poly(jd_poly(brown_tempy) + mu_far)**2)
yfd_l_y_brown = zd_poly(brown_tempy) + mu_far + zerr_poly(zd_poly(brown_tempy) + mu_far)
yfd_r_x_brown = zd_poly(brown_tempy) - jd_poly(brown_tempy) + Azj + np.sqrt(zerr_poly(zd_poly(brown_tempy) + mu_far + Az)**2 + jerr_poly(jd_poly(brown_tempy) + mu_far + Aj)**2)
yfd_r_y_brown = zd_poly(brown_tempy) + mu_far + Az - zerr_poly(zd_poly(brown_tempy) + mu_far + Az)
yfc_l_x_brown = zc_poly(brown_tempy) - jc_poly(brown_tempy) - np.sqrt(zerr_poly(zc_poly(brown_tempy) + mu_far)**2 + jerr_poly(jc_poly(brown_tempy) + mu_far)**2)
yfc_l_y_brown = zc_poly(brown_tempy) + mu_far + zerr_poly(zc_poly(brown_tempy) + mu_far)
yfc_r_x_brown = zc_poly(brown_tempy) - jc_poly(brown_tempy) + Azj + np.sqrt(zerr_poly(zc_poly(brown_tempy) + mu_far + Az)**2 + jerr_poly(jc_poly(brown_tempy) + mu_far + Aj)**2)
yfc_r_y_brown = zc_poly(brown_tempy) + mu_far + Az - zerr_poly(zc_poly(brown_tempy) + mu_far + Az)
ond_l_x_brown = zdo_poly(brown_tempo) - jdo_poly(brown_tempo) - np.sqrt(zerr_poly(zdo_poly(brown_tempo) + mu_near)**2 + jerr_poly(jdo_poly(brown_tempo) + mu_near)**2)
ond_l_y_brown = zdo_poly(brown_tempo) + mu_near + zerr_poly(zdo_poly(brown_tempo) + mu_near)
ond_r_x_brown = zdo_poly(brown_tempo) - jdo_poly(brown_tempo) + Azj + np.sqrt(zerr_poly(zdo_poly(brown_tempo) + mu_near + Az)**2 + jerr_poly(jdo_poly(brown_tempo) + mu_near + Aj)**2)
ond_r_y_brown = zdo_poly(brown_tempo) + mu_near + Az - zerr_poly(zdo_poly(brown_tempo) + mu_near + Az)
onc_l_x_brown = zco_poly(brown_tempo) - jco_poly(brown_tempo) - np.sqrt(zerr_poly(zco_poly(brown_tempo) + mu_near)**2 + jerr_poly(jco_poly(brown_tempo) + mu_near)**2)
onc_l_y_brown = zco_poly(brown_tempo) + mu_near + zerr_poly(zco_poly(brown_tempo) + mu_near)
onc_r_x_brown = zco_poly(brown_tempo) - jco_poly(brown_tempo) + Azj + np.sqrt(zerr_poly(zco_poly(brown_tempo) + mu_near + Az)**2 + jerr_poly(jco_poly(brown_tempo) + mu_near + Aj)**2)
onc_r_y_brown = zco_poly(brown_tempo) + mu_near + Az - zerr_poly(zco_poly(brown_tempo) + mu_near + Az)
ofd_l_x_brown = zdo_poly(brown_tempo) - jdo_poly(brown_tempo) - np.sqrt(zerr_poly(zdo_poly(brown_tempo) + mu_far)**2 + jerr_poly(jdo_poly(brown_tempo) + mu_far)**2)
ofd_l_y_brown = zdo_poly(brown_tempo) + mu_far + zerr_poly(zdo_poly(brown_tempo) + mu_far)
ofd_r_x_brown = zdo_poly(brown_tempo) - jdo_poly(brown_tempo) + Azj + np.sqrt(zerr_poly(zdo_poly(brown_tempo) + mu_far + Az)**2 + jerr_poly(jdo_poly(brown_tempo) + mu_far + Aj)**2)
ofd_r_y_brown = zdo_poly(brown_tempo) + mu_far + Az - zerr_poly(zdo_poly(brown_tempo) + mu_far + Az)
ofc_l_x_brown = zco_poly(brown_tempo) - jco_poly(brown_tempo) - np.sqrt(zerr_poly(zco_poly(brown_tempo) + mu_far)**2 + jerr_poly(jco_poly(brown_tempo) + mu_far)**2)
ofc_l_y_brown = zco_poly(brown_tempo) + mu_far + zerr_poly(zco_poly(brown_tempo) + mu_far)
ofc_r_x_brown = zco_poly(brown_tempo) - jco_poly(brown_tempo) + Azj + np.sqrt(zerr_poly(zco_poly(brown_tempo) + mu_far + Az)**2 + jerr_poly(jco_poly(brown_tempo) + mu_far + Aj)**2)
ofc_r_y_brown = zco_poly(brown_tempo) + mu_far + Az - zerr_poly(zco_poly(brown_tempo) + mu_far + Az)

In [ ]:
ipmo_mag = (ynd_l_y,ynd_r_y,ync_l_y,ync_r_y,yfd_l_y,yfd_r_y,yfc_l_y,yfc_r_y,ond_l_y,ond_r_y,onc_l_y,onc_r_y,ofd_l_y,ofd_r_y,ofc_l_y,ofc_r_y)
print(max(ipmo_mag))
print(min(ipmo_mag))

In [ ]:
bd_mag = (ynd_l_y_brown,ynd_r_y_brown,ync_l_y_brown,ync_r_y_brown,yfd_l_y_brown,yfd_r_y_brown,yfc_l_y_brown,yfc_r_y_brown,ond_l_y_brown,ond_r_y_brown,onc_l_y_brown,onc_r_y_brown,ofd_l_y_brown,ofd_r_y_brown,ofc_l_y_brown,ofc_r_y_brown)
print(max(bd_mag))
print(min(bd_mag))

In [ ]:
'An example of how the error affects the main model line'''
plt.figure(figsize=(40,20))
plt.plot(mag_z-mag_j,mag_z,'.')
plt.plot(np.sqrt(zerr_poly(zc_poly(full_range)+8.074486080165673+Az)**2+jerr_poly(jc_poly(full_range)+8.074486080165673+Aj)**2),zc_poly(full_range)+8.074486080165673+0.5006359319)
plt.plot((zc_poly(full_range)-jc_poly(full_range)+(1*0.2192914539)),zc_poly(full_range)+8.074486080165673+0.5006359319)
#plt.plot((zc_poly(full_range)-jc_poly(full_range)+(0*0.2192914539))-np.sqrt(zerr_poly(zc_poly(full_range)+8.074486080165673+0.5006359319)),zc_poly(full_range)+8.074486080165673+0.5006359319)
plt.plot((zc_poly(full_range)-jc_poly(full_range)+(1*0.2192914539))-np.sqrt(zerr_poly(zc_poly(full_range)+8.074486080165673+Az)**2+jerr_poly(jc_poly(full_range)+8.074486080165673+Aj)**2),zc_poly(full_range)+8.074486080165673+0.5006359319)
plt.plot((zc_poly(full_range)-jc_poly(full_range)+(1*0.2192914539))+np.sqrt(zerr_poly(zc_poly(full_range)+8.074486080165673+Az)**2+jerr_poly(jc_poly(full_range)+8.074486080165673+Aj)**2),zc_poly(full_range)+8.074486080165673+0.5006359319)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
'''This one shows extinction effects'''
plt.figure(figsize=(40,20))
plt.plot(mag_z-mag_j,mag_z,'.')
plt.plot((zc_poly(full_range)-jc_poly(full_range)+(0*0.2192914539)),zc_poly(full_range)+8.074486080165673+0*0.5006359319)
plt.plot((zc_poly(full_range)-jc_poly(full_range)+(1*0.2192914539)),zc_poly(full_range)+8.074486080165673+1*0.5006359319)
plt.plot((zc_poly(full_range)-jc_poly(full_range)+(2*0.2192914539)),zc_poly(full_range)+8.074486080165673+2*0.5006359319)
#plt.ylim(21,24)
plt.gca().invert_yaxis()
#plt.xlim(1.5,3.5)
plt.show()

In [ ]:
bd_range_1myr = range(1300,2931)
full_range = range(1300,4000)
full_rangey = range(1300,4000)
full_rangeo = range(1600,4000)
full_rangec = range(1300,4000)
full_ranged = range(1650,4000)

In [ ]:
'''This and the next 9 figures are for the various CMD combinations, with model lines'''
plt.figure(figsize=(40,20))
plt.rc('xtick',labelsize=28)
plt.rc('ytick',labelsize=28)
plt.plot(mag_z-mag_y,mag_z,'.',color='grey')
plt.plot(zd_poly(full_ranged)-yd_poly(full_ranged) - np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_near)**2 + yerr_poly(yd_poly(full_ranged) + mu_near)**2),zd_poly(full_ranged) + mu_near + zerr_poly(zd_poly(full_ranged) + mu_near),color = 'C0',label='1 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(zd_poly(full_ranged)-yd_poly(full_ranged) + Azy + np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_near + Az)**2 + yerr_poly(yd_poly(full_ranged) + mu_near + Ay)**2), zd_poly(full_ranged) + mu_near + Az - zerr_poly(zd_poly(full_ranged) + mu_near + Az),color = 'C0',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-yc_poly(full_rangec) - np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_near)**2 + yerr_poly(yc_poly(full_rangec) + mu_near)**2),zc_poly(full_rangec) + mu_near + zerr_poly(zc_poly(full_rangec) + mu_near),color = 'C1',label='1 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-yc_poly(full_rangec) + Azy + np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_near + Az)**2 + yerr_poly(yc_poly(full_rangec) + mu_near + Ay)**2) , zc_poly(full_rangec) + mu_near + Az - zerr_poly(zc_poly(full_rangec) + mu_near + Az),color = 'C1',linewidth=4.0)
plt.plot(zd_poly(full_ranged)-yd_poly(full_ranged) - np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_far)**2 + yerr_poly(yd_poly(full_ranged) + mu_far)**2),zd_poly(full_ranged) + mu_far + zerr_poly(zd_poly(full_ranged) + mu_far),color = 'C2',label='1 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(zd_poly(full_ranged)-yd_poly(full_ranged) + Azy + np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_far + Az)**2 + yerr_poly(yd_poly(full_ranged) + mu_far + Ay)**2), zd_poly(full_ranged) + mu_far + Az - zerr_poly(zd_poly(full_ranged) + mu_far + Az),color = 'C2',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-yc_poly(full_rangec) - np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_far)**2 + yerr_poly(yc_poly(full_rangec) + mu_far)**2),zc_poly(full_rangec) + mu_far + zerr_poly(zc_poly(full_rangec) + mu_far),color = 'C3',label='1 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-yc_poly(full_rangec) + Azy + np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_far + Az)**2 + yerr_poly(yc_poly(full_rangec) + mu_far + Ay)**2), zc_poly(full_rangec) + mu_far + Az - zerr_poly(zc_poly(full_rangec) + mu_far + Az),color = 'C3',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-ydo_poly(full_ranged) - np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_near)**2 + yerr_poly(ydo_poly(full_ranged) + mu_near)**2),zdo_poly(full_ranged) + mu_near + zerr_poly(zdo_poly(full_ranged) + mu_near),color = 'C4',label='5 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-ydo_poly(full_ranged) + Azy + np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_near + Az)**2 + yerr_poly(ydo_poly(full_ranged) + mu_near + Ay)**2), zdo_poly(full_ranged) + mu_near + Az - zerr_poly(zdo_poly(full_ranged) + mu_near + Az),color = 'C4',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-yco_poly(full_rangec) - np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_near)**2 + yerr_poly(yco_poly(full_rangec) + mu_near)**2),zco_poly(full_rangec) + mu_near + zerr_poly(zco_poly(full_rangec) + mu_near),color = 'C5',label='5 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-yco_poly(full_rangec) + Azy + np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_near + Az)**2 + yerr_poly(yco_poly(full_rangec) + mu_near + Ay)**2), zco_poly(full_rangec) + mu_near + Az - zerr_poly(zco_poly(full_rangec) + mu_near + Az),color = 'C5',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-ydo_poly(full_ranged) - np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_far)**2 + yerr_poly(ydo_poly(full_ranged) + mu_far)**2),zdo_poly(full_ranged) + mu_far + zerr_poly(zdo_poly(full_ranged) + mu_far),color = 'C6',label='5 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-ydo_poly(full_ranged) + Azy + np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_far + Az)**2 + yerr_poly(ydo_poly(full_ranged) + mu_far + Ay)**2), zdo_poly(full_ranged) + mu_far + Az - zerr_poly(zdo_poly(full_ranged) + mu_far + Az),color = 'C6',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-yco_poly(full_rangec) - np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_far)**2 + yerr_poly(yco_poly(full_rangec) + mu_far)**2),zco_poly(full_rangec) + mu_far + zerr_poly(zco_poly(full_rangec) + mu_far),color = 'C8',label='5 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-yco_poly(full_rangec) + Azy + np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_far + Az)**2 + yerr_poly(yco_poly(full_rangec) + mu_far + Ay)**2), zco_poly(full_rangec) + mu_far + Az - zerr_poly(zco_poly(full_rangec) + mu_far + Az),color = 'C8',linewidth=4.0)
plt.arrow(1.5,18,Azy,Az,width=0.01,length_includes_head=True,color='r')
plt.text(1.5,17.9,'$A_v$ = 1.1',fontsize=30)
plt.ylim(13,24)
plt.xlim(-0.5,2.5)
#plt.ylim(18.75,21)
#plt.xlim(2,2.5)
plt.legend(fontsize=32)
plt.gca().invert_yaxis()
plt.xlabel('Z-Y Mag', fontsize=40)
plt.ylabel('Z Mag', fontsize=40)
plt.savefig('cmd_zy.png')
plt.show()

In [ ]:
plt.figure(figsize=(40,20))
plt.rc('xtick',labelsize=28)
plt.rc('ytick',labelsize=28)
plt.plot(mag_z-mag_j,mag_z,'.',color='grey')
plt.plot(zd_poly(full_ranged)-jd_poly(full_ranged) - np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_near)**2 + jerr_poly(jd_poly(full_ranged) + mu_near)**2),zd_poly(full_ranged) + mu_near + zerr_poly(zd_poly(full_ranged) + mu_near),color = 'C0',label='1 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(zd_poly(full_ranged)-jd_poly(full_ranged) + Azj + np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_near + Az)**2 + jerr_poly(jd_poly(full_ranged) + mu_near + Aj)**2), zd_poly(full_ranged) + mu_near + Az - zerr_poly(zd_poly(full_ranged) + mu_near + Az),color = 'C0',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-jc_poly(full_rangec) - np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_near)**2 + jerr_poly(jc_poly(full_rangec) + mu_near)**2),zc_poly(full_rangec) + mu_near + zerr_poly(zc_poly(full_rangec) + mu_near),color = 'C1',label='1 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-jc_poly(full_rangec) + Azj + np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_near + Az)**2 + jerr_poly(jc_poly(full_rangec) + mu_near + Aj)**2) , zc_poly(full_rangec) + mu_near + Az - zerr_poly(zc_poly(full_rangec) + mu_near + Az),color = 'C1',linewidth=4.0)
plt.plot(zd_poly(full_ranged)-jd_poly(full_ranged) - np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_far)**2 + jerr_poly(jd_poly(full_ranged) + mu_far)**2),zd_poly(full_ranged) + mu_far + zerr_poly(zd_poly(full_ranged) + mu_far),color = 'C2',label='1 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(zd_poly(full_ranged)-jd_poly(full_ranged) + Azj + np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_far + Az)**2 + jerr_poly(jd_poly(full_ranged) + mu_far + Aj)**2), zd_poly(full_ranged) + mu_far + Az - zerr_poly(zd_poly(full_ranged) + mu_far + Az),color = 'C2',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-jc_poly(full_rangec) - np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_far)**2 + jerr_poly(jc_poly(full_rangec) + mu_far)**2),zc_poly(full_rangec) + mu_far + zerr_poly(zc_poly(full_rangec) + mu_far),color = 'C3',label='1 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-jc_poly(full_rangec) + Azj + np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_far + Az)**2 + jerr_poly(jc_poly(full_rangec) + mu_far + Aj)**2), zc_poly(full_rangec) + mu_far + Az - zerr_poly(zc_poly(full_rangec) + mu_far + Az),color = 'C3',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-jdo_poly(full_ranged) - np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_near)**2 + jerr_poly(jdo_poly(full_ranged) + mu_near)**2),zdo_poly(full_ranged) + mu_near + zerr_poly(zdo_poly(full_ranged) + mu_near),color = 'C4',label='5 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-jdo_poly(full_ranged) + Azj + np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_near + Az)**2 + jerr_poly(jdo_poly(full_ranged) + mu_near + Aj)**2), zdo_poly(full_ranged) + mu_near + Az - zerr_poly(zdo_poly(full_ranged) + mu_near + Az),color = 'C4',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-jco_poly(full_rangec) - np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_near)**2 + jerr_poly(jco_poly(full_rangec) + mu_near)**2),zco_poly(full_rangec) + mu_near + zerr_poly(zco_poly(full_rangec) + mu_near),color = 'C5',label='5 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-jco_poly(full_rangec) + Azj + np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_near + Az)**2 + jerr_poly(jco_poly(full_rangec) + mu_near + Aj)**2), zco_poly(full_rangec) + mu_near + Az - zerr_poly(zco_poly(full_rangec) + mu_near + Az),color = 'C5',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-jdo_poly(full_ranged) - np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_far)**2 + jerr_poly(jdo_poly(full_ranged) + mu_far)**2),zdo_poly(full_ranged) + mu_far + zerr_poly(zdo_poly(full_ranged) + mu_far),color = 'C6',label='5 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-jdo_poly(full_ranged) + Azj + np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_far + Az)**2 + jerr_poly(jdo_poly(full_ranged) + mu_far + Aj)**2), zdo_poly(full_ranged) + mu_far + Az - zerr_poly(zdo_poly(full_ranged) + mu_far + Az),color = 'C6',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-jco_poly(full_rangec) - np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_far)**2 + jerr_poly(jco_poly(full_rangec) + mu_far)**2),zco_poly(full_rangec) + mu_far + zerr_poly(zco_poly(full_rangec) + mu_far),color = 'C8',label='5 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-jco_poly(full_rangec) + Azj + np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_far + Az)**2 + jerr_poly(jco_poly(full_rangec) + mu_far + Aj)**2), zco_poly(full_rangec) + mu_far + Az - zerr_poly(zco_poly(full_rangec) + mu_far + Az),color = 'C8',linewidth=4.0)
plt.arrow(2.5,18,Azj,Az,width=0.01,length_includes_head=True,color='r')
plt.text(2.5,17.9,'$A_v$ = 1.1',fontsize=30)
plt.ylim(13,24)
plt.xlim(0,4)
#plt.ylim(18.75,21)
#plt.xlim(2,2.5)
plt.legend(fontsize=32)
plt.gca().invert_yaxis()
plt.xlabel('Z-J Mag', fontsize=40)
plt.ylabel('Z Mag', fontsize=40)
plt.savefig('cmd_zj.png')
plt.show()

In [ ]:
plt.figure(figsize=(40,20))
plt.rc('xtick',labelsize=28)
plt.rc('ytick',labelsize=28)
plt.plot(mag_z-mag_h,mag_z,'.',color='grey')
plt.plot(zd_poly(full_ranged)-hd_poly(full_ranged) - np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_near)**2 + herr_poly(hd_poly(full_ranged) + mu_near)**2),zd_poly(full_ranged) + mu_near + zerr_poly(zd_poly(full_ranged) + mu_near),color = 'C0',label='1 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(zd_poly(full_ranged)-hd_poly(full_ranged) + Azh + np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_near + Az)**2 + herr_poly(hd_poly(full_ranged) + mu_near + Ah)**2), zd_poly(full_ranged) + mu_near + Az - zerr_poly(zd_poly(full_ranged) + mu_near + Az),color = 'C0',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-hc_poly(full_rangec) - np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_near)**2 + herr_poly(hc_poly(full_rangec) + mu_near)**2),zc_poly(full_rangec) + mu_near + zerr_poly(zc_poly(full_rangec) + mu_near),color = 'C1',label='1 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-hc_poly(full_rangec) + Azh + np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_near + Az)**2 + herr_poly(hc_poly(full_rangec) + mu_near + Ah)**2) , zc_poly(full_rangec) + mu_near + Az - zerr_poly(zc_poly(full_rangec) + mu_near + Az),color = 'C1',linewidth=4.0)
plt.plot(zd_poly(full_ranged)-hd_poly(full_ranged) - np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_far)**2 + herr_poly(hd_poly(full_ranged) + mu_far)**2),zd_poly(full_ranged) + mu_far + zerr_poly(zd_poly(full_ranged) + mu_far),color = 'C2',label='1 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(zd_poly(full_ranged)-hd_poly(full_ranged) + Azh + np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_far + Az)**2 + herr_poly(hd_poly(full_ranged) + mu_far + Ah)**2), zd_poly(full_ranged) + mu_far + Az - zerr_poly(zd_poly(full_ranged) + mu_far + Az),color = 'C2',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-hc_poly(full_rangec) - np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_far)**2 + herr_poly(hc_poly(full_rangec) + mu_far)**2),zc_poly(full_rangec) + mu_far + zerr_poly(zc_poly(full_rangec) + mu_far),color = 'C3',label='1 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-hc_poly(full_rangec) + Azh + np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_far + Az)**2 + herr_poly(hc_poly(full_rangec) + mu_far + Ah)**2), zc_poly(full_rangec) + mu_far + Az - zerr_poly(zc_poly(full_rangec) + mu_far + Az),color = 'C3',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-hdo_poly(full_ranged) - np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_near)**2 + herr_poly(hdo_poly(full_ranged) + mu_near)**2),zdo_poly(full_ranged) + mu_near + zerr_poly(zdo_poly(full_ranged) + mu_near),color = 'C4',label='5 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-hdo_poly(full_ranged) + Azh + np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_near + Az)**2 + herr_poly(hdo_poly(full_ranged) + mu_near + Ah)**2), zdo_poly(full_ranged) + mu_near + Az - zerr_poly(zdo_poly(full_ranged) + mu_near + Az),color = 'C4',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-hco_poly(full_rangec) - np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_near)**2 + herr_poly(hco_poly(full_rangec) + mu_near)**2),zco_poly(full_rangec) + mu_near + zerr_poly(zco_poly(full_rangec) + mu_near),color = 'C5',label='5 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-hco_poly(full_rangec) + Azh + np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_near + Az)**2 + herr_poly(hco_poly(full_rangec) + mu_near + Ah)**2), zco_poly(full_rangec) + mu_near + Az - zerr_poly(zco_poly(full_rangec) + mu_near + Az),color = 'C5',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-hdo_poly(full_ranged) - np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_far)**2 + herr_poly(hdo_poly(full_ranged) + mu_far)**2),zdo_poly(full_ranged) + mu_far + zerr_poly(zdo_poly(full_ranged) + mu_far),color = 'C6',label='5 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-hdo_poly(full_ranged) + Azh + np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_far + Az)**2 + herr_poly(hdo_poly(full_ranged) + mu_far + Ah)**2), zdo_poly(full_ranged) + mu_far + Az - zerr_poly(zdo_poly(full_ranged) + mu_far + Az),color = 'C6',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-hco_poly(full_rangec) - np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_far)**2 + herr_poly(hco_poly(full_rangec) + mu_far)**2),zco_poly(full_rangec) + mu_far + zerr_poly(zco_poly(full_rangec) + mu_far),color = 'C8',label='5 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-hco_poly(full_rangec) + Azh + np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_far + Az)**2 + herr_poly(hco_poly(full_rangec) + mu_far + Ah)**2), zco_poly(full_rangec) + mu_far + Az - zerr_poly(zco_poly(full_rangec) + mu_far + Az),color = 'C8',linewidth=4.0)
plt.arrow(3,18,Azh,Az,width=0.01,length_includes_head=True,color='r')
plt.text(3,17.9,'$A_v$ = 1.1',fontsize=30)
plt.ylim(13,24)
plt.xlim(0.5,4.25)
#plt.ylim(18.75,21)
#plt.xlim(2,2.5)
plt.legend(fontsize=32)
plt.gca().invert_yaxis()
plt.xlabel('Z-H Mag', fontsize=40)
plt.ylabel('Z Mag', fontsize=40)
plt.savefig('cmd_zh.png')
plt.show()

In [ ]:
plt.figure(figsize=(40,20))
plt.rc('xtick',labelsize=28)
plt.rc('ytick',labelsize=28)
plt.plot(mag_z-mag_k,mag_z,'.',color='grey')
plt.plot(zd_poly(full_ranged)-kd_poly(full_ranged) - np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_near)**2 + kerr_poly(kd_poly(full_ranged) + mu_near)**2),zd_poly(full_ranged) + mu_near + zerr_poly(zd_poly(full_ranged) + mu_near),color = 'C0',label='1 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(zd_poly(full_ranged)-kd_poly(full_ranged) + Azk + np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_near + Az)**2 + kerr_poly(kd_poly(full_ranged) + mu_near + Ak)**2), zd_poly(full_ranged) + mu_near + Az - zerr_poly(zd_poly(full_ranged) + mu_near + Az),color = 'C0',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-kc_poly(full_rangec) - np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_near)**2 + kerr_poly(kc_poly(full_rangec) + mu_near)**2),zc_poly(full_rangec) + mu_near + zerr_poly(zc_poly(full_rangec) + mu_near),color = 'C1',label='1 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-kc_poly(full_rangec) + Azk + np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_near + Az)**2 + kerr_poly(kc_poly(full_rangec) + mu_near + Ak)**2) , zc_poly(full_rangec) + mu_near + Az - zerr_poly(zc_poly(full_rangec) + mu_near + Az),color = 'C1',linewidth=4.0)
plt.plot(zd_poly(full_ranged)-kd_poly(full_ranged) - np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_far)**2 + kerr_poly(kd_poly(full_ranged) + mu_far)**2),zd_poly(full_ranged) + mu_far + zerr_poly(zd_poly(full_ranged) + mu_far),color = 'C2',label='1 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(zd_poly(full_ranged)-kd_poly(full_ranged) + Azk + np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_far + Az)**2 + kerr_poly(kd_poly(full_ranged) + mu_far + Ak)**2), zd_poly(full_ranged) + mu_far + Az - zerr_poly(zd_poly(full_ranged) + mu_far + Az),color = 'C2',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-kc_poly(full_rangec) - np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_far)**2 + kerr_poly(kc_poly(full_rangec) + mu_far)**2),zc_poly(full_rangec) + mu_far + zerr_poly(zc_poly(full_rangec) + mu_far),color = 'C3',label='1 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(zc_poly(full_rangec)-kc_poly(full_rangec) + Azk + np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_far + Az)**2 + kerr_poly(kc_poly(full_rangec) + mu_far + Ak)**2), zc_poly(full_rangec) + mu_far + Az - zerr_poly(zc_poly(full_rangec) + mu_far + Az),color = 'C3',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-kdo_poly(full_ranged) - np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_near)**2 + kerr_poly(kdo_poly(full_ranged) + mu_near)**2),zdo_poly(full_ranged) + mu_near + zerr_poly(zdo_poly(full_ranged) + mu_near),color = 'C4',label='5 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-kdo_poly(full_ranged) + Azk + np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_near + Az)**2 + kerr_poly(kdo_poly(full_ranged) + mu_near + Ak)**2), zdo_poly(full_ranged) + mu_near + Az - zerr_poly(zdo_poly(full_ranged) + mu_near + Az),color = 'C4',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-kco_poly(full_rangec) - np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_near)**2 + kerr_poly(kco_poly(full_rangec) + mu_near)**2),zco_poly(full_rangec) + mu_near + zerr_poly(zco_poly(full_rangec) + mu_near),color = 'C5',label='5 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-kco_poly(full_rangec) + Azk + np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_near + Az)**2 + kerr_poly(kco_poly(full_rangec) + mu_near + Ak)**2), zco_poly(full_rangec) + mu_near + Az - zerr_poly(zco_poly(full_rangec) + mu_near + Az),color = 'C5',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-kdo_poly(full_ranged) - np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_far)**2 + kerr_poly(kdo_poly(full_ranged) + mu_far)**2),zdo_poly(full_ranged) + mu_far + zerr_poly(zdo_poly(full_ranged) + mu_far),color = 'C6',label='5 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(zdo_poly(full_ranged)-kdo_poly(full_ranged) + Azk + np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_far + Az)**2 + kerr_poly(kdo_poly(full_ranged) + mu_far + Ak)**2), zdo_poly(full_ranged) + mu_far + Az - zerr_poly(zdo_poly(full_ranged) + mu_far + Az),color = 'C6',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-kco_poly(full_rangec) - np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_far)**2 + kerr_poly(kco_poly(full_rangec) + mu_far)**2),zco_poly(full_rangec) + mu_far + zerr_poly(zco_poly(full_rangec) + mu_far),color = 'C8',label='5 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-kco_poly(full_rangec) + Azk + np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_far + Az)**2 + kerr_poly(kco_poly(full_rangec) + mu_far + Ak)**2), zco_poly(full_rangec) + mu_far + Az - zerr_poly(zco_poly(full_rangec) + mu_far + Az),color = 'C8',linewidth=4.0)
plt.arrow(3.5,18,Azk,Az,width=0.01,length_includes_head=True,color='r')
plt.text(3.5,17.9,'$A_v$ = 1.1',fontsize=30)
plt.ylim(13,24)
plt.xlim(.5,5)
#plt.ylim(18.75,21)
#plt.xlim(2,2.5)
plt.legend(fontsize=32)
plt.gca().invert_yaxis()
plt.xlabel('Z-Ks Mag', fontsize=40)
plt.ylabel('Z Mag', fontsize=40)
plt.savefig('cmd_zk.png')
plt.show()

In [ ]:
plt.figure(figsize=(40,20))
plt.rc('xtick',labelsize=28)
plt.rc('ytick',labelsize=28)
plt.plot(mag_y-mag_j,mag_y,'.',color='grey')
plt.plot(yd_poly(full_ranged)-jd_poly(full_ranged) - np.sqrt(yerr_poly(yd_poly(full_ranged) + mu_near)**2 + jerr_poly(jd_poly(full_ranged) + mu_near)**2),yd_poly(full_ranged) + mu_near + yerr_poly(yd_poly(full_ranged) + mu_near),color = 'C0',label='1 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(yd_poly(full_ranged)-jd_poly(full_ranged) + Ayj + np.sqrt(yerr_poly(yd_poly(full_ranged) + mu_near + Ay)**2 + jerr_poly(jd_poly(full_ranged) + mu_near + Aj)**2), yd_poly(full_ranged) + mu_near + Ay - yerr_poly(yd_poly(full_ranged) + mu_near + Ay),color = 'C0',linewidth=4.0)
plt.plot(yc_poly(full_rangec)-jc_poly(full_rangec) - np.sqrt(yerr_poly(yc_poly(full_rangec) + mu_near)**2 + jerr_poly(jc_poly(full_rangec) + mu_near)**2),yc_poly(full_rangec) + mu_near + yerr_poly(yc_poly(full_rangec) + mu_near),color = 'C1',label='1 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(yc_poly(full_rangec)-jc_poly(full_rangec) + Ayj + np.sqrt(yerr_poly(yc_poly(full_rangec) + mu_near + Ay)**2 + jerr_poly(jc_poly(full_rangec) + mu_near + Aj)**2) , yc_poly(full_rangec) + mu_near + Ay - yerr_poly(yc_poly(full_rangec) + mu_near + Ay),color = 'C1',linewidth=4.0)
plt.plot(yd_poly(full_ranged)-jd_poly(full_ranged) - np.sqrt(yerr_poly(yd_poly(full_ranged) + mu_far)**2 + jerr_poly(jd_poly(full_ranged) + mu_far)**2),yd_poly(full_ranged) + mu_far + yerr_poly(yd_poly(full_ranged) + mu_far),color = 'C2',label='1 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(yd_poly(full_ranged)-jd_poly(full_ranged) + Ayj + np.sqrt(yerr_poly(yd_poly(full_ranged) + mu_far + Ay)**2 + jerr_poly(jd_poly(full_ranged) + mu_far + Aj)**2), yd_poly(full_ranged) + mu_far + Ay - yerr_poly(yd_poly(full_ranged) + mu_far + Ay),color = 'C2',linewidth=4.0)
plt.plot(yc_poly(full_rangec)-jc_poly(full_rangec) - np.sqrt(yerr_poly(yc_poly(full_rangec) + mu_far)**2 + jerr_poly(jc_poly(full_rangec) + mu_far)**2),yc_poly(full_rangec) + mu_far + yerr_poly(yc_poly(full_rangec) + mu_far),color = 'C3',label='1 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(yc_poly(full_rangec)-jc_poly(full_rangec) + Ayj + np.sqrt(yerr_poly(yc_poly(full_rangec) + mu_far + Ay)**2 + jerr_poly(jc_poly(full_rangec) + mu_far + Aj)**2), yc_poly(full_rangec) + mu_far + Ay - yerr_poly(yc_poly(full_rangec) + mu_far + Ay),color = 'C3',linewidth=4.0)
plt.plot(ydo_poly(full_ranged)-jdo_poly(full_ranged) - np.sqrt(yerr_poly(ydo_poly(full_ranged) + mu_near)**2 + jerr_poly(jdo_poly(full_ranged) + mu_near)**2),ydo_poly(full_ranged) + mu_near + yerr_poly(ydo_poly(full_ranged) + mu_near),color = 'C4',label='5 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(ydo_poly(full_ranged)-jdo_poly(full_ranged) + Ayj + np.sqrt(yerr_poly(ydo_poly(full_ranged) + mu_near + Ay)**2 + jerr_poly(jdo_poly(full_ranged) + mu_near + Aj)**2), ydo_poly(full_ranged) + mu_near + Ay - yerr_poly(ydo_poly(full_ranged) + mu_near + Ay),color = 'C4',linewidth=4.0)
plt.plot(yco_poly(full_rangec)-jco_poly(full_rangec) - np.sqrt(yerr_poly(yco_poly(full_rangec) + mu_near)**2 + jerr_poly(jco_poly(full_rangec) + mu_near)**2),yco_poly(full_rangec) + mu_near + yerr_poly(yco_poly(full_rangec) + mu_near),color = 'C5',label='5 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(yco_poly(full_rangec)-jco_poly(full_rangec) + Ayj + np.sqrt(yerr_poly(yco_poly(full_rangec) + mu_near + Ay)**2 + jerr_poly(jco_poly(full_rangec) + mu_near + Aj)**2), yco_poly(full_rangec) + mu_near + Ay - yerr_poly(yco_poly(full_rangec) + mu_near + Ay),color = 'C5',linewidth=4.0)
plt.plot(ydo_poly(full_ranged)-jdo_poly(full_ranged) - np.sqrt(yerr_poly(ydo_poly(full_ranged) + mu_far)**2 + jerr_poly(jdo_poly(full_ranged) + mu_far)**2),ydo_poly(full_ranged) + mu_far + yerr_poly(ydo_poly(full_ranged) + mu_far),color = 'C6',label='5 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(ydo_poly(full_ranged)-jdo_poly(full_ranged) + Ayj + np.sqrt(yerr_poly(ydo_poly(full_ranged) + mu_far + Ay)**2 + jerr_poly(jdo_poly(full_ranged) + mu_far + Aj)**2), ydo_poly(full_ranged) + mu_far + Ay - yerr_poly(ydo_poly(full_ranged) + mu_far + Ay),color = 'C6',linewidth=4.0)
plt.plot(yco_poly(full_rangec)-jco_poly(full_rangec) - np.sqrt(yerr_poly(yco_poly(full_rangec) + mu_far)**2 + jerr_poly(jco_poly(full_rangec) + mu_far)**2),yco_poly(full_rangec) + mu_far + yerr_poly(yco_poly(full_rangec) + mu_far),color = 'C8',label='5 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(yco_poly(full_rangec)-jco_poly(full_rangec) + Ayj + np.sqrt(yerr_poly(yco_poly(full_rangec) + mu_far + Ay)**2 + jerr_poly(jco_poly(full_rangec) + mu_far + Aj)**2), yco_poly(full_rangec) + mu_far + Ay - yerr_poly(yco_poly(full_rangec) + mu_far + Ay),color = 'C8',linewidth=4.0)
plt.arrow(1.2,17,Ayj,Ay,width=0.01,length_includes_head=True,color='r')
plt.text(1.2,16.9,'$A_v$ = 1.1',fontsize=30)
plt.ylim(13,22.25)
plt.xlim(0,2)
#plt.ylim(18.75,21)
#plt.xlim(2,2.5)
plt.legend(fontsize=32)
plt.gca().invert_yaxis()
plt.xlabel('Y-J Mag', fontsize=40)
plt.ylabel('Y Mag', fontsize=40)
plt.savefig('cmd_yj.png')
plt.show()

In [ ]:
plt.figure(figsize=(40,20))
plt.rc('xtick',labelsize=28)
plt.rc('ytick',labelsize=28)
plt.plot(mag_y-mag_h,mag_y,'.',color='grey')
plt.plot(yd_poly(full_ranged)-hd_poly(full_ranged) - np.sqrt(yerr_poly(yd_poly(full_ranged) + mu_near)**2 + herr_poly(hd_poly(full_ranged) + mu_near)**2),yd_poly(full_ranged) + mu_near + yerr_poly(yd_poly(full_ranged) + mu_near),color = 'C0',label='1 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(yd_poly(full_ranged)-hd_poly(full_ranged) + Ayh + np.sqrt(yerr_poly(yd_poly(full_ranged) + mu_near + Ay)**2 + herr_poly(hd_poly(full_ranged) + mu_near + Ah)**2), yd_poly(full_ranged) + mu_near + Ay - yerr_poly(yd_poly(full_ranged) + mu_near + Ay),color = 'C0',linewidth=4.0)
plt.plot(yc_poly(full_rangec)-hc_poly(full_rangec) - np.sqrt(yerr_poly(yc_poly(full_rangec) + mu_near)**2 + herr_poly(hc_poly(full_rangec) + mu_near)**2),yc_poly(full_rangec) + mu_near + yerr_poly(yc_poly(full_rangec) + mu_near),color = 'C1',label='1 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(yc_poly(full_rangec)-hc_poly(full_rangec) + Ayh + np.sqrt(yerr_poly(yc_poly(full_rangec) + mu_near + Ay)**2 + herr_poly(hc_poly(full_rangec) + mu_near + Ah)**2) , yc_poly(full_rangec) + mu_near + Ay - yerr_poly(yc_poly(full_rangec) + mu_near + Ay),color = 'C1',linewidth=4.0)
plt.plot(yd_poly(full_ranged)-hd_poly(full_ranged) - np.sqrt(yerr_poly(yd_poly(full_ranged) + mu_far)**2 + herr_poly(hd_poly(full_ranged) + mu_far)**2),yd_poly(full_ranged) + mu_far + yerr_poly(yd_poly(full_ranged) + mu_far),color = 'C2',label='1 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(yd_poly(full_ranged)-hd_poly(full_ranged) + Ayh + np.sqrt(yerr_poly(yd_poly(full_ranged) + mu_far + Ay)**2 + herr_poly(hd_poly(full_ranged) + mu_far + Ah)**2), yd_poly(full_ranged) + mu_far + Ay - yerr_poly(yd_poly(full_ranged) + mu_far + Ay),color = 'C2',linewidth=4.0)
plt.plot(yc_poly(full_rangec)-hc_poly(full_rangec) - np.sqrt(yerr_poly(yc_poly(full_rangec) + mu_far)**2 + herr_poly(hc_poly(full_rangec) + mu_far)**2),yc_poly(full_rangec) + mu_far + yerr_poly(yc_poly(full_rangec) + mu_far),color = 'C3',label='1 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(yc_poly(full_rangec)-hc_poly(full_rangec) + Ayh + np.sqrt(yerr_poly(yc_poly(full_rangec) + mu_far + Ay)**2 + herr_poly(hc_poly(full_rangec) + mu_far + Ah)**2), yc_poly(full_rangec) + mu_far + Ay - yerr_poly(yc_poly(full_rangec) + mu_far + Ay),color = 'C3',linewidth=4.0)
plt.plot(ydo_poly(full_ranged)-hdo_poly(full_ranged) - np.sqrt(yerr_poly(ydo_poly(full_ranged) + mu_near)**2 + herr_poly(hdo_poly(full_ranged) + mu_near)**2),ydo_poly(full_ranged) + mu_near + yerr_poly(ydo_poly(full_ranged) + mu_near),color = 'C4',label='5 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(ydo_poly(full_ranged)-hdo_poly(full_ranged) + Ayh + np.sqrt(yerr_poly(ydo_poly(full_ranged) + mu_near + Ay)**2 + herr_poly(hdo_poly(full_ranged) + mu_near + Ah)**2), ydo_poly(full_ranged) + mu_near + Ay - yerr_poly(ydo_poly(full_ranged) + mu_near + Ay),color = 'C4',linewidth=4.0)
plt.plot(yco_poly(full_rangec)-hco_poly(full_rangec) - np.sqrt(yerr_poly(yco_poly(full_rangec) + mu_near)**2 + herr_poly(hco_poly(full_rangec) + mu_near)**2),yco_poly(full_rangec) + mu_near + yerr_poly(yco_poly(full_rangec) + mu_near),color = 'C5',label='5 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(yco_poly(full_rangec)-hco_poly(full_rangec) + Ayh + np.sqrt(yerr_poly(yco_poly(full_rangec) + mu_near + Ay)**2 + herr_poly(hco_poly(full_rangec) + mu_near + Ah)**2), yco_poly(full_rangec) + mu_near + Ay - yerr_poly(yco_poly(full_rangec) + mu_near + Ay),color = 'C5',linewidth=4.0)
plt.plot(ydo_poly(full_ranged)-hdo_poly(full_ranged) - np.sqrt(yerr_poly(ydo_poly(full_ranged) + mu_far)**2 + herr_poly(hdo_poly(full_ranged) + mu_far)**2),ydo_poly(full_ranged) + mu_far + yerr_poly(ydo_poly(full_ranged) + mu_far),color = 'C6',label='5 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(ydo_poly(full_ranged)-hdo_poly(full_ranged) + Ayh + np.sqrt(yerr_poly(ydo_poly(full_ranged) + mu_far + Ay)**2 + herr_poly(hdo_poly(full_ranged) + mu_far + Ah)**2), ydo_poly(full_ranged) + mu_far + Ay - yerr_poly(ydo_poly(full_ranged) + mu_far + Ay),color = 'C6',linewidth=4.0)
plt.plot(yco_poly(full_rangec)-hco_poly(full_rangec) - np.sqrt(yerr_poly(yco_poly(full_rangec) + mu_far)**2 + herr_poly(hco_poly(full_rangec) + mu_far)**2),yco_poly(full_rangec) + mu_far + yerr_poly(yco_poly(full_rangec) + mu_far),color = 'C8',label='5 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(yco_poly(full_rangec)-hco_poly(full_rangec) + Ayh + np.sqrt(yerr_poly(yco_poly(full_rangec) + mu_far + Ay)**2 + herr_poly(hco_poly(full_rangec) + mu_far + Ah)**2), yco_poly(full_rangec) + mu_far + Ay - yerr_poly(yco_poly(full_rangec) + mu_far + Ay),color = 'C8',linewidth=4.0)
plt.arrow(1.75,17,Ayh,Ay,width=0.01,length_includes_head=True,color='r')
plt.text(1.75,16.9,'$A_v$ = 1.1',fontsize=30)
plt.ylim(13,22.25)
plt.xlim(0.5,3)
#plt.ylim(18.75,21)
#plt.xlim(2,2.5)
plt.legend(fontsize=32)
plt.gca().invert_yaxis()
plt.xlabel('Y-H Mag', fontsize=40)
plt.ylabel('Y Mag', fontsize=40)
plt.savefig('cmd_yh.png')
plt.show()

In [ ]:
plt.figure(figsize=(40,20))
plt.rc('xtick',labelsize=28)
plt.rc('ytick',labelsize=28)
plt.plot(mag_y-mag_k,mag_y,'.',color='grey')
plt.plot(yd_poly(full_ranged)-kd_poly(full_ranged) - np.sqrt(yerr_poly(yd_poly(full_ranged) + mu_near)**2 + kerr_poly(kd_poly(full_ranged) + mu_near)**2),yd_poly(full_ranged) + mu_near + yerr_poly(yd_poly(full_ranged) + mu_near),color = 'C0',label='1 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(yd_poly(full_ranged)-kd_poly(full_ranged) + Ayk + np.sqrt(yerr_poly(yd_poly(full_ranged) + mu_near + Ay)**2 + kerr_poly(kd_poly(full_ranged) + mu_near + Ak)**2), yd_poly(full_ranged) + mu_near + Ay - yerr_poly(yd_poly(full_ranged) + mu_near + Ay),color = 'C0',linewidth=4.0)
plt.plot(yc_poly(full_rangec)-kc_poly(full_rangec) - np.sqrt(yerr_poly(yc_poly(full_rangec) + mu_near)**2 + kerr_poly(kc_poly(full_rangec) + mu_near)**2),yc_poly(full_rangec) + mu_near + yerr_poly(yc_poly(full_rangec) + mu_near),color = 'C1',label='1 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(yc_poly(full_rangec)-kc_poly(full_rangec) + Ayk + np.sqrt(yerr_poly(yc_poly(full_rangec) + mu_near + Ay)**2 + kerr_poly(kc_poly(full_rangec) + mu_near + Ak)**2) , yc_poly(full_rangec) + mu_near + Ay - yerr_poly(yc_poly(full_rangec) + mu_near + Ay),color = 'C1',linewidth=4.0)
plt.plot(yd_poly(full_ranged)-kd_poly(full_ranged) - np.sqrt(yerr_poly(yd_poly(full_ranged) + mu_far)**2 + kerr_poly(kd_poly(full_ranged) + mu_far)**2),yd_poly(full_ranged) + mu_far + yerr_poly(yd_poly(full_ranged) + mu_far),color = 'C2',label='1 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(yd_poly(full_ranged)-kd_poly(full_ranged) + Ayk + np.sqrt(yerr_poly(yd_poly(full_ranged) + mu_far + Ay)**2 + kerr_poly(kd_poly(full_ranged) + mu_far + Ak)**2), yd_poly(full_ranged) + mu_far + Ay - yerr_poly(yd_poly(full_ranged) + mu_far + Ay),color = 'C2',linewidth=4.0)
plt.plot(yc_poly(full_rangec)-kc_poly(full_rangec) - np.sqrt(yerr_poly(yc_poly(full_rangec) + mu_far)**2 + kerr_poly(kc_poly(full_rangec) + mu_far)**2),yc_poly(full_rangec) + mu_far + yerr_poly(yc_poly(full_rangec) + mu_far),color = 'C3',label='1 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(yc_poly(full_rangec)-kc_poly(full_rangec) + Ayk + np.sqrt(yerr_poly(yc_poly(full_rangec) + mu_far + Ay)**2 + kerr_poly(kc_poly(full_rangec) + mu_far + Ak)**2), yc_poly(full_rangec) + mu_far + Ay - yerr_poly(yc_poly(full_rangec) + mu_far + Ay),color = 'C3',linewidth=4.0)
plt.plot(ydo_poly(full_ranged)-kdo_poly(full_ranged) - np.sqrt(yerr_poly(ydo_poly(full_ranged) + mu_near)**2 + kerr_poly(kdo_poly(full_ranged) + mu_near)**2),ydo_poly(full_ranged) + mu_near + yerr_poly(ydo_poly(full_ranged) + mu_near),color = 'C4',label='5 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(ydo_poly(full_ranged)-kdo_poly(full_ranged) + Ayk + np.sqrt(yerr_poly(ydo_poly(full_ranged) + mu_near + Ay)**2 + kerr_poly(kdo_poly(full_ranged) + mu_near + Ak)**2), ydo_poly(full_ranged) + mu_near + Ay - yerr_poly(ydo_poly(full_ranged) + mu_near + Ay),color = 'C4',linewidth=4.0)
plt.plot(yco_poly(full_rangec)-kco_poly(full_rangec) - np.sqrt(yerr_poly(yco_poly(full_rangec) + mu_near)**2 + kerr_poly(kco_poly(full_rangec) + mu_near)**2),yco_poly(full_rangec) + mu_near + yerr_poly(yco_poly(full_rangec) + mu_near),color = 'C5',label='5 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(yco_poly(full_rangec)-kco_poly(full_rangec) + Ayk + np.sqrt(yerr_poly(yco_poly(full_rangec) + mu_near + Ay)**2 + kerr_poly(kco_poly(full_rangec) + mu_near + Ak)**2), yco_poly(full_rangec) + mu_near + Ay - yerr_poly(yco_poly(full_rangec) + mu_near + Ay),color = 'C5',linewidth=4.0)
plt.plot(ydo_poly(full_ranged)-kdo_poly(full_ranged) - np.sqrt(yerr_poly(ydo_poly(full_ranged) + mu_far)**2 + kerr_poly(kdo_poly(full_ranged) + mu_far)**2),ydo_poly(full_ranged) + mu_far + yerr_poly(ydo_poly(full_ranged) + mu_far),color = 'C6',label='5 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(ydo_poly(full_ranged)-kdo_poly(full_ranged) + Ayk + np.sqrt(yerr_poly(ydo_poly(full_ranged) + mu_far + Ay)**2 + kerr_poly(kdo_poly(full_ranged) + mu_far + Ak)**2), ydo_poly(full_ranged) + mu_far + Ay - yerr_poly(ydo_poly(full_ranged) + mu_far + Ay),color = 'C6',linewidth=4.0)
plt.plot(yco_poly(full_rangec)-kco_poly(full_rangec) - np.sqrt(yerr_poly(yco_poly(full_rangec) + mu_far)**2 + kerr_poly(kco_poly(full_rangec) + mu_far)**2),yco_poly(full_rangec) + mu_far + yerr_poly(yco_poly(full_rangec) + mu_far),color = 'C8',label='5 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(yco_poly(full_rangec)-kco_poly(full_rangec) + Ayk + np.sqrt(yerr_poly(yco_poly(full_rangec) + mu_far + Ay)**2 + kerr_poly(kco_poly(full_rangec) + mu_far + Ak)**2), yco_poly(full_rangec) + mu_far + Ay - yerr_poly(yco_poly(full_rangec) + mu_far + Ay),color = 'C8',linewidth=4.0)
plt.arrow(2,16,Ayk,Ay,width=0.01,length_includes_head=True,color='r')
plt.text(2,15.9,'$A_v$ = 1.1',fontsize=30)
plt.ylim(13,22.25)
plt.xlim(0.75,4)
#plt.ylim(18.75,21)
#plt.xlim(2,2.5)
plt.legend(fontsize=32)
plt.gca().invert_yaxis()
plt.xlabel('Y-Ks Mag', fontsize=40)
plt.ylabel('Y Mag', fontsize=40)
plt.savefig('cmd_yk.png')
plt.show()

In [ ]:
plt.figure(figsize=(40,20))
plt.rc('xtick',labelsize=28)
plt.rc('ytick',labelsize=28)
plt.plot(mag_j-mag_h,mag_j,'.',color='grey')
plt.plot(jd_poly(full_ranged)-hd_poly(full_ranged) - np.sqrt(jerr_poly(jd_poly(full_ranged) + mu_near)**2 + herr_poly(hd_poly(full_ranged) + mu_near)**2), jd_poly(full_ranged) + mu_near + jerr_poly(jd_poly(full_ranged) + mu_near),color = 'C0',label='1 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(jd_poly(full_ranged)-hd_poly(full_ranged) + Ajh + np.sqrt(jerr_poly(jd_poly(full_ranged) + mu_near + Aj)**2 + herr_poly(hd_poly(full_ranged) + mu_near + Ah)**2), jd_poly(full_ranged) + mu_near + Aj - jerr_poly(jd_poly(full_ranged) + mu_near + Aj),color = 'C0',linewidth=4.0)
plt.plot(jc_poly(full_rangec)-hc_poly(full_rangec) - np.sqrt(jerr_poly(jc_poly(full_rangec) + mu_near)**2 + herr_poly(hc_poly(full_rangec) + mu_near)**2), jc_poly(full_rangec) + mu_near + jerr_poly(jc_poly(full_rangec) + mu_near),color = 'C1',label='1 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(jc_poly(full_rangec)-hc_poly(full_rangec) + Ajh + np.sqrt(jerr_poly(jc_poly(full_rangec) + mu_near + Aj)**2 + herr_poly(hc_poly(full_rangec) + mu_near + Ah)**2) , jc_poly(full_rangec) + mu_near + Aj - jerr_poly(jc_poly(full_rangec) + mu_near + Aj),color = 'C1',linewidth=4.0)
plt.plot(jd_poly(full_ranged)-hd_poly(full_ranged) - np.sqrt(jerr_poly(jd_poly(full_ranged) + mu_far)**2 + herr_poly(hd_poly(full_ranged) + mu_far)**2), jd_poly(full_ranged) + mu_far + jerr_poly(jd_poly(full_ranged) + mu_far),color = 'C2',label='1 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(jd_poly(full_ranged)-hd_poly(full_ranged) + Ajh + np.sqrt(jerr_poly(jd_poly(full_ranged) + mu_far + Aj)**2 + herr_poly(hd_poly(full_ranged) + mu_far + Ah)**2), jd_poly(full_ranged) + mu_far + Aj - jerr_poly(jd_poly(full_ranged) + mu_far + Aj),color = 'C2',linewidth=4.0)
plt.plot(jc_poly(full_rangec)-hc_poly(full_rangec) - np.sqrt(jerr_poly(jc_poly(full_rangec) + mu_far)**2 + herr_poly(hc_poly(full_rangec) + mu_far)**2), jc_poly(full_rangec) + mu_far + jerr_poly(jc_poly(full_rangec) + mu_far),color = 'C3',label='1 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(jc_poly(full_rangec)-hc_poly(full_rangec) + Ajh + np.sqrt(jerr_poly(jc_poly(full_rangec) + mu_far + Aj)**2 + herr_poly(hc_poly(full_rangec) + mu_far + Ah)**2), jc_poly(full_rangec) + mu_far + Aj - jerr_poly(jc_poly(full_rangec) + mu_far + Aj),color = 'C3',linewidth=4.0)
plt.plot(jdo_poly(full_ranged)-hdo_poly(full_ranged) - np.sqrt(jerr_poly(jdo_poly(full_ranged) + mu_near)**2 + herr_poly(hdo_poly(full_ranged) + mu_near)**2), jdo_poly(full_ranged) + mu_near + jerr_poly(jdo_poly(full_ranged) + mu_near),color = 'C4',label='5 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(jdo_poly(full_ranged)-hdo_poly(full_ranged) + Ajh + np.sqrt(jerr_poly(jdo_poly(full_ranged) + mu_near + Aj)**2 + herr_poly(hdo_poly(full_ranged) + mu_near + Ah)**2), jdo_poly(full_ranged) + mu_near + Aj - jerr_poly(jdo_poly(full_ranged) + mu_near + Aj),color = 'C4',linewidth=4.0)
plt.plot(jco_poly(full_rangec)-hco_poly(full_rangec) - np.sqrt(jerr_poly(jco_poly(full_rangec) + mu_near)**2 + herr_poly(hco_poly(full_rangec) + mu_near)**2), jco_poly(full_rangec) + mu_near + jerr_poly(jco_poly(full_rangec) + mu_near),color = 'C5',label='5 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(jco_poly(full_rangec)-hco_poly(full_rangec) + Ajh + np.sqrt(jerr_poly(jco_poly(full_rangec) + mu_near + Aj)**2 + herr_poly(hco_poly(full_rangec) + mu_near + Ah)**2), jco_poly(full_rangec) + mu_near + Aj - jerr_poly(jco_poly(full_rangec) + mu_near + Aj),color = 'C5',linewidth=4.0)
plt.plot(jdo_poly(full_ranged)-hdo_poly(full_ranged) - np.sqrt(jerr_poly(jdo_poly(full_ranged) + mu_far)**2 + herr_poly(hdo_poly(full_ranged) + mu_far)**2), jdo_poly(full_ranged) + mu_far + jerr_poly(jdo_poly(full_ranged) + mu_far),color = 'C6',label='5 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(jdo_poly(full_ranged)-hdo_poly(full_ranged) + Ajh + np.sqrt(jerr_poly(jdo_poly(full_ranged) + mu_far + Aj)**2 + herr_poly(hdo_poly(full_ranged) + mu_far + Ah)**2), jdo_poly(full_ranged) + mu_far + Aj - jerr_poly(jdo_poly(full_ranged) + mu_far + Aj),color = 'C6',linewidth=4.0)
plt.plot(jco_poly(full_rangec)-hco_poly(full_rangec) - np.sqrt(jerr_poly(jco_poly(full_rangec) + mu_far)**2 + herr_poly(hco_poly(full_rangec) + mu_far)**2), jco_poly(full_rangec) + mu_far + jerr_poly(jco_poly(full_rangec) + mu_far),color = 'C8',label='5 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(jco_poly(full_rangec)-hco_poly(full_rangec) + Ajh + np.sqrt(jerr_poly(jco_poly(full_rangec) + mu_far + Aj)**2 + herr_poly(hco_poly(full_rangec) + mu_far + Ah)**2), jco_poly(full_rangec) + mu_far + Aj - jerr_poly(jco_poly(full_rangec) + mu_far + Aj),color = 'C8',linewidth=4.0)
plt.arrow(1,16,Ajh,Aj,width=0.01,length_includes_head=True,color='r')
plt.text(1,15.9,'$A_v$ = 1.1',fontsize=30)
plt.ylim(12.5,21.5)
plt.xlim(-0.5,2)
#plt.ylim(18.75,21)
#plt.xlim(2,2.5)
plt.legend(fontsize=32)
plt.gca().invert_yaxis()
plt.xlabel('J-H Mag', fontsize=40)
plt.ylabel('J Mag', fontsize=40)
plt.savefig('cmd_jh.png')
plt.show()

In [ ]:
plt.figure(figsize=(40,20))
plt.rc('xtick',labelsize=28)
plt.rc('ytick',labelsize=28)
plt.plot(mag_j-mag_k,mag_j,'.',color='grey')
plt.plot(jd_poly(full_ranged)-kd_poly(full_ranged) - np.sqrt(jerr_poly(jd_poly(full_ranged) + mu_near)**2 + kerr_poly(kd_poly(full_ranged) + mu_near)**2),jd_poly(full_ranged) + mu_near + jerr_poly(jd_poly(full_ranged) + mu_near),color = 'C0',label='1 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(jd_poly(full_ranged)-kd_poly(full_ranged) + Ajk + np.sqrt(jerr_poly(jd_poly(full_ranged) + mu_near + Aj)**2 + kerr_poly(kd_poly(full_ranged) + mu_near + Ak)**2), jd_poly(full_ranged) + mu_near + Aj - jerr_poly(jd_poly(full_ranged) + mu_near + Aj),color = 'C0',linewidth=4.0)
plt.plot(jc_poly(full_rangec)-kc_poly(full_rangec) - np.sqrt(jerr_poly(jc_poly(full_rangec) + mu_near)**2 + kerr_poly(kc_poly(full_rangec) + mu_near)**2),jc_poly(full_rangec) + mu_near + jerr_poly(jc_poly(full_rangec) + mu_near),color = 'C1',label='1 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(jc_poly(full_rangec)-kc_poly(full_rangec) + Ajk + np.sqrt(jerr_poly(jc_poly(full_rangec) + mu_near + Aj)**2 + kerr_poly(kc_poly(full_rangec) + mu_near + Ak)**2) , jc_poly(full_rangec) + mu_near + Aj - jerr_poly(jc_poly(full_rangec) + mu_near + Aj),color = 'C1',linewidth=4.0)
plt.plot(jd_poly(full_ranged)-kd_poly(full_ranged) - np.sqrt(jerr_poly(jd_poly(full_ranged) + mu_far)**2 + kerr_poly(kd_poly(full_ranged) + mu_far)**2),jd_poly(full_ranged) + mu_far + jerr_poly(jd_poly(full_ranged) + mu_far),color = 'C2',label='1 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(jd_poly(full_ranged)-kd_poly(full_ranged) + Ajk + np.sqrt(jerr_poly(jd_poly(full_ranged) + mu_far + Aj)**2 + kerr_poly(kd_poly(full_ranged) + mu_far + Ak)**2), jd_poly(full_ranged) + mu_far + Aj - jerr_poly(jd_poly(full_ranged) + mu_far + Aj),color = 'C2',linewidth=4.0)
plt.plot(jc_poly(full_rangec)-kc_poly(full_rangec) - np.sqrt(jerr_poly(jc_poly(full_rangec) + mu_far)**2 + kerr_poly(kc_poly(full_rangec) + mu_far)**2),jc_poly(full_rangec) + mu_far + jerr_poly(jc_poly(full_rangec) + mu_far),color = 'C3',label='1 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(jc_poly(full_rangec)-kc_poly(full_rangec) + Ajk + np.sqrt(jerr_poly(jc_poly(full_rangec) + mu_far + Aj)**2 + kerr_poly(kc_poly(full_rangec) + mu_far + Ak)**2), jc_poly(full_rangec) + mu_far + Aj - jerr_poly(jc_poly(full_rangec) + mu_far + Aj),color = 'C3',linewidth=4.0)
plt.plot(jdo_poly(full_ranged)-kdo_poly(full_ranged) - np.sqrt(jerr_poly(jdo_poly(full_ranged) + mu_near)**2 + kerr_poly(kdo_poly(full_ranged) + mu_near)**2),jdo_poly(full_ranged) + mu_near + jerr_poly(jdo_poly(full_ranged) + mu_near),color = 'C4',label='5 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(jdo_poly(full_ranged)-kdo_poly(full_ranged) + Ajk + np.sqrt(jerr_poly(jdo_poly(full_ranged) + mu_near + Aj)**2 + kerr_poly(kdo_poly(full_ranged) + mu_near + Ak)**2), jdo_poly(full_ranged) + mu_near + Aj - jerr_poly(jdo_poly(full_ranged) + mu_near + Aj),color = 'C4',linewidth=4.0)
plt.plot(jco_poly(full_rangec)-kco_poly(full_rangec) - np.sqrt(jerr_poly(jco_poly(full_rangec) + mu_near)**2 + kerr_poly(kco_poly(full_rangec) + mu_near)**2),jco_poly(full_rangec) + mu_near + jerr_poly(jco_poly(full_rangec) + mu_near),color = 'C5',label='5 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(jco_poly(full_rangec)-kco_poly(full_rangec) + Ajk + np.sqrt(jerr_poly(jco_poly(full_rangec) + mu_near + Aj)**2 + kerr_poly(kco_poly(full_rangec) + mu_near + Ak)**2), jco_poly(full_rangec) + mu_near + Aj - jerr_poly(jco_poly(full_rangec) + mu_near + Aj),color = 'C5',linewidth=4.0)
plt.plot(jdo_poly(full_ranged)-kdo_poly(full_ranged) - np.sqrt(jerr_poly(jdo_poly(full_ranged) + mu_far)**2 + kerr_poly(kdo_poly(full_ranged) + mu_far)**2),jdo_poly(full_ranged) + mu_far + jerr_poly(jdo_poly(full_ranged) + mu_far),color = 'C6',label='5 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(jdo_poly(full_ranged)-kdo_poly(full_ranged) + Ajk + np.sqrt(jerr_poly(jdo_poly(full_ranged) + mu_far + Aj)**2 + kerr_poly(kdo_poly(full_ranged) + mu_far + Ak)**2), jdo_poly(full_ranged) + mu_far + Aj - jerr_poly(jdo_poly(full_ranged) + mu_far + Aj),color = 'C6',linewidth=4.0)
plt.plot(jco_poly(full_rangec)-kco_poly(full_rangec) - np.sqrt(jerr_poly(jco_poly(full_rangec) + mu_far)**2 + kerr_poly(kco_poly(full_rangec) + mu_far)**2),jco_poly(full_rangec) + mu_far + jerr_poly(jco_poly(full_rangec) + mu_far),color = 'C8',label='5 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(jco_poly(full_rangec)-kco_poly(full_rangec) + Ajk + np.sqrt(jerr_poly(jco_poly(full_rangec) + mu_far + Aj)**2 + kerr_poly(kco_poly(full_rangec) + mu_far + Ak)**2), jco_poly(full_rangec) + mu_far + Aj - jerr_poly(jco_poly(full_rangec) + mu_far + Aj),color = 'C8',linewidth=4.0)
plt.arrow(1.25,14,Ajk,Aj,width=0.01,length_includes_head=True,color='r')
plt.text(1.25,13.9,'$A_v$ = 1.1',fontsize=30)
plt.ylim(12.5,21.5)
plt.xlim(0.25,2.5)
#plt.ylim(18.75,21)
#plt.xlim(2,2.5)
plt.legend(fontsize=32)
plt.gca().invert_yaxis()
plt.xlabel('J-Ks Mag', fontsize=40)
plt.ylabel('J Mag', fontsize=40)
plt.savefig('cmd_jk.png')
plt.show()

In [ ]:
plt.figure(figsize=(40,20))
plt.rc('xtick',labelsize=28)
plt.rc('ytick',labelsize=28)
plt.plot(mag_h-mag_k,mag_h,'.',color='grey')
plt.plot(hd_poly(full_ranged)-kd_poly(full_ranged) - np.sqrt(herr_poly(hd_poly(full_ranged) + mu_near)**2 + kerr_poly(kd_poly(full_ranged) + mu_near)**2),hd_poly(full_ranged) + mu_near + herr_poly(hd_poly(full_ranged) + mu_near),color = 'C0',label='1 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(hd_poly(full_ranged)-kd_poly(full_ranged) + Ahk + np.sqrt(herr_poly(hd_poly(full_ranged) + mu_near + Ah)**2 + kerr_poly(kd_poly(full_ranged) + mu_near + Ak)**2), hd_poly(full_ranged) + mu_near + Ah - herr_poly(hd_poly(full_ranged) + mu_near + Ah),color = 'C0',linewidth=4.0)
plt.plot(hc_poly(full_rangec)-kc_poly(full_rangec) - np.sqrt(herr_poly(hc_poly(full_rangec) + mu_near)**2 + kerr_poly(kc_poly(full_rangec) + mu_near)**2),hc_poly(full_rangec) + mu_near + herr_poly(hc_poly(full_rangec) + mu_near),color = 'C1',label='1 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(hc_poly(full_rangec)-kc_poly(full_rangec) + Ahk + np.sqrt(herr_poly(hc_poly(full_rangec) + mu_near + Ah)**2 + kerr_poly(kc_poly(full_rangec) + mu_near + Ak)**2) , hc_poly(full_rangec) + mu_near + Ah - herr_poly(hc_poly(full_rangec) + mu_near + Ah),color = 'C1',linewidth=4.0)
plt.plot(hd_poly(full_ranged)-kd_poly(full_ranged) - np.sqrt(herr_poly(hd_poly(full_ranged) + mu_far)**2 + kerr_poly(kd_poly(full_ranged) + mu_far)**2),hd_poly(full_ranged) + mu_far + herr_poly(hd_poly(full_ranged) + mu_far),color = 'C2',label='1 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(hd_poly(full_ranged)-kd_poly(full_ranged) + Ahk + np.sqrt(herr_poly(hd_poly(full_ranged) + mu_far + Ah)**2 + kerr_poly(kd_poly(full_ranged) + mu_far + Ak)**2), hd_poly(full_ranged) + mu_far + Ah - herr_poly(hd_poly(full_ranged) + mu_far + Ah),color = 'C2',linewidth=4.0)
plt.plot(hc_poly(full_rangec)-kc_poly(full_rangec) - np.sqrt(herr_poly(hc_poly(full_rangec) + mu_far)**2 + kerr_poly(kc_poly(full_rangec) + mu_far)**2),hc_poly(full_rangec) + mu_far + herr_poly(hc_poly(full_rangec) + mu_far),color = 'C3',label='1 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(hc_poly(full_rangec)-kc_poly(full_rangec) + Ahk + np.sqrt(herr_poly(hc_poly(full_rangec) + mu_far + Ah)**2 + kerr_poly(kc_poly(full_rangec) + mu_far + Ak)**2), hc_poly(full_rangec) + mu_far + Ah - herr_poly(hc_poly(full_rangec) + mu_far + Ah),color = 'C3',linewidth=4.0)
plt.plot(hdo_poly(full_ranged)-kdo_poly(full_ranged) - np.sqrt(herr_poly(hdo_poly(full_ranged) + mu_near)**2 + kerr_poly(kdo_poly(full_ranged) + mu_near)**2),hdo_poly(full_ranged) + mu_near + herr_poly(hdo_poly(full_ranged) + mu_near),color = 'C4',label='5 Mya - 350 pc - Dusty',linewidth=4.0)
plt.plot(hdo_poly(full_ranged)-kdo_poly(full_ranged) + Ahk + np.sqrt(herr_poly(hdo_poly(full_ranged) + mu_near + Ah)**2 + kerr_poly(kdo_poly(full_ranged) + mu_near + Ak)**2), hdo_poly(full_ranged) + mu_near + Ah - herr_poly(hdo_poly(full_ranged) + mu_near + Ah),color = 'C4',linewidth=4.0)
plt.plot(hco_poly(full_rangec)-kco_poly(full_rangec) - np.sqrt(herr_poly(hco_poly(full_rangec) + mu_near)**2 + kerr_poly(kco_poly(full_rangec) + mu_near)**2),hco_poly(full_rangec) + mu_near + herr_poly(hco_poly(full_rangec) + mu_near),color = 'C5',label='5 Mya - 350 pc - Cond',linewidth=4.0)
plt.plot(hco_poly(full_rangec)-kco_poly(full_rangec) + Ahk + np.sqrt(herr_poly(hco_poly(full_rangec) + mu_near + Ah)**2 + kerr_poly(kco_poly(full_rangec) + mu_near + Ak)**2), hco_poly(full_rangec) + mu_near + Ah - herr_poly(hco_poly(full_rangec) + mu_near + Ah),color = 'C5',linewidth=4.0)
plt.plot(hdo_poly(full_ranged)-kdo_poly(full_ranged) - np.sqrt(herr_poly(hdo_poly(full_ranged) + mu_far)**2 + kerr_poly(kdo_poly(full_ranged) + mu_far)**2),hdo_poly(full_ranged) + mu_far + herr_poly(hdo_poly(full_ranged) + mu_far),color = 'C6',label='5 Mya - 420 pc - Dusty',linewidth=4.0)
plt.plot(hdo_poly(full_ranged)-kdo_poly(full_ranged) + Ahk + np.sqrt(herr_poly(hdo_poly(full_ranged) + mu_far + Ah)**2 + kerr_poly(kdo_poly(full_ranged) + mu_far + Ak)**2), hdo_poly(full_ranged) + mu_far + Ah - herr_poly(hdo_poly(full_ranged) + mu_far + Ah),color = 'C6',linewidth=4.0)
plt.plot(hco_poly(full_rangec)-kco_poly(full_rangec) - np.sqrt(herr_poly(hco_poly(full_rangec) + mu_far)**2 + kerr_poly(kco_poly(full_rangec) + mu_far)**2),hco_poly(full_rangec) + mu_far + herr_poly(hco_poly(full_rangec) + mu_far),color = 'C8',label='5 Mya - 420 pc - Cond',linewidth=4.0)
plt.plot(hco_poly(full_rangec)-kco_poly(full_rangec) + Ahk + np.sqrt(herr_poly(hco_poly(full_rangec) + mu_far + Ah)**2 + kerr_poly(kco_poly(full_rangec) + mu_far + Ak)**2), hco_poly(full_rangec) + mu_far + Ah - herr_poly(hco_poly(full_rangec) + mu_far + Ah),color = 'C8',linewidth=4.0)
plt.arrow(0.7,14,Ahk,Ah,width=0.01,length_includes_head=True,color='r')
plt.text(0.7,13.9,'$A_v$ = 1.1',fontsize=30)
plt.ylim(12,20)
plt.xlim(-0.25,1.5)
#plt.ylim(18.75,21)
#plt.xlim(2,2.5)
plt.legend(fontsize=32)
plt.gca().invert_yaxis()
plt.xlabel('H-Ks Mag', fontsize=40)
plt.ylabel('H Mag', fontsize=40)
plt.savefig('cmd_hk.png')
plt.show()

In [ ]:
plt.figure(figsize=(40,20))
plt.rc('xtick',labelsize=28)
plt.rc('ytick',labelsize=28)
plt.plot(mag_z-mag_j,mag_z,'.',color='grey')
#plt.plot(zd_poly(full_ranged)-jd_poly(full_ranged) - np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_near)**2 + jerr_poly(jd_poly(full_ranged) + mu_near)**2),zd_poly(full_ranged) + mu_near + zerr_poly(zd_poly(full_ranged) + mu_near),color = 'C0',label='1 Mya - 350 pc - Dusty',linewidth=4.0)
#plt.plot(zd_poly(full_ranged)-jd_poly(full_ranged) + Azj + np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_near + Az)**2 + jerr_poly(jd_poly(full_ranged) + mu_near + Aj)**2), zd_poly(full_ranged) + mu_near + Az - zerr_poly(zd_poly(full_ranged) + mu_near + Az),color = 'C0',linewidth=4.0)
#plt.plot(zc_poly(full_rangec)-jc_poly(full_rangec) - np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_near)**2 + jerr_poly(jc_poly(full_rangec) + mu_near)**2),zc_poly(full_rangec) + mu_near + zerr_poly(zc_poly(full_rangec) + mu_near),color = 'C1',label='1 Mya - 350 pc - Cond',linewidth=4.0)
#plt.plot(zc_poly(full_rangec)-jc_poly(full_rangec) + Azj + np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_near + Az)**2 + jerr_poly(jc_poly(full_rangec) + mu_near + Aj)**2) , zc_poly(full_rangec) + mu_near + Az - zerr_poly(zc_poly(full_rangec) + mu_near + Az),color = 'C1',linewidth=4.0)
#plt.plot(zd_poly(full_ranged)-jd_poly(full_ranged) - np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_far)**2 + jerr_poly(jd_poly(full_ranged) + mu_far)**2),zd_poly(full_ranged) + mu_far + zerr_poly(zd_poly(full_ranged) + mu_far),color = 'C2',label='1 Mya - 420 pc - Dusty',linewidth=4.0)
#plt.plot(zd_poly(full_ranged)-jd_poly(full_ranged) + Azj + np.sqrt(zerr_poly(zd_poly(full_ranged) + mu_far + Az)**2 + jerr_poly(jd_poly(full_ranged) + mu_far + Aj)**2), zd_poly(full_ranged) + mu_far + Az - zerr_poly(zd_poly(full_ranged) + mu_far + Az),color = 'C2',linewidth=4.0)
#plt.plot(zc_poly(full_rangec)-jc_poly(full_rangec) - np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_far)**2 + jerr_poly(jc_poly(full_rangec) + mu_far)**2),zc_poly(full_rangec) + mu_far + zerr_poly(zc_poly(full_rangec) + mu_far),color = 'C3',label='1 Mya - 420 pc - Cond',linewidth=4.0)
#plt.plot(zc_poly(full_rangec)-jc_poly(full_rangec) + Azj + np.sqrt(zerr_poly(zc_poly(full_rangec) + mu_far + Az)**2 + jerr_poly(jc_poly(full_rangec) + mu_far + Aj)**2), zc_poly(full_rangec) + mu_far + Az - zerr_poly(zc_poly(full_rangec) + mu_far + Az),color = 'C3',linewidth=4.0)
#plt.plot(zdo_poly(full_ranged)-jdo_poly(full_ranged) - np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_near)**2 + jerr_poly(jdo_poly(full_ranged) + mu_near)**2),zdo_poly(full_ranged) + mu_near + zerr_poly(zdo_poly(full_ranged) + mu_near),color = 'C4',label='5 Mya - 350 pc - Dusty',linewidth=4.0)
#plt.plot(zdo_poly(full_ranged)-jdo_poly(full_ranged) + Azj + np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_near + Az)**2 + jerr_poly(jdo_poly(full_ranged) + mu_near + Aj)**2), zdo_poly(full_ranged) + mu_near + Az - zerr_poly(zdo_poly(full_ranged) + mu_near + Az),color = 'C4',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-jco_poly(full_rangec) - np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_near)**2 + jerr_poly(jco_poly(full_rangec) + mu_near)**2),zco_poly(full_rangec) + mu_near + zerr_poly(zco_poly(full_rangec) + mu_near),color = 'C0',label='5 Mya - 350 pc - Cond w/ error',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-jco_poly(full_rangec) + Azj + np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_near + Az)**2 + jerr_poly(jco_poly(full_rangec) + mu_near + Aj)**2), zco_poly(full_rangec) + mu_near + Az - zerr_poly(zco_poly(full_rangec) + mu_near + Az),color = 'C0',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-jco_poly(full_rangec) ,zco_poly(full_rangec) + mu_near , color = 'C1',label='5 Mya - 350 pc - Cond w/o error',linewidth=4.0)
plt.plot(zco_poly(full_rangec)-jco_poly(full_rangec) + Azj , zco_poly(full_rangec) + mu_near + Az,color = 'C1',linewidth=4.0)
#plt.plot(zdo_poly(full_ranged)-jdo_poly(full_ranged) - np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_far)**2 + jerr_poly(jdo_poly(full_ranged) + mu_far)**2),zdo_poly(full_ranged) + mu_far + zerr_poly(zdo_poly(full_ranged) + mu_far),color = 'C6',label='5 Mya - 420 pc - Dusty',linewidth=4.0)
#plt.plot(zdo_poly(full_ranged)-jdo_poly(full_ranged) + Azj + np.sqrt(zerr_poly(zdo_poly(full_ranged) + mu_far + Az)**2 + jerr_poly(jdo_poly(full_ranged) + mu_far + Aj)**2), zdo_poly(full_ranged) + mu_far + Az - zerr_poly(zdo_poly(full_ranged) + mu_far + Az),color = 'C6',linewidth=4.0)
#plt.plot(zco_poly(full_rangec)-jco_poly(full_rangec) - np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_far)**2 + jerr_poly(jco_poly(full_rangec) + mu_far)**2),zco_poly(full_rangec) + mu_far + zerr_poly(zco_poly(full_rangec) + mu_far),color = 'C8',label='5 Mya - 420 pc - Cond',linewidth=4.0)
#plt.plot(zco_poly(full_rangec)-jco_poly(full_rangec) + Azj + np.sqrt(zerr_poly(zco_poly(full_rangec) + mu_far + Az)**2 + jerr_poly(jco_poly(full_rangec) + mu_far + Aj)**2), zco_poly(full_rangec) + mu_far + Az - zerr_poly(zco_poly(full_rangec) + mu_far + Az),color = 'C8',linewidth=4.0)
plt.arrow(2.1,19.5,Azj,Az,width=0.01,length_includes_head=True,color='r')
plt.text(2.1,19.4,'$A_v$ = 1.1',fontsize=30)
plt.ylim(17.5,21.5)
plt.xlim(1,3)
#plt.ylim(18.75,21)
#plt.xlim(2,2.5)
plt.legend(fontsize=32)
plt.gca().invert_yaxis()
plt.xlabel('Z-J Mag', fontsize=40)
plt.ylabel('Z Mag', fontsize=40)
plt.savefig('skinny_part_woerror.png')
plt.show()

In [ ]:
def candidate_stats(file,output_name,start_temp=1200,max_temp=4500):
    
    '''This program finds the temperatures and masses of each object in the input file, and then creates
    a table with that information. This version uses all possible set-ups. Following ones use specific 
    types of set-ups.'''

    hdul = fits.open(file)
    
    RA_ICRS = hdul[1].data['RA_ICRS']
    Dec_ICRS = hdul[1].data['Dec_ICRS']
    zmag = hdul[1].data['Mag_Z']
    ymag = hdul[1].data['Mag_Y']
    jmag = hdul[1].data['Mag_J']
    hmag = hdul[1].data['Mag_H']
    kmag = hdul[1].data['Mag_K']

    temp_min = []
    temp_mean = []
    temp_max = []
    mass_min = []
    mass_mean_y = []
    mass_mean_o = []
    mass_max = []

    for y in zmag:
        temps = []

        x = start_temp
        while (zd_poly(x) + mu_near + zerr_poly(zd_poly(x) + mu_near)) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zd_poly(x) + mu_near + Az - zerr_poly(zd_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_near + zerr_poly(zc_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_near + Az - zerr_poly(zc_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zd_poly(x) + mu_far + zerr_poly(zd_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zd_poly(x) + mu_far + Az - zerr_poly(zd_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_far + zerr_poly(zc_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_far + Az - zerr_poly(zc_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zdo_poly(x) + mu_near + zerr_poly(zdo_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zdo_poly(x) + mu_near + Az - zerr_poly(zdo_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_near + zerr_poly(zco_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_near + Az - zerr_poly(zco_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zdo_poly(x) + mu_far + zerr_poly(zdo_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zdo_poly(x) + mu_far + Az - zerr_poly(zdo_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_far + zerr_poly(zco_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_far + Az - zerr_poly(zco_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        temp_min.append(np.min(temps))
        temp_mean.append(np.mean(temps))
        temp_max.append(np.max(temps))
        if np.min([mass_poly(np.min(temps)),masso_poly(np.min(temps))]) > 0.0016:
            mass_min.append(np.min([mass_poly(np.min(temps)),masso_poly(np.min(temps))]))
        else:
            mass_min.append(np.max([mass_poly(np.min(temps)),masso_poly(np.min(temps))]))
        mass_mean_y.append(mass_poly(np.mean(temps)))
        mass_mean_o.append(masso_poly(np.mean(temps)))
        mass_max.append(np.max([mass_poly(np.max(temps)),masso_poly(np.max(temps))]))
        
    for x in range(len(temp_min)):
        if mass_min[x] > mass_mean_y[x]:
            mass_mean_y[x] = mass_min[x]
        if mass_min[x] > mass_mean_o[x]:
            mass_mean_o[x] = mass_min[x]
        if mass_max[x] < mass_mean_y[x]:
            mass_mean_y[x] = mass_max[x]
        if mass_max[x] < mass_mean_o[x]:
            mass_mean_o[x] = mass_max[x]
    
    col1 = fits.Column(name='RA_ICRS',format='D',unit='degree',array=RA_ICRS)
    col2 = fits.Column(name='Dec_ICRS',format='D',unit='degree',array=Dec_ICRS)
    col3 = fits.Column(name='Mag_Z',format='D',unit='mag',array=zmag)
    col4 = fits.Column(name='Mag_Y',format='D',unit='mag',array=ymag)
    col5 = fits.Column(name='Mag_J',format='D',unit='mag',array=jmag)
    col6 = fits.Column(name='Mag_H',format='D',unit='mag',array=hmag)
    col7 = fits.Column(name='Mag_K',format='D',unit='mag',array=kmag)
    col8 = fits.Column(name='Min_Temp',format='F',unit='K',array=temp_min)
    col9 = fits.Column(name='Mean_Temp',format='F',unit='K',array=temp_mean)
    col10 = fits.Column(name='Max_Temp',format='F',unit='K',array=temp_max)
    col11 = fits.Column(name='Min_Mass',format='F',unit='M_sol',array=mass_min)
    col12 = fits.Column(name='Mean_Mass_Young',format='F',unit='M_sol',array=mass_mean_y)
    col13 = fits.Column(name='Mean_Mass_Old',format='F',unit='M_sol',array=mass_mean_o)
    col14 = fits.Column(name='Max_Mass',format='F',unit='M_sol',array=mass_max)
    
    cols = fits.ColDefs([col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14])
    
    '''Here we make the fits file'''

    output_fits = output_name + '.fits'
    data = fits.BinTableHDU.from_columns(cols)
    data.writeto(output_fits)

In [ ]:
def candidate_stats_young(file,output_name,start_temp=1200,max_temp=4500):
    
    '''This program finds the temperatures and masses of each object in the input file, and then creates
    a table with that information. This version uses 1Myr set-ups.'''

    hdul = fits.open(file)
    
    RA_ICRS = hdul[1].data['RA_ICRS']
    Dec_ICRS = hdul[1].data['Dec_ICRS']
    zmag = hdul[1].data['Mag_Z']
    ymag = hdul[1].data['Mag_Y']
    jmag = hdul[1].data['Mag_J']
    hmag = hdul[1].data['Mag_H']
    kmag = hdul[1].data['Mag_K']

    temp_min = []
    temp_mean = []
    temp_max = []
    mass_min = []
    mass_mean = []
    mass_max = []

    for y in zmag:
        temps = []

        x = start_temp
        while (zd_poly(x) + mu_near + zerr_poly(zd_poly(x) + mu_near)) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zd_poly(x) + mu_near + Az - zerr_poly(zd_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_near + zerr_poly(zc_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_near + Az - zerr_poly(zc_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zd_poly(x) + mu_far + zerr_poly(zd_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zd_poly(x) + mu_far + Az - zerr_poly(zd_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_far + zerr_poly(zc_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_far + Az - zerr_poly(zc_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        """x = start_temp
        while zdo_poly(x) + mu_near + zerr_poly(zdo_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zdo_poly(x) + mu_near + Az - zerr_poly(zdo_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_near + zerr_poly(zco_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_near + Az - zerr_poly(zco_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zdo_poly(x) + mu_far + zerr_poly(zdo_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zdo_poly(x) + mu_far + Az - zerr_poly(zdo_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_far + zerr_poly(zco_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_far + Az - zerr_poly(zco_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)"""
        
        temps_okay = []
        
        for alpha in temps:
            if mass_poly(alpha) > 0.0:
                temps_okay.append(alpha)
                
        temp_min.append(np.min(temps_okay))
        temp_mean.append(np.mean(temps_okay))
        temp_max.append(np.max(temps_okay))
        
        mass_okay = []
        for beta in temps_okay:
            mass_okay.append(mass_poly(beta))
            
        mass_min.append(np.min(mass_okay)) 
        mass_mean.append(np.mean(mass_okay))
        mass_max.append(np.max(mass_okay))

    col1 = fits.Column(name='RA_ICRS',format='D',unit='degree',array=RA_ICRS)
    col2 = fits.Column(name='Dec_ICRS',format='D',unit='degree',array=Dec_ICRS)
    col3 = fits.Column(name='Mag_Z',format='D',unit='mag',array=zmag)
    col4 = fits.Column(name='Mag_Y',format='D',unit='mag',array=ymag)
    col5 = fits.Column(name='Mag_J',format='D',unit='mag',array=jmag)
    col6 = fits.Column(name='Mag_H',format='D',unit='mag',array=hmag)
    col7 = fits.Column(name='Mag_K',format='D',unit='mag',array=kmag)
    col8 = fits.Column(name='Min_Temp',format='F',unit='K',array=temp_min)
    col9 = fits.Column(name='Mean_Temp',format='F',unit='K',array=temp_mean)
    col10 = fits.Column(name='Max_Temp',format='F',unit='K',array=temp_max)
    col11 = fits.Column(name='Min_Mass',format='F',unit='M_sol',array=mass_min)
    col12 = fits.Column(name='Mean_Mass',format='F',unit='M_sol',array=mass_mean)
    col14 = fits.Column(name='Max_Mass',format='F',unit='M_sol',array=mass_max)
    
    cols = fits.ColDefs([col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col14])
    
    '''Here we make the fits file'''

    output_fits = output_name + '.fits'
    data = fits.BinTableHDU.from_columns(cols)
    data.writeto(output_fits)

In [ ]:
def candidate_stats_old(file,output_name,start_temp=1200,max_temp=4500):
    
    '''This program finds the temperatures and masses of each object in the input file, and then creates
    a table with that information. This version uses 5Myr set-ups.'''

    hdul = fits.open(file)
    
    RA_ICRS = hdul[1].data['RA_ICRS']
    Dec_ICRS = hdul[1].data['Dec_ICRS']
    zmag = hdul[1].data['Mag_Z']
    ymag = hdul[1].data['Mag_Y']
    jmag = hdul[1].data['Mag_J']
    hmag = hdul[1].data['Mag_H']
    kmag = hdul[1].data['Mag_K']

    temp_min = []
    temp_mean = []
    temp_max = []
    mass_min = []
    mass_mean = []
    mass_max = []

    for y in zmag:
        temps = []

        """x = start_temp
        while (zd_poly(x) + mu_near + zerr_poly(zd_poly(x) + mu_near)) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zd_poly(x) + mu_near + Az - zerr_poly(zd_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_near + zerr_poly(zc_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_near + Az - zerr_poly(zc_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zd_poly(x) + mu_far + zerr_poly(zd_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zd_poly(x) + mu_far + Az - zerr_poly(zd_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_far + zerr_poly(zc_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_far + Az - zerr_poly(zc_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)"""

        x = start_temp
        while zdo_poly(x) + mu_near + zerr_poly(zdo_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zdo_poly(x) + mu_near + Az - zerr_poly(zdo_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_near + zerr_poly(zco_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_near + Az - zerr_poly(zco_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zdo_poly(x) + mu_far + zerr_poly(zdo_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zdo_poly(x) + mu_far + Az - zerr_poly(zdo_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_far + zerr_poly(zco_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_far + Az - zerr_poly(zco_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
        
        temps_okay = []
        
        for alpha in temps:
            if masso_poly(alpha) > 0.0:
                temps_okay.append(alpha)
                
        temp_min.append(np.min(temps_okay))
        temp_mean.append(np.mean(temps_okay))
        temp_max.append(np.max(temps_okay))
        
        mass_okay = []
        for beta in temps_okay:
            mass_okay.append(masso_poly(beta))
            
        mass_min.append(np.min(mass_okay)) 
        mass_mean.append(np.mean(mass_okay))
        mass_max.append(np.max(mass_okay))

    col1 = fits.Column(name='RA_ICRS',format='D',unit='degree',array=RA_ICRS)
    col2 = fits.Column(name='Dec_ICRS',format='D',unit='degree',array=Dec_ICRS)
    col3 = fits.Column(name='Mag_Z',format='D',unit='mag',array=zmag)
    col4 = fits.Column(name='Mag_Y',format='D',unit='mag',array=ymag)
    col5 = fits.Column(name='Mag_J',format='D',unit='mag',array=jmag)
    col6 = fits.Column(name='Mag_H',format='D',unit='mag',array=hmag)
    col7 = fits.Column(name='Mag_K',format='D',unit='mag',array=kmag)
    col8 = fits.Column(name='Min_Temp',format='F',unit='K',array=temp_min)
    col9 = fits.Column(name='Mean_Temp',format='F',unit='K',array=temp_mean)
    col10 = fits.Column(name='Max_Temp',format='F',unit='K',array=temp_max)
    col11 = fits.Column(name='Min_Mass',format='F',unit='M_sol',array=mass_min)
    col12 = fits.Column(name='Mean_Mass',format='F',unit='M_sol',array=mass_mean)
    col14 = fits.Column(name='Max_Mass',format='F',unit='M_sol',array=mass_max)
    
    cols = fits.ColDefs([col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col14])
    
    '''Here we make the fits file'''

    output_fits = output_name + '.fits'
    data = fits.BinTableHDU.from_columns(cols)
    data.writeto(output_fits)

In [ ]:
def candidate_stats_dusty(file,output_name,start_temp=1500,max_temp=4500):
    
    '''This program finds the temperatures and masses of each object in the input file, and then creates
    a table with that information. This version uses Dusty set-ups.'''

    hdul = fits.open(file)
    
    RA_ICRS = hdul[1].data['RA_ICRS']
    Dec_ICRS = hdul[1].data['Dec_ICRS']
    zmag = hdul[1].data['Mag_Z']
    ymag = hdul[1].data['Mag_Y']
    jmag = hdul[1].data['Mag_J']
    hmag = hdul[1].data['Mag_H']
    kmag = hdul[1].data['Mag_K']

    temp_min = []
    temp_mean = []
    temp_max = []
    mass_min = []
    mass_mean_y = []
    mass_mean_o = []
    mass_max = []

    for y in zmag:
        temps = []

        x = start_temp
        while (zd_poly(x) + mu_near + zerr_poly(zd_poly(x) + mu_near)) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zd_poly(x) + mu_near + Az - zerr_poly(zd_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        """x = start_temp
        while zc_poly(x) + mu_near + zerr_poly(zc_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_near + Az - zerr_poly(zc_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)"""

        x = start_temp
        while zd_poly(x) + mu_far + zerr_poly(zd_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zd_poly(x) + mu_far + Az - zerr_poly(zd_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        """x = start_temp
        while zc_poly(x) + mu_far + zerr_poly(zc_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_far + Az - zerr_poly(zc_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)"""

        x = start_temp
        while zdo_poly(x) + mu_near + zerr_poly(zdo_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zdo_poly(x) + mu_near + Az - zerr_poly(zdo_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        """x = start_temp
        while zco_poly(x) + mu_near + zerr_poly(zco_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_near + Az - zerr_poly(zco_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)"""
                
        x = start_temp
        while zdo_poly(x) + mu_far + zerr_poly(zdo_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zdo_poly(x) + mu_far + Az - zerr_poly(zdo_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        """x = start_temp
        while zco_poly(x) + mu_far + zerr_poly(zco_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_far + Az - zerr_poly(zco_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)"""
                
        temp_min.append(np.min(temps))
        temp_mean.append(np.mean(temps))
        temp_max.append(np.max(temps))
        mass_min.append(np.min([mass_poly(np.min(temps)),masso_poly(np.min(temps))]))
        mass_mean_y.append(mass_poly(np.mean(temps)))
        mass_mean_o.append(masso_poly(np.mean(temps)))
        mass_max.append(np.max([mass_poly(np.max(temps)),masso_poly(np.max(temps))]))
    
    col1 = fits.Column(name='RA_ICRS',format='D',unit='degree',array=RA_ICRS)
    col2 = fits.Column(name='Dec_ICRS',format='D',unit='degree',array=Dec_ICRS)
    col3 = fits.Column(name='Mag_Z',format='D',unit='mag',array=zmag)
    col4 = fits.Column(name='Mag_Y',format='D',unit='mag',array=ymag)
    col5 = fits.Column(name='Mag_J',format='D',unit='mag',array=jmag)
    col6 = fits.Column(name='Mag_H',format='D',unit='mag',array=hmag)
    col7 = fits.Column(name='Mag_K',format='D',unit='mag',array=kmag)
    col8 = fits.Column(name='Min_Temp',format='F',unit='K',array=temp_min)
    col9 = fits.Column(name='Mean_Temp',format='F',unit='K',array=temp_mean)
    col10 = fits.Column(name='Max_Temp',format='F',unit='K',array=temp_max)
    col11 = fits.Column(name='Min_Mass',format='F',unit='M_sol',array=mass_min)
    col12 = fits.Column(name='Mean_Mass_Young',format='F',unit='M_sol',array=mass_mean_y)
    col13 = fits.Column(name='Mean_Mass_Old',format='F',unit='M_sol',array=mass_mean_o)
    col14 = fits.Column(name='Max_Mass',format='F',unit='M_sol',array=mass_max)
    
    cols = fits.ColDefs([col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14])
    
    '''Here we make the fits file'''

    output_fits = output_name + '.fits'
    data = fits.BinTableHDU.from_columns(cols)
    data.writeto(output_fits)

In [ ]:
def candidate_stats_cond(file,output_name,start_temp=900,max_temp=4500):
    
    '''This program finds the temperatures and masses of each object in the input file, and then creates
    a table with that information. This version uses Cond set-ups.'''

    hdul = fits.open(file)
    
    RA_ICRS = hdul[1].data['RA_ICRS']
    Dec_ICRS = hdul[1].data['Dec_ICRS']
    zmag = hdul[1].data['Mag_Z']
    ymag = hdul[1].data['Mag_Y']
    jmag = hdul[1].data['Mag_J']
    hmag = hdul[1].data['Mag_H']
    kmag = hdul[1].data['Mag_K']

    temp_min = []
    temp_mean = []
    temp_max = []
    mass_min = []
    mass_mean_y = []
    mass_mean_o = []
    mass_max = []

    for y in zmag:
        temps = []

        """x = start_temp
        while (zd_poly(x) + mu_near + zerr_poly(zd_poly(x) + mu_near)) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zd_poly(x) + mu_near + Az - zerr_poly(zd_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)"""

        x = start_temp
        while zc_poly(x) + mu_near + zerr_poly(zc_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_near + Az - zerr_poly(zc_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        """x = start_temp
        while zd_poly(x) + mu_far + zerr_poly(zd_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zd_poly(x) + mu_far + Az - zerr_poly(zd_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)"""

        x = start_temp
        while zc_poly(x) + mu_far + zerr_poly(zc_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zc_poly(x) + mu_far + Az - zerr_poly(zc_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        """x = start_temp
        while zdo_poly(x) + mu_near + zerr_poly(zdo_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)

        x = start_temp
        while zdo_poly(x) + mu_near + Az - zerr_poly(zdo_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)"""
                
        x = start_temp
        while zco_poly(x) + mu_near + zerr_poly(zco_poly(x) + mu_near) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_near + Az - zerr_poly(zco_poly(x) + mu_near + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        """x = start_temp
        while zdo_poly(x) + mu_far + zerr_poly(zdo_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zdo_poly(x) + mu_far + Az - zerr_poly(zdo_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)"""
                
        x = start_temp
        while zco_poly(x) + mu_far + zerr_poly(zco_poly(x) + mu_far) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
                
        x = start_temp
        while zco_poly(x) + mu_far + Az - zerr_poly(zco_poly(x) + mu_far + Az) > y and x < max_temp:
            x = x+1    
        if x != start_temp:
            if x != max_temp:
                temps.append(x)
         
        if temps != []:
            temp_min.append(np.min(temps))
            temp_mean.append(np.mean(temps))
            temp_max.append(np.max(temps))
            mass_min.append(np.min([mass_poly(np.min(temps)),masso_poly(np.min(temps))]))
            mass_mean_y.append(mass_poly(np.mean(temps)))
            mass_mean_o.append(masso_poly(np.mean(temps)))
            mass_max.append(np.max([mass_poly(np.max(temps)),masso_poly(np.max(temps))]))
    
    col1 = fits.Column(name='RA_ICRS',format='D',unit='degree',array=RA_ICRS)
    col2 = fits.Column(name='Dec_ICRS',format='D',unit='degree',array=Dec_ICRS)
    col3 = fits.Column(name='Mag_Z',format='D',unit='mag',array=zmag)
    col4 = fits.Column(name='Mag_Y',format='D',unit='mag',array=ymag)
    col5 = fits.Column(name='Mag_J',format='D',unit='mag',array=jmag)
    col6 = fits.Column(name='Mag_H',format='D',unit='mag',array=hmag)
    col7 = fits.Column(name='Mag_K',format='D',unit='mag',array=kmag)
    col8 = fits.Column(name='Min_Temp',format='F',unit='K',array=temp_min)
    col9 = fits.Column(name='Mean_Temp',format='F',unit='K',array=temp_mean)
    col10 = fits.Column(name='Max_Temp',format='F',unit='K',array=temp_max)
    col11 = fits.Column(name='Min_Mass',format='F',unit='M_sol',array=mass_min)
    col12 = fits.Column(name='Mean_Mass_Young',format='F',unit='M_sol',array=mass_mean_y)
    col13 = fits.Column(name='Mean_Mass_Old',format='F',unit='M_sol',array=mass_mean_o)
    col14 = fits.Column(name='Max_Mass',format='F',unit='M_sol',array=mass_max)
    
    cols = fits.ColDefs([col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14])
    
    '''Here we make the fits file'''

    output_fits = output_name + '.fits'
    data = fits.BinTableHDU.from_columns(cols)
    data.writeto(output_fits)

In [ ]:
candidate_stats_dusty('Final_Progress/final_working_0821.fits','temp_mass_dusty')

In [ ]:
candidate_stats_cond('Final_Progress/final_working_0821.fits','temp_mass_cond')

In [ ]:
candidate_stats('Final_Progress/final_working_0821.fits','0904_temp_mass')

In [ ]:
candidate_stats_old('Final_Progress/final_working_0821.fits','0909_temp_mass_old')

In [ ]:
candidate_stats_young('Final_Progress/final_working_0821.fits','0909_temp_mass_young')

In [ ]:
'''The next few cells were used to calculate the area and density of the field of view and a couple
other important areas. It is just a lot of simple math.'''
galaxy_x_1 = 168.842
galaxy_x_2 = 168.843
galaxy_x_3 = 167.373
galaxy_x_4 = 167.374
galaxy_y_1 = -2.051
galaxy_y_2 = -0.851
galaxy_y_3 = -0.856
galaxy_y_4 = -2.053
field_x_1 = 85.1201
field_x_2 = 82.8387
field_x_3 = 82.273
field_x_4 = 84.5136
field_y_1 = 0.0260
field_y_2 = 0.6387
field_y_3 = -1.591
field_y_4 = -2.1839
cut1_x_1 = 83.4215
cut1_x_2 = 83.5151
cut1_x_3 = 83.5831
cut1_x_4 = 83.487
cut1_y_1 = -1.8838
cut1_y_2 = -1.5260
cut1_y_3 = -1.5413
cut1_y_4 = -1.902
cut2_x_1 = 83.585
cut2_x_2 = 83.653
cut2_x_3 = 83.7503
cut2_x_4 = 83.6846
cut2_y_1 = -1.930
cut2_y_2 = -1.947
cut2_y_3 = -1.5885
cut2_y_4 = -1.5720
cut3_x_1 = 83.7976
cut3_x_2 = 83.7333
cut3_x_3 = 83.7961
cut3_x_4 = 83.8634
cut3_y_1 = -0.7389
cut3_y_2 = -0.7105
cut3_y_3 = -0.4851
cut3_y_4 = -0.4995
cut4_x_1 = 83.9645
cut4_x_2 = 84.0329
cut4_x_3 = 83.9677
cut4_x_4 = 83.8942
cut4_y_1 = -0.5265
cut4_y_2 = -0.5457
cut4_y_3 = -0.7834
cut4_y_4 = -0.7623
field2_x_1 = 82.564
field2_x_2 = field_x_3
field2_x_3 = 83.672
field2_x_4 = 83.988
field2_y_1 = -0.407
field2_y_2 = field_y_3
field2_y_3 = -1.953
field2_y_4 = -0.789
field3_x_1 = 83.656
field3_x_2 = 83.969
field3_x_3 = 84.839
field3_x_4 = field_x_4
field3_y_1 = -1.948
field3_y_2 = -0.783
field3_y_3 = -1.0164
field3_y_4 = field_y_4
field6_x_1 = 82.528
field6_x_2 = field_x_2
field6_x_3 = 84.267
field6_x_4 = 83.953
field6_y_1 = -0.524
field6_y_2 = field_y_2
field6_y_3 = 0.255
field6_y_4 = -0.911
field7_x_1 = 83.936
field7_x_2 = 84.248
field7_x_3 = field_x_1
field7_x_4 = 84.806
field7_y_1 = -0.902
field7_y_2 = 0.259
field7_y_3 = field_y_1
field7_y_4 = -1.140

In [ ]:
galaxy_area = abs(((galaxy_x_1*galaxy_y_2 - galaxy_y_1 * galaxy_x_2) + 
 (galaxy_x_2*galaxy_y_3 - galaxy_y_2 * galaxy_x_3) + 
 (galaxy_x_3*galaxy_y_4 - galaxy_y_3 * galaxy_x_4) + 
 (galaxy_x_4*galaxy_y_1 - galaxy_y_4 * galaxy_x_1))/2)
print(galaxy_area)

In [ ]:
field_area = abs(((field_x_1*field_y_2 - field_y_1 * field_x_2) + 
 (field_x_2*field_y_3 - field_y_2 * field_x_3) + 
 (field_x_3*field_y_4 - field_y_3 * field_x_4) + 
 (field_x_4*field_y_1 - field_y_4 * field_x_1))/2)
print(field_area)

In [ ]:
cut1_area = abs(((cut1_x_1*cut1_y_2 - cut1_y_1 * cut1_x_2) + 
 (cut1_x_2*cut1_y_3 - cut1_y_2 * cut1_x_3) + 
 (cut1_x_3*cut1_y_4 - cut1_y_3 * cut1_x_4) + 
 (cut1_x_4*cut1_y_1 - cut1_y_4 * cut1_x_1))/2)
print(cut1_area)

In [ ]:
cut2_area = abs(((cut2_x_1*cut2_y_2 - cut2_y_1 * cut2_x_2) + 
 (cut2_x_2*cut2_y_3 - cut2_y_2 * cut2_x_3) + 
 (cut2_x_3*cut2_y_4 - cut2_y_3 * cut2_x_4) + 
 (cut2_x_4*cut2_y_1 - cut2_y_4 * cut2_x_1))/2)
print(cut2_area)

In [ ]:
cut3_area = abs(((cut3_x_1*cut3_y_2 - cut3_y_1 * cut3_x_2) + 
 (cut3_x_2*cut3_y_3 - cut3_y_2 * cut3_x_3) + 
 (cut3_x_3*cut3_y_4 - cut3_y_3 * cut3_x_4) + 
 (cut3_x_4*cut3_y_1 - cut3_y_4 * cut3_x_1))/2)
print(cut3_area)

In [ ]:
cut4_area = abs(((cut4_x_1*cut4_y_2 - cut4_y_1 * cut4_x_2) + 
 (cut4_x_2*cut4_y_3 - cut4_y_2 * cut4_x_3) + 
 (cut4_x_3*cut4_y_4 - cut4_y_3 * cut4_x_4) + 
 (cut4_x_4*cut4_y_1 - cut4_y_4 * cut4_x_1))/2)
print(cut4_area)

In [ ]:
field2_area = abs(((field2_x_1*field2_y_2 - field2_y_1 * field2_x_2) + 
 (field2_x_2*field2_y_3 - field2_y_2 * field2_x_3) + 
 (field2_x_3*field2_y_4 - field2_y_3 * field2_x_4) + 
 (field2_x_4*field2_y_1 - field2_y_4 * field2_x_1))/2) - cut1_area - cut2_area
print(field2_area)

In [ ]:
field3_area = abs(((field3_x_1*field3_y_2 - field3_y_1 * field3_x_2) + 
 (field3_x_2*field3_y_3 - field3_y_2 * field3_x_3) + 
 (field3_x_3*field3_y_4 - field3_y_3 * field3_x_4) + 
 (field3_x_4*field3_y_1 - field3_y_4 * field3_x_1))/2)
print(field3_area)

In [ ]:
field6_area = abs(((field6_x_1*field6_y_2 - field6_y_1 * field6_x_2) + 
 (field6_x_2*field6_y_3 - field6_y_2 * field6_x_3) + 
 (field6_x_3*field6_y_4 - field6_y_3 * field6_x_4) + 
 (field6_x_4*field6_y_1 - field6_y_4 * field6_x_1))/2) - cut3_area - cut4_area
print(field6_area)

In [ ]:
field7_area = abs(((field7_x_1*field7_y_2 - field7_y_1 * field7_x_2) + 
 (field7_x_2*field7_y_3 - field7_y_2 * field7_x_3) + 
 (field7_x_3*field7_y_4 - field7_y_3 * field7_x_4) + 
 (field7_x_4*field7_y_1 - field7_y_4 * field7_x_1))/2)
print(field7_area)

In [ ]:
final_field_area = field_area - cut1_area - cut2_area - cut3_area - cut4_area 

In [ ]:
print(galaxy_area)
print(final_field_area)

In [ ]:
'''I start calculating densities with the obtained areas.'''

ipmo = 114
pos_ipmo = 71
bd = 89
pos_bd = 520
yso = 154
likely_ipmo = 160
likely_bd = 274
likely_yso = 514

In [ ]:
likely_yso/final_field_area

In [ ]:
final_field_area_sr = final_field_area/3283
print(final_field_area_sr)
r2 = 75
r1 = 25
V1 = (final_field_area_sr/3)*(r1**3)
print(V1)
V2 = (final_field_area_sr/3)*(r2**3)
print(V2)
V_T = (final_field_area_sr/3)*(r2**3 - r1**3)
print(V_T)
cont_min = 2*10**(-3)
cont_max = 8*10**(-3)
print(V_T*cont_min)
print(V_T*cont_max)

In [ ]:
galaxy_density = 3/galaxy_area

In [ ]:
galaxy_density

In [ ]:
galaxy_density * final_field_area

In [ ]:
galaxy_density

In [ ]:
caballero_size = 1.5*1.5
print(caballero_size)

In [ ]:
cab_final_size = caballero_size*1.75

In [ ]:
cab_galaxy_density = 11/cab_final_size

In [ ]:
cab_galaxy_density * final_field_area

In [ ]:
final_field_area

In [ ]:
total = 948
ipmo = 112
poss_ipmo = 62
bd = 98
poss_bd = 490
yso = 186
total_2 = 383
ipmo_2 = 27
poss_ipmo_2 = 30
bd_2 = 39
poss_bd_2 = 209
yso_2 = 78
total_3 = 255
ipmo_3 = 23
poss_ipmo_3 = 13
bd_3 = 27
poss_bd_3 = 139
yso_3 = 53
total_6 = 260
ipmo_6 = 42
poss_ipmo_6 = 16
bd_6 = 38
poss_bd_6 = 118
yso_6 = 46
total_7 = 181
ipmo_7 = 34
poss_ipmo_7 = 11
bd_7 = 19
poss_bd_7 = 83
yso_7 = 34

In [ ]:
total_total_density = total/final_field_area
ipmo_total_density = (ipmo-2)/final_field_area
possipmo_total_density = poss_ipmo/final_field_area
bd_total_density = bd/final_field_area
possbd_total_density = poss_bd/final_field_area
yso_total_density = yso/final_field_area
comb_ipmo_total_density = (ipmo + poss_ipmo)/final_field_area

total_2_density = total_2/field2_area
ipmo_2_density = ipmo_2/field2_area
possipmo_2_density = poss_ipmo_2/field2_area
bd_2_density = bd_2/field2_area
possbd_2_density = poss_bd_2/field2_area
yso_2_density = yso_2/field2_area
comb_ipmo_2_density = (ipmo_2 + poss_ipmo_2)/field2_area

total_3_density = total_3/field3_area
ipmo_3_density = ipmo_3/field3_area
possipmo_3_density = poss_ipmo_3/field3_area
bd_3_density = bd_3/field3_area
possbd_3_density = poss_bd_3/field3_area
yso_3_density = yso_3/field3_area
comb_ipmo_3_density = (ipmo_3 + poss_ipmo_3)/field3_area

total_6_density = total_6/field6_area
ipmo_6_density = ipmo_6/field6_area
possipmo_6_density = poss_ipmo_6/field6_area
bd_6_density = bd_6/field6_area
possbd_6_density = poss_bd_6/field6_area
yso_6_density = yso_6/field6_area
comb_ipmo_6_density = (ipmo_6 + poss_ipmo_6)/field6_area

total_7_density = total_7/field7_area
ipmo_7_density = ipmo_7/field7_area
possipmo_7_density = poss_ipmo_7/field7_area
bd_7_density = bd_7/field7_area
possbd_7_density = poss_bd_7/field7_area
yso_7_density = yso_7/field7_area
comb_ipmo_7_density = (ipmo_7 + poss_ipmo_7)/field7_area

peak_density = ipmo_total_density
ipmo_density = comb_ipmo_total_density
bd_density = possipmo_total_density + bd_total_density + possbd_total_density
yso_density = possbd_total_density + yso_total_density

In [ ]:
print(total_total_density)
print(peak_density)
print(ipmo_density)
print(bd_density)
print(yso_density)
print(ipmo-2)
print(ipmo + poss_ipmo)
print(poss_ipmo + bd + poss_bd)
print(poss_bd + yso)
print(total)

In [ ]:
print(total_total_density)
print(total_2_density)
print(total_3_density)
print(total_6_density)
print(total_7_density)

In [ ]:
print(ipmo_total_density)
print(ipmo_2_density)
print(ipmo_3_density)
print(ipmo_6_density)
print(ipmo_7_density)

In [ ]:
print(possipmo_total_density)
print(possipmo_2_density)
print(possipmo_3_density)
print(possipmo_6_density)
print(possipmo_7_density)

In [ ]:
print(bd_total_density)
print(bd_2_density)
print(bd_3_density)
print(bd_6_density)
print(bd_7_density)

In [ ]:
print(possbd_total_density)
print(possbd_2_density)
print(possbd_3_density)
print(possbd_6_density)
print(possbd_7_density)

In [ ]:
print(yso_total_density)
print(yso_2_density)
print(yso_3_density)
print(yso_6_density)
print(yso_7_density)

In [ ]:
print(ipmo_total_density + possipmo_total_density)
print(ipmo_2_density + possipmo_2_density)
print(ipmo_3_density + possipmo_3_density)
print(ipmo_6_density + possipmo_6_density)
print(ipmo_7_density + possipmo_7_density)

In [ ]:
def vosa(file,output_name):
    
    '''This program takes a fits file and makes it into a form that can then be saved as a cvs file,
    which is then ready to be used on the VOSA website, to make it into their format.'''

    hdul = fits.open(file)
    
    Ob_ID = hdul[1].data['Object_ID']
    RA_ICRS = hdul[1].data['RA_ICRS']
    Dec_ICRS = hdul[1].data['Dec_ICRS']
    zmag = hdul[1].data['Mag_Z']
    ymag = hdul[1].data['Mag_Y']
    jmag = hdul[1].data['Mag_J']
    hmag = hdul[1].data['Mag_H']
    kmag = hdul[1].data['Mag_K']
    av = hdul[1].data['Av']
    distance = hdul[1].data['Distance']
    
    zmag_err = zerr_poly(zmag)
    ymag_err = yerr_poly(ymag)
    jmag_err = jerr_poly(jmag)
    hmag_err = herr_poly(hmag)
    kmag_err = kerr_poly(kmag)
    
    col1 = fits.Column(name='RA_ICRS',format='D',unit='degree',array=RA_ICRS)
    col2 = fits.Column(name='Dec_ICRS',format='D',unit='degree',array=Dec_ICRS)
    col3 = fits.Column(name='Mag_Z',format='D',unit='mag',array=zmag)
    col3a = fits.Column(name='Mag_Z_Error',format='D',unit='mag',array=zmag_err)
    col4 = fits.Column(name='Mag_Y',format='D',unit='mag',array=ymag)
    col4a = fits.Column(name='Mag_Y_Error',format='D',unit='mag',array=ymag_err)
    col5 = fits.Column(name='Mag_J',format='D',unit='mag',array=jmag)
    col5a = fits.Column(name='Mag_J_Error',format='D',unit='mag',array=jmag_err)
    col6 = fits.Column(name='Mag_H',format='D',unit='mag',array=hmag)
    col6a = fits.Column(name='Mag_H_Error',format='D',unit='mag',array=hmag_err)
    col7 = fits.Column(name='Mag_K',format='D',unit='mag',array=kmag)
    col7a = fits.Column(name='Mag_K_Error',format='D',unit='mag',array=kmag_err)
    col8 = fits.Column(name='Av',format='10A',array=av)
    col9 = fits.Column(name='Distance',format='I',array=distance)
    col10 = fits.Column(name='Object_ID',format='I',array=Ob_ID)
    
    cols = fits.ColDefs([col10,col1,col2,col3,col3a,col4,col4a,col5,col5a,col6,col6a,col7,col7a,col8,col9])
    
    '''Here we make the fits file'''

    output_fits = output_name + '.fits'
    data = fits.BinTableHDU.from_columns(cols)
    data.writeto(output_fits)

In [ ]:
vosa('vosa_ready_full.fits','vosa_ready_full_01')

In [ ]:
m_sol = 1.98847*10**30
m_jup = 1.89813*10**27

In [ ]:
def make_table(file,output_name):
    
    '''This takes a fits file and turns it into one that can be used to easily make Latex tables.'''

    hdul = fits.open(file)
    
    Ob_ID = hdul[1].data['Object_ID']
    RA_ICRS = hdul[1].data['RA_dms']
    Dec_ICRS = hdul[1].data['Dec_dms']
    zmag = hdul[1].data['Mag_Z']
    ymag = hdul[1].data['Mag_Y']
    jmag = hdul[1].data['Mag_J']
    hmag = hdul[1].data['Mag_H']
    kmag = hdul[1].data['Mag_K']
    filters = hdul[1].data['Filters']
    counts = hdul[1].data['Total Counts']
    perc = hdul[1].data['Percentage Count']
    temp_min = hdul[1].data['Min_Temp']
    temp_mean = hdul[1].data['Mean_Temp']
    temp_max = hdul[1].data['Max_Temp']
    mass_min = hdul[1].data['Min_Mass']
    mass_mean = hdul[1].data['Mean_Mass']
    mass_max = hdul[1].data['Max_Mass']
    
    allpoints = list(range(len(hdul[1].data)))
    
    Dec_fixed = [0 for z in allpoints]
    
    zmag = np.around(zmag,decimals=2)
    ymag = np.around(ymag,decimals=2)
    jmag = np.around(jmag,decimals=2)
    hmag = np.around(hmag,decimals=2)
    kmag = np.around(kmag,decimals=2)
    perc = np.around(perc,decimals=3)
    mass_min = np.around(mass_min,decimals=5)
    mass_mean = np.around(mass_mean,decimals=5)
    mass_max = np.around(mass_max,decimals=5)
    
    for x in range(len(allpoints)):
        Dec_fixed[x] = Dec_ICRS[x][0]+Dec_ICRS[x][2]+'$^{\circ}$ '+Dec_ICRS[x][4:6]+'\' '+Dec_ICRS[x][7:11]+'\" '
    
    
    col0 = fits.Column(name='Object_ID',format='I',array=Ob_ID)
    col1 = fits.Column(name='RA_ICRS',format='11A',unit='degree',array=RA_ICRS)
    col2 = fits.Column(name='Dec_ICRS',format='132A',unit='degree',array=Dec_fixed)
    col3 = fits.Column(name='Mag_Z',format='D2',unit='mag',array=zmag)
    col4 = fits.Column(name='Mag_Y',format='D2',unit='mag',array=ymag)
    col5 = fits.Column(name='Mag_J',format='D2',unit='mag',array=jmag)
    col6 = fits.Column(name='Mag_H',format='D2',unit='mag',array=hmag)
    col7 = fits.Column(name='Mag_K',format='D2',unit='mag',array=kmag)
    col8 = fits.Column(name='Filters',format='I',unit='Counts',array=filters)
    col9 = fits.Column(name='Counts',format='I',unit='Counts',array=counts)
    col10 = fits.Column(name='Perc',format='D2',unit='Percentage',array=perc)
    col11 = fits.Column(name='Min_Temp',format='I',unit='K',array=temp_min)
    col12 = fits.Column(name='Mean_Temp',format='I',unit='K',array=temp_mean)
    col13 = fits.Column(name='Max_Temp',format='I',unit='K',array=temp_max)
    col14 = fits.Column(name='Min_Mass',format='E5',unit='M_sol',array=mass_min)
    col15 = fits.Column(name='Mean_Mass',format='E5',unit='M_sol',array=mass_mean)
    col17 = fits.Column(name='Max_Mass',format='E5',unit='M_sol',array=mass_max)
    
    cols = fits.ColDefs([col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,
                         col10,col11,col12,col13,col14,col15,col17])
    
    '''Here we make the fits file'''

    output_fits = output_name + '.fits'
    data = fits.BinTableHDU.from_columns(cols)
    data.writeto(output_fits)

In [ ]:
make_table('Final_Progress/final_old.fits','final_table_old')
make_table('Final_Progress/final_young.fits','final_table_young')

In [ ]:
'''The next few cells are used to make the density plots.'''
hdul = fits.open('Final_Progress/final.fits')

In [ ]:
x_cand = hdul[1].data['RA_ICRS']
y_cand = hdul[1].data['Dec_ICRS']
x_small = hdul[1].data['RA_ICRS'][0:104]
y_small = hdul[1].data['Dec_ICRS'][0:104]
x_ipmo = hdul[1].data['RA_ICRS'][0:175]
y_ipmo = hdul[1].data['Dec_ICRS'][0:175]
x_bd = hdul[1].data['RA_ICRS'][115:763]
y_bd = hdul[1].data['Dec_ICRS'][115:763]
x_yso = hdul[1].data['RA_ICRS'][274:-1]
y_yso = hdul[1].data['Dec_ICRS'][274:-1]

x_small_or = x_small
y_small_or = y_small

In [ ]:
densities_x = [83.20233047,83.95468407,83.50452033,84.12083681]
densities_y = [-1.64303844,-1.56590665,-0.72506968,-0.39312178]
n = ['D','A', 'C','B']

r = .67/2
circ_area=np.pi * r**2

In [ ]:
xy = np.vstack([x_cand,y_cand])
z = gaussian_kde(xy)(xy)

idx = z.argsort()
x, y, z = x_cand[idx], y_cand[idx], z[idx]

fig, ax = plt.subplots()
fig.set_size_inches(15,9)
ax.set_facecolor((64/255, 3/255, 80/255))
im = ax.scatter(x, y, c=z, s=10000, edgecolor='')
v = np.linspace(-.1, 2.0, 100, endpoint=True)
cb = plt.colorbar(im,fraction=0.05)
cb.ax.set_yticklabels([int(0.05*(total_total_density/circ_area)),int(0.10*(total_total_density/circ_area)),
                       int(0.15*(total_total_density/circ_area)),int(0.20*(total_total_density/circ_area)),
                       int(0.25*(total_total_density/circ_area)),int(0.30*(total_total_density/circ_area))],size=16)
cb.set_label('Surface Density per Square Degree',size=20)
#ax.scatter(x, y, color='k',s=1)
ax.scatter(84.05224370308734,-1.1913836528004638, color='r', marker ='*', s=120)
ax.scatter(densities_x,densities_y, color='b', marker = 'D', s=24)
#for i, txt in enumerate(n):
#    ax.annotate(txt, (densities_x[i], densities_y[i]))
plt.title('Candidate Density',fontsize=36)
plt.xlabel('RA', fontsize=28)
plt.ylabel('Dec', fontsize=28)
plt.gca().invert_xaxis()
plt.savefig('cand_density_plot_04')

In [ ]:
xy = np.vstack([x_small,y_small])
z = gaussian_kde(xy)(xy)

idx = z.argsort()
x, y, z = x_small[idx], y_small[idx], z[idx]

fig, ax = plt.subplots()
fig.set_size_inches(15,9)
ax.set_facecolor((64/255, 3/255, 80/255))
im = ax.scatter(x, y, c=z, s=10000, edgecolor='')
cb = plt.colorbar(im,fraction=0.046)
cb.set_label('Surface Density per Square Degree',size=20)
cb.ax.set_yticklabels([int(0.05*(ipmo_total_density/circ_area)),int(0.10*(ipmo_total_density/circ_area)),
                       int(0.15*(ipmo_total_density/circ_area)),int(0.20*(ipmo_total_density/circ_area)),
                       int(0.25*(ipmo_total_density/circ_area)),int(0.30*(ipmo_total_density/circ_area))],size=16)
#ax.scatter(x_small_or, y_small_or, color='k',s=1)
ax.scatter(84.05224370308734,-1.1913836528004638, color='r', marker ='*', s=120)
plt.title('Peak Candidate Density',fontsize=36)
plt.xlabel('RA', fontsize=28)
plt.ylabel('Dec', fontsize=28)
plt.gca().invert_xaxis()
plt.savefig('smallest_density_plot_04')

In [ ]:
xy = np.vstack([x_ipmo,y_ipmo])
z = gaussian_kde(xy)(xy)

idx = z.argsort()
x, y, z = x_ipmo[idx], y_ipmo[idx], z[idx]

fig, ax = plt.subplots()
fig.set_size_inches(15,9)
ax.set_facecolor((64/255, 3/255, 80/255))
im = ax.scatter(x, y, c=z, s=10000, edgecolor='')
cb = plt.colorbar(im,fraction=0.046)
cb.set_label('Surface Density per Square Degree',size=20)
cb.ax.set_yticklabels([int(0.05*(comb_ipmo_total_density/circ_area)),int(0.10*(comb_ipmo_total_density/circ_area)),
                       int(0.15*(comb_ipmo_total_density/circ_area)),int(0.20*(comb_ipmo_total_density/circ_area)),
                       int(0.25*(comb_ipmo_total_density/circ_area)),int(0.30*(comb_ipmo_total_density/circ_area))],size=16)
#ax.scatter(x_ipmo_or, y_ipmo_or, color='k',s=1)
ax.scatter(84.05224370308734,-1.1913836528004638, color='r', marker ='*', s=120)
plt.title('Candidate iPMO Density',fontsize=36)
plt.xlabel('RA', fontsize=28)
plt.ylabel('Dec', fontsize=28)
plt.gca().invert_xaxis()
plt.savefig('ipmo_density_plot_01')

In [ ]:
xy = np.vstack([x_bd,y_bd])
z = gaussian_kde(xy)(xy)

idx = z.argsort()
x, y, z = x_bd[idx], y_bd[idx], z[idx]

fig, ax = plt.subplots()
fig.set_size_inches(15,9)
ax.set_facecolor((64/255, 3/255, 80/255))
im = ax.scatter(x, y, c=z, s=10000, edgecolor='')
cb = plt.colorbar(im,fraction=0.046)
cb.set_label('Surface Density per Square Degree',size=20)
cb.ax.set_yticklabels([int(0.05*(bd_density/circ_area)),int(0.10*(bd_density/circ_area)),
                       int(0.15*(bd_density/circ_area)),int(0.20*(bd_density/circ_area)),
                       int(0.25*(bd_density/circ_area)),int(0.30*(bd_density/circ_area))],size=16)
#ax.scatter(x, y, color='k',s=1)
ax.scatter(84.05224370308734,-1.1913836528004638, color='r', marker ='*', s=120)
plt.title('BD Candidate Density',fontsize=36)
plt.xlabel('RA', fontsize=28)
plt.ylabel('Dec', fontsize=28)
plt.gca().invert_xaxis()
plt.savefig('bd_density_plot_01')

In [ ]:
xy = np.vstack([x_yso,y_yso])
z = gaussian_kde(xy)(xy)

idx = z.argsort()
x, y, z = x_yso[idx], y_yso[idx], z[idx]
norm = cm.colors.Normalize(vmax=abs(z).max(), vmin=-abs(z).max())
fig, ax = plt.subplots()
fig.set_size_inches(15,9)
ax.set_facecolor((64/255, 3/255, 80/255))
im = ax.scatter(x, y, c=z, s=10000, edgecolor='')
cb = plt.colorbar(im,fraction=0.046)
cb.set_label('Surface Density per Square Degree',size=20)
cb.ax.set_yticklabels([int(0.05*(yso_density/circ_area)),int(0.10*(yso_density/circ_area)),
                       int(0.15*(yso_density/circ_area)),int(0.20*(yso_density/circ_area)),
                       int(0.25*(yso_density/circ_area)),int(0.30*(yso_density/circ_area))],size=16)
#ax.scatter(x, y, color='k',s=1)
ax.scatter(84.05224370308734,-1.1913836528004638, color='r', marker ='*', s=120)
plt.title('YSO Candidate Density',fontsize=36)
plt.xlabel('RA', fontsize=28)
plt.ylabel('Dec', fontsize=28)
plt.gca().invert_xaxis()
plt.savefig('yso_density_plot_01')

In [1]:
def coord_convert(file,output_name):
    info = fits.open(file)
    tbl = info[1].data
    
    RA_ICRS = []
    Dec_ICRS = []
    
    for x in range(len(tbl)):
        ra_h = int(str(tbl[x][1])[2:3])
        ra_m = int(str(tbl[x][1])[3:5])
        ra_s = float(str(tbl[x][1])[5:9])/100
        dec_sign = np.sign(float(str(tbl[x][1])[9:17]))
        dec_h = int(str(tbl[x][1])[10:12])
        dec_m = int(str(tbl[x][1])[12:14])
        dec_s = float(str(tbl[x][1])[14:17])/10
        
        ra = ra_h*15 + ra_m*15/60 + ra_s*15/3600
        dec = dec_sign*(dec_h + dec_m/60 + dec_s/3600)
        
        RA_ICRS.append(ra)
        Dec_ICRS.append(dec)
        
    col3a = fits.Column(name='RA_ICRS',format='D',unit='deg',array=np.array(RA_ICRS))
    col4a = fits.Column(name='Dec_ICRS',format='D',unit='deg',array=np.array(Dec_ICRS))
    
    cols = fits.ColDefs([col3a,col4a])
    
    output_fits = output_name + '.fits'
    
    data = fits.BinTableHDU.from_columns(cols)
    data.writeto(output_fits)

In [ ]:
coord_convert('J_A+A_485_931_tablea15.dat.fits','caballero15')